In [ ]:
import pandas as pd
import glob
from ast import literal_eval
#import pycircos
from Bio import SeqIO
import matplotlib.pyplot as plt

In [ ]:
table = pd.read_table("all_inserts_explore_cleaned.csv", comment="#",header=None, converters={})
table.columns = ['prime', 'ins_len', 'points', 'path', 'path_distro']
table

In [ ]:
#turn it into list
table.points = table.points.apply(literal_eval)
table.path = table.path.apply(literal_eval)
table.path_distro = table.path_distro.apply(literal_eval)

In [ ]:
ins_fams = {}  # gene fam names along path, only insert
ins_distros = {} #gene distributions along path, only contains insert
nearest_nayb = {} #same as previous +- 1
anchor = {} #the anchor (core gene) for each insert
for indy, row in table.iterrows():
    
    #we want +- 1 gene from ins in distros?
    line_dict1 = row["path"]
    ins_fams[row["prime"]] = line_dict1[row["points"][0]:row["points"][1]]
    ins_distros[row['prime']] = row['path_distro'][row['points'][0]:row["points"][1]]
    nearest_nayb[row['prime']] = row['path_distro'][row['points'][0]-1:row["points"][1]+1]
    anchor[row['prime']] = row['path'][row['points'][0]-1]

In [ ]:
ins_fams

In [ ]:
#see if there are overlaps
from itertools import combinations
for k1, k2 in combinations(ins_fams, 2):
    s = bool(set(ins_fams[k1]) & set(ins_fams[k2]))
    if s == True:
        print("{0} \t {2} \t  {1} \t {3}".format(k1, k2, len(ins_fams[k1]), len(ins_fams[k2])), file=open("insert_overlaps.tab",'a'))

In [ ]:
windows_format = {}
for f, items in ins_fams.items():
    windows_format["_".join(f.split("|"))] = items

In [ ]:
ins_fams

In [ ]:
#puts everything in the right folder
for thangs, fams in ins_fams.items():
    base = "_".join(thangs.split("|"))
    for i,j in enumerate(fams):
        if ins_distros[thangs][i] > 1:
            shutil.copy(j+".aln.fas", base +".ins/")
            print(base+".ins/")
        if ins_distros[thangs][i] == 1:
            shutil.copy(j+".fasta", base +".ins/")

## Regionality of INS

In [ ]:
insert_length = {}
for insert, phams in ins_fams.items():
    insert_length[insert] = len(phams)
insert_length

In [ ]:
cd /media/tian/Backup_Protected/working/halo_pg/latest_intersect_cog/ffn/20201109_isolates/fasta_fnas/virus/PROKKA_10292021/clusts/hmmers/hmmerout/hits/cripsr/panaroo/reannotate/single_gene/gffs/results/aligned_ins/nadK_0.ins

In [ ]:
locus_regions = {"HGOBCNMM":"Baja","FBLKDDDN":"Spain","GGMENCCH":"Spain","KNKCDEML":"Spain","GJADGJJH":"Spain","NBEGIDKM":"Spain","MAEDJJLC":"Spain","PPAMLKML":"Spain","ANLDKFMB":"Spain","ALMPNPLM":"Spain","ILDLLNBB":"Spain","MDOHONPH":"Spain","ICFJOHJH":"Spain","CBGFCALD":"Spain","LBEKBKDJ":"Spain","MJNIIBMG":"Spain","BAIOBGAL":"Spain","OEMCECNB":"Spain","KALPBOHI":"Spain","HIKCDMLK":"Iran","CNFKGNAC":"Iran","AFDIGNOC":"Iran","ANBHONDD":"Iran","NDEKODAF":"Iran","DPBEEGGP":"Mongolia","DHOHKKHL":"Mongolia","ADHCADEO":"Iran","MHIPNNDE":"SanFran","AMNKGELI":"SanFran","PFJGCJHP":"Namibia","CHLMDKPE":"Israel","NDDDIKEP":"Israel","ONGGBCKC":"Israel","PMNODAEP":"Israel","BOLLBKIO":"Israel","GGAGLIPG":"Israel","IHJLPFIO":"Israel","PNCEMHLO":"Israel","HIIAAINO":"Israel","MGKDKBAB":"Israel","MLIKFGOG":"Israel","LOFCOCJN":"Israel","LBMBHCAM":"Israel","PFAGPCGD":"Israel","KLCBHBCF":"Israel","OFBONHDF":"Israel","PBGIHMLN":"Israel"}

In [ ]:
genome_regions = {"12-10-3-MGM":"Baja","AMM011":"Spain","AMM013":"Spain","AMM015":"Spain","AMM016":"Spain","AMM018":"Spain","AMM027":"Spain","AMM028":"Spain","AMM032":"Spain","AMM034":"Spain","AMM036":"Spain","AMM037":"Spain","AMM044":"Spain","AMM045":"Spain","AMM059":"Spain","AMM063":"Spain","AMM081":"Spain","AMM088":"Spain","AMM093":"Spain","C191":"Iran","Ec15":"Iran","fb21":"Iran","G37":"Iran","Ga36":"Iran","Hxluteum":"Mongolia","Hxorientale":"Mongolia","LD3":"Iran","M4-25-10-8A":"SanFran","M5-25-10-8B":"SanFran","SD690R":"Namibia","UG_101":"Israel","UG_110":"Israel","UG_113":"Israel","UG_114":"Israel","UG_115":"Israel","UG_123":"Israel","UG_125":"Israel","UG_126":"Israel","UG_23":"Israel","UG_37":"Israel","UG_39":"Israel","UG_3":"Israel","UG_66":"Israel","UG_68":"Israel","UG_82":"Israel","UG_86":"Israel","UG_96":"Israel"}

In [ ]:
len(genome_regions)

In [ ]:
mean_ins_phams_distro = {}
for insert, phams in ins_distros.items():
    mean_ins_phams_distro[insert] = mean(phams)

In [ ]:
clean_ins =[]
dirty_ins = []

for insert, phams in mean_ins_phams_distro.items():
    if type(phams) == int:
        clean_ins.append("_".join(insert.split("|")))
    else:
        dirty_ins.append("_".join(insert.split("|")))

In [ ]:
regions = {}
for align in glob.glob("*.fas"):
    locations  =[]
    with open(align,'r') as ofh:
        for line in ofh:
            line = line.strip()
            if line.startswith(">"):
                locations.append(genome_regions[line.split(";")[0][1:]])
    regions[align[:-8]] = locations
            

In [ ]:
regions

In [ ]:
import Bio
from Bio.SeqUtils import GC
from statistics import mean
from statistics import stdev
from Bio import AlignIO
import dendropy
from dendropy.calculate import popgenstat

In [ ]:
pooled_fams = []
for keys, vals in ins_fams.items():
    for f in vals:
        pooled_fams.append(f)
        

In [ ]:
bool(set(core) & set(pooled))

In [ ]:
core = []
with open("core_alignment_header.embl", 'r') as ofh:
    for line in ofh:
        line = line.strip()
        if 'label' in line:
            core.append(line.split("=")[1])

In [ ]:
pooled = set(pooled_fams)

In [ ]:
inverted = {}
for k, v in ins_fams.items():
    for z in v:
        inverted[z] = k


In [ ]:
len(inverted)

In [ ]:
Hotelfor f in glob.glob("*.fas"):
    gc = []
    seq_len = []
    if f[:-4] in core:
        for record in SeqIO.parse(f, "fasta"):
            gc.append(GC(record.seq))
            seq_len.append(len(record.seq))
        seqs = dendropy.DnaCharacterMatrix.get(path=f,schema='fasta')
        print("{} \t {} \t {} \t {}".format(f, mean(gc), mean(seq_len), popgenstat.average_number_of_pairwise_differences(seqs)),
             file=open("core_stats_roo.tab",'a'))
        
    
    if f[:-8] in pooled:
        for record in SeqIO.parse(f, "fasta"):
            gc.append(GC(record.seq))
            seq_len.append(len(record.seq))
        seqs = dendropy.DnaCharacterMatrix.get(path=f,schema='fasta')
        print("{} \t {} \t{} \t {} \t {}".format(f, inverted[f[:-8]], mean(gc), mean(seq_len), popgenstat.average_number_of_pairwise_differences(seqs)),
             file=open("insert_stats_roo.tab",'a'))

In [ ]:
gc_core = []
len_core = []
normal_pair_diffs = []
with open("core_stats_roo.tab",'r') as ofh:
    for line in ofh:
        line = line.strip()
        gc_core.append(float(line.split("\t")[1]))
        len_core.append(int(line.split("\t")[2]))
        normal_pair_diffs.append(float(float(line.split("\t")[3])/float(line.split("\t")[2])))
        
print("{} \t {} \t {}".format(mean(gc_core), mean(len_core), mean(normal_pair_diffs)))

In [ ]:
gc_ins = []
len_ins = []
ins_normal_pair_diffs = []
with open("insert_stats_roo.tab",'r') as ofh:
    for line in ofh:
        line = line.strip()
        gc_ins.append(float(line.split("\t")[2]))
        len_ins.append(int(line.split("\t")[3]))
        ins_normal_pair_diffs.append(float(float(line.split("\t")[4])/float(line.split("\t")[3])))
        
print("{} \t {} \t {}".format(mean(gc_ins), mean(len_ins), mean(ins_normal_pair_diffs)))

#gc and diffs significant

In [ ]:
plt.hist()

In [ ]:
from scipy.stats import ks_2samp
ks_2samp(len_core, len_ins)

In [ ]:
for f in glob.glob("*.ffn"):    
    with open(f,'r+') as ofh:
        for line in ofh:
            line = line.strip()
            if "fb21" in line:
                print(line + "\n"+ next(ofh), file=open('anchors_blast.fasta','a'))

In [ ]:
for key in nearest_nayb:
    print(nearest_nayb[key][0])
#6345, 6554, 6160,

## Intergenic distance

In [ ]:
all_ins = set(pooled_fams)
all_ins

In [ ]:
with open("gene_presence_absence.csv") as ofh:
    for line in ofh:
        if line.split(",")[0] in all_ins:
            print(line.strip(), file=open("insert_only_presence_absen.csv",'a'))
            
        else:
            continue

In [ ]:
all_records = ["HGOBCNMM_02661","FBLKDDDN_02539","GGMENCCH_02677","KNKCDEML_02291","GJADGJJH_02677","NBEGIDKM_03008","MAEDJJLC_02576","PPAMLKML_03501"
,"ANLDKFMB_03265","ALMPNPLM_00997","ILDLLNBB_03182","MDOHONPH_03155","ICFJOHJH_03281","CBGFCALD_02079","LBEKBKDJ_02681","MJNIIBMG_02904"
,"BAIOBGAL_03257","OEMCECNB_02986","KALPBOHI_01422","HIKCDMLK_03127","AFDIGNOC_00351","DHOHKKHL_03368","ADHCADEO_02044","MHIPNNDE_00758"
,"AMNKGELI_00619","CHLMDKPE_01052","NDDDIKEP_01854","ONGGBCKC_01321","PMNODAEP_02487","BOLLBKIO_01828","GGAGLIPG_02103","IHJLPFIO_01816"
,"PNCEMHLO_02282","MGKDKBAB_02147","LOFCOCJN_02147","PFAGPCGD_02122","KLCBHBCF_02059","OFBONHDF_01847","PBGIHMLN_03555","HGOBCNMM_01592"
,"FBLKDDDN_02016","GGMENCCH_00946","KNKCDEML_00370","GJADGJJH_00370","NBEGIDKM_02175","MAEDJJLC_02154","PPAMLKML_02011","ICFJOHJH_00374"
,"CBGFCALD_00370","LBEKBKDJ_02557","MJNIIBMG_00370","BAIOBGAL_01880","OEMCECNB_00875","KALPBOHI_02407","HIKCDMLK_03575","CNFKGNAC_02177"
,"AFDIGNOC_01411","ANBHONDD_01184","NDEKODAF_01652","DPBEEGGP_01444","DHOHKKHL_02866","ADHCADEO_03117","MHIPNNDE_01828","AMNKGELI_00193"
,"CHLMDKPE_01725","NDDDIKEP_00303","ONGGBCKC_00401","PMNODAEP_00729","BOLLBKIO_00429","GGAGLIPG_00372","IHJLPFIO_00429","PNCEMHLO_00349"
,"MGKDKBAB_00729","LOFCOCJN_00729","PFAGPCGD_00314","KLCBHBCF_00429","OFBONHDF_00429","PBGIHMLN_00227","HGOBCNMM_02908","FBLKDDDN_02765","GGMENCCH_00866","KNKCDEML_01145","GJADGJJH_00450","NBEGIDKM_02373","MAEDJJLC_02074","ANLDKFMB_01238","ALMPNPLM_01256","ICFJOHJH_00435","LBEKBKDJ_01217","MJNIIBMG_00434","CNFKGNAC_00460","AFDIGNOC_01348","ANBHONDD_01246","NDEKODAF_02901","DPBEEGGP_01382","ADHCADEO_02363","MHIPNNDE_00058","AMNKGELI_00124","PFJGCJHP_00810","CHLMDKPE_01786","NDDDIKEP_00367","ONGGBCKC_00465","PMNODAEP_00665","BOLLBKIO_00493","GGAGLIPG_00435","IHJLPFIO_00493","PNCEMHLO_00286","HIIAAINO_00441","MGKDKBAB_00665","MLIKFGOG_00441","LOFCOCJN_00665","LBMBHCAM_01169","PFAGPCGD_00378","KLCBHBCF_00493","OFBONHDF_00493","PBGIHMLN_00166","HGOBCNMM_00132","KNKCDEML_03812","PPAMLKML_00256","ANLDKFMB_00780","ALMPNPLM_01695","ILDLLNBB_01129","MDOHONPH_01173","ICFJOHJH_01379","CBGFCALD_01071","BAIOBGAL_00431","OEMCECNB_02083","KALPBOHI_01254","CNFKGNAC_00653","AFDIGNOC_02587","ANBHONDD_00165","NDEKODAF_00078","DPBEEGGP_00421","ADHCADEO_03181","MHIPNNDE_01063","AMNKGELI_00777","PFJGCJHP_00267","CHLMDKPE_00213","NDDDIKEP_01363","ONGGBCKC_00830","PMNODAEP_01551","BOLLBKIO_01337","GGAGLIPG_01532","IHJLPFIO_01553","PNCEMHLO_01470","HIIAAINO_01854","MGKDKBAB_01416","MLIKFGOG_01854","LOFCOCJN_01416","LBMBHCAM_00771","PFAGPCGD_01438","KLCBHBCF_01568","OFBONHDF_01341","PBGIHMLN_01392","HGOBCNMM_02749","FBLKDDDN_01673","GGMENCCH_00201","KNKCDEML_02429","GJADGJJH_01115","NBEGIDKM_01312","MAEDJJLC_01409","PPAMLKML_01888","ANLDKFMB_02244","ALMPNPLM_02289","ILDLLNBB_00231","MDOHONPH_00227","ICFJOHJH_00985","CBGFCALD_01771","LBEKBKDJ_01882","MJNIIBMG_01058","BAIOBGAL_01610","OEMCECNB_00190","KALPBOHI_00827","HIKCDMLK_01381","CNFKGNAC_02588","AFDIGNOC_00674","ANBHONDD_02768","NDEKODAF_01097","NDEKODAF_01096","DPBEEGGP_00933","DHOHKKHL_01270","ADHCADEO_02278","MHIPNNDE_02931","AMNKGELI_02446","AMNKGELI_02445","PFJGCJHP_02382","CHLMDKPE_02427","GGAGLIPG_00909","PNCEMHLO_00930","HIIAAINO_02178","MLIKFGOG_02605","LBMBHCAM_02035","PBGIHMLN_02152","HGOBCNMM_02660","FBLKDDDN_02538","GGMENCCH_02676","KNKCDEML_02290","GJADGJJH_02676","NBEGIDKM_03007","MAEDJJLC_02577","PPAMLKML_03502","ANLDKFMB_03264","ALMPNPLM_00996","ILDLLNBB_03181","MDOHONPH_03156","ICFJOHJH_03280","CBGFCALD_02080","LBEKBKDJ_02682","MJNIIBMG_02903","BAIOBGAL_03256","OEMCECNB_02985","KALPBOHI_01423","HIKCDMLK_03126","AFDIGNOC_00350","DHOHKKHL_03367","ADHCADEO_02043","MHIPNNDE_00757","CHLMDKPE_01053","NDDDIKEP_01855","ONGGBCKC_01322","PMNODAEP_02488","BOLLBKIO_01829","GGAGLIPG_02104","IHJLPFIO_01817","PNCEMHLO_02281","MGKDKBAB_02146","LOFCOCJN_02146","PFAGPCGD_02121","KLCBHBCF_02060","OFBONHDF_01848","HGOBCNMM_01551","FBLKDDDN_00644","GGMENCCH_01880","KNKCDEML_00891","GJADGJJH_01880","NBEGIDKM_00508","MAEDJJLC_00565","PPAMLKML_00002","ALMPNPLM_01927","CBGFCALD_00838","LBEKBKDJ_00565","MJNIIBMG_01643","BAIOBGAL_00725","OEMCECNB_01850","KALPBOHI_00001","CNFKGNAC_00885","AFDIGNOC_02352","ANBHONDD_00399","NDEKODAF_00371","DPBEEGGP_00655","ADHCADEO_02889","MHIPNNDE_02140","AMNKGELI_01490","CHLMDKPE_00444","NDDDIKEP_01594","ONGGBCKC_01061","PMNODAEP_01320","BOLLBKIO_01568","GGAGLIPG_01298","IHJLPFIO_01322","PNCEMHLO_01704","MGKDKBAB_01647","LOFCOCJN_01647","PFAGPCGD_01207","KLCBHBCF_01799","OFBONHDF_01110","PBGIHMLN_01098","HGOBCNMM_02532","FBLKDDDN_03276","GGMENCCH_03297","KNKCDEML_03197","GJADGJJH_03280","NBEGIDKM_02876","MAEDJJLC_03187","ANLDKFMB_02961","ALMPNPLM_03304","MDOHONPH_01371","LBEKBKDJ_03310","MJNIIBMG_03091","HIKCDMLK_03863","CNFKGNAC_01604","AFDIGNOC_03500","ANBHONDD_03208","NDEKODAF_02187","DPBEEGGP_03360","MHIPNNDE_01710","AMNKGELI_03441","PFJGCJHP_03139","NDDDIKEP_02981","ONGGBCKC_02997","PMNODAEP_03071","BOLLBKIO_03060","GGAGLIPG_02870","IHJLPFIO_03106","PNCEMHLO_02987","HIIAAINO_03403","MGKDKBAB_03129","MLIKFGOG_03169","LOFCOCJN_03129","LBMBHCAM_03225","PFAGPCGD_02780","KLCBHBCF_03097","OFBONHDF_03075","PBGIHMLN_03365","HGOBCNMM_02656","FBLKDDDN_02534","GGMENCCH_02672","KNKCDEML_02286","GJADGJJH_02672","NBEGIDKM_03003","MAEDJJLC_02581","PPAMLKML_03506","8_refound_74_stop","ALMPNPLM_00992","ILDLLNBB_03177","MDOHONPH_03160","12_refound_106_stop","CBGFCALD_02084","LBEKBKDJ_02686","MJNIIBMG_02899","BAIOBGAL_03252","OEMCECNB_02981","KALPBOHI_01427","HIKCDMLK_03123","AFDIGNOC_00346","DHOHKKHL_03364","ADHCADEO_02040","MHIPNNDE_00753","CHLMDKPE_01057","NDDDIKEP_01859","ONGGBCKC_01326","PMNODAEP_02492","BOLLBKIO_01833","GGAGLIPG_02108","IHJLPFIO_01821","PNCEMHLO_02277","MGKDKBAB_02142","LOFCOCJN_02142","PFAGPCGD_02117","KLCBHBCF_02064","OFBONHDF_01852","HGOBCNMM_02657","FBLKDDDN_02535","GGMENCCH_02673","KNKCDEML_02287","GJADGJJH_02673","NBEGIDKM_03004","MAEDJJLC_02580","PPAMLKML_03505","ANLDKFMB_03261","ALMPNPLM_00993","ILDLLNBB_03178","MDOHONPH_03159","ICFJOHJH_03277","CBGFCALD_02083","LBEKBKDJ_02685","MJNIIBMG_02900","BAIOBGAL_03253","OEMCECNB_02982","KALPBOHI_01426","HIKCDMLK_03124","AFDIGNOC_00347","DHOHKKHL_03365","ADHCADEO_02041","MHIPNNDE_00754","CHLMDKPE_01056","NDDDIKEP_01858","ONGGBCKC_01325","PMNODAEP_02491","BOLLBKIO_01832","GGAGLIPG_02107","IHJLPFIO_01820","PNCEMHLO_02278","MGKDKBAB_02143","LOFCOCJN_02143","PFAGPCGD_02118","KLCBHBCF_02063","OFBONHDF_01851","HGOBCNMM_02659","FBLKDDDN_02537","GGMENCCH_02675","KNKCDEML_02289","GJADGJJH_02675","NBEGIDKM_03006","MAEDJJLC_02578","PPAMLKML_03503","ANLDKFMB_03263","ALMPNPLM_00995","ILDLLNBB_03180","MDOHONPH_03157","ICFJOHJH_03279","CBGFCALD_02081","LBEKBKDJ_02683","MJNIIBMG_02902","BAIOBGAL_03255","OEMCECNB_02984","KALPBOHI_01424","HIKCDMLK_03125","AFDIGNOC_00349","DHOHKKHL_03366","ADHCADEO_02042","MHIPNNDE_00756","CHLMDKPE_01054","NDDDIKEP_01856","ONGGBCKC_01323","PMNODAEP_02489","BOLLBKIO_01830","GGAGLIPG_02105","IHJLPFIO_01818","PNCEMHLO_02280","MGKDKBAB_02145","LOFCOCJN_02145","PFAGPCGD_02120","KLCBHBCF_02061","OFBONHDF_01849","HGOBCNMM_02534","FBLKDDDN_03274","GGMENCCH_03299","KNKCDEML_02957","GJADGJJH_03282","NBEGIDKM_02874","MAEDJJLC_03189","ANLDKFMB_02963","ALMPNPLM_03302","MDOHONPH_01373","LBEKBKDJ_03308","MJNIIBMG_03093","HIKCDMLK_03821","CNFKGNAC_02890","ANBHONDD_03209","NDEKODAF_02185","DPBEEGGP_03361","ADHCADEO_03390","MHIPNNDE_01712","AMNKGELI_03438","PFJGCJHP_03137","NDDDIKEP_02978","ONGGBCKC_03000","PMNODAEP_03074","BOLLBKIO_03057","GGAGLIPG_02872","IHJLPFIO_03103","PNCEMHLO_02989","HIIAAINO_03400","MGKDKBAB_03126","MLIKFGOG_03172","LOFCOCJN_03126","LBMBHCAM_03228","PFAGPCGD_02783","KLCBHBCF_03100","OFBONHDF_03078","PBGIHMLN_03362","HGOBCNMM_02658","FBLKDDDN_02536","GGMENCCH_02674","KNKCDEML_02288","GJADGJJH_02674","NBEGIDKM_03005","MAEDJJLC_02579","PPAMLKML_03504","ANLDKFMB_03262","ILDLLNBB_03179","MDOHONPH_03158","ICFJOHJH_03278","CBGFCALD_02082","LBEKBKDJ_02684","MJNIIBMG_02901","BAIOBGAL_03254","OEMCECNB_02983","KALPBOHI_01425","AFDIGNOC_00348","ANBHONDD_01688","DPBEEGGP_01183","MHIPNNDE_00755","PFJGCJHP_02628","CHLMDKPE_01055","NDDDIKEP_01857","ONGGBCKC_01324","PMNODAEP_02490","BOLLBKIO_01831","GGAGLIPG_02106","IHJLPFIO_01819","PNCEMHLO_02279","MGKDKBAB_02144","LOFCOCJN_02144","PFAGPCGD_02119","KLCBHBCF_02062","OFBONHDF_01850","FBLKDDDN_00249","GGMENCCH_02275","KNKCDEML_01705","GJADGJJH_02275","NBEGIDKM_00974","MAEDJJLC_00960","ANLDKFMB_00953","ILDLLNBB_01290","MDOHONPH_01012","CBGFCALD_01233","LBEKBKDJ_00960","BAIOBGAL_00255","OEMCECNB_02245","KALPBOHI_02522","CNFKGNAC_01594","AFDIGNOC_02916","ANBHONDD_02196","NDEKODAF_00773","DPBEEGGP_00248","ADHCADEO_01710","AMNKGELI_01132","PFJGCJHP_01033","NDDDIKEP_01201","ONGGBCKC_01905","PMNODAEP_02872","BOLLBKIO_01175","IHJLPFIO_01715","HIIAAINO_02016","MGKDKBAB_01254","MLIKFGOG_02016","LOFCOCJN_01254","LBMBHCAM_00933","PFAGPCGD_01600","KLCBHBCF_01406","OFBONHDF_01503","PBGIHMLN_01616","HGOBCNMM_02655","FBLKDDDN_02533","GGMENCCH_02671","KNKCDEML_02285","GJADGJJH_02671","NBEGIDKM_03002","MAEDJJLC_02582","PPAMLKML_03507","ANLDKFMB_03260","ALMPNPLM_00991","ILDLLNBB_03176","MDOHONPH_03161","ICFJOHJH_03276","CBGFCALD_02085","LBEKBKDJ_02687","MJNIIBMG_02898","BAIOBGAL_03251","OEMCECNB_02980","KALPBOHI_01428","AFDIGNOC_00345","ADHCADEO_02039","MHIPNNDE_00752","PFJGCJHP_02629","CHLMDKPE_01058","NDDDIKEP_01860","ONGGBCKC_01327","PMNODAEP_02493","BOLLBKIO_01834","GGAGLIPG_02109","IHJLPFIO_01822","PNCEMHLO_02276","MGKDKBAB_02141","LOFCOCJN_02141","PFAGPCGD_02116","KLCBHBCF_02065","OFBONHDF_01853","HGOBCNMM_03184","FBLKDDDN_03508","GGMENCCH_03506","KNKCDEML_03467","GJADGJJH_03505","NBEGIDKM_03576","MAEDJJLC_03402","PPAMLKML_03055","ANLDKFMB_02494","ALMPNPLM_03355","ILDLLNBB_03229","MDOHONPH_03174","ICFJOHJH_02682","CBGFCALD_03576","LBEKBKDJ_03370","MJNIIBMG_03421","BAIOBGAL_03463","OEMCECNB_03404","KALPBOHI_03445","HIKCDMLK_02672","CNFKGNAC_03230","AFDIGNOC_03427","ANBHONDD_03523","DPBEEGGP_03379","DHOHKKHL_03063","ADHCADEO_02335","MHIPNNDE_03069","AMNKGELI_01241","PFJGCJHP_01297","CHLMDKPE_03518","GGAGLIPG_03301","PNCEMHLO_03318","HIIAAINO_03131","MLIKFGOG_03235","LBMBHCAM_03011","PBGIHMLN_03424","FBLKDDDN_00247","GGMENCCH_02277","KNKCDEML_01703","GJADGJJH_02277","NBEGIDKM_00976","MAEDJJLC_00962","PPAMLKML_02694","ANLDKFMB_00955","ILDLLNBB_01292","MDOHONPH_01010","CBGFCALD_01235","LBEKBKDJ_00962","MJNIIBMG_02359","BAIOBGAL_00253","OEMCECNB_02247","KALPBOHI_02524","CNFKGNAC_01592","ANBHONDD_02198","NDEKODAF_00775","DPBEEGGP_00246","AMNKGELI_01130","PFJGCJHP_01035","CHLMDKPE_03030","NDDDIKEP_01199","ONGGBCKC_01903","PMNODAEP_02870","BOLLBKIO_01173","IHJLPFIO_01717","HIIAAINO_02018","MGKDKBAB_01252","MLIKFGOG_02018","LOFCOCJN_01252","LBMBHCAM_00935","PFAGPCGD_01602","KLCBHBCF_01404","OFBONHDF_01505","HGOBCNMM_03182","FBLKDDDN_03506","GGMENCCH_03504","KNKCDEML_03469","GJADGJJH_03507","NBEGIDKM_03578","MAEDJJLC_03404","PPAMLKML_03057","ANLDKFMB_02496","ALMPNPLM_03357","ILDLLNBB_03231","MDOHONPH_03176","ICFJOHJH_03341","CBGFCALD_03578","LBEKBKDJ_03368","MJNIIBMG_03423","BAIOBGAL_03465","OEMCECNB_03406","KALPBOHI_03266","HIKCDMLK_02670","CNFKGNAC_03228","AFDIGNOC_03425","ANBHONDD_03521"
,"DPBEEGGP_03377","DHOHKKHL_03065","ADHCADEO_02333","MHIPNNDE_03071","AMNKGELI_01243","PFJGCJHP_01299","CHLMDKPE_03500","GGAGLIPG_03303","PNCEMHLO_03316","HIIAAINO_03133","MLIKFGOG_03237","LBMBHCAM_03013","PBGIHMLN_03426","HGOBCNMM_03183","FBLKDDDN_03507","GGMENCCH_03505","KNKCDEML_03468","GJADGJJH_03506","NBEGIDKM_03577","MAEDJJLC_03403","PPAMLKML_03056","ANLDKFMB_02495","ALMPNPLM_03356","ILDLLNBB_03230","MDOHONPH_03175","ICFJOHJH_03342","CBGFCALD_03577","LBEKBKDJ_03369","MJNIIBMG_03422","BAIOBGAL_03464","OEMCECNB_03405","KALPBOHI_03265","HIKCDMLK_02671","CNFKGNAC_03229","AFDIGNOC_03426","ANBHONDD_03522","DPBEEGGP_03378","DHOHKKHL_03064","ADHCADEO_02334","MHIPNNDE_03070","AMNKGELI_01242","PFJGCJHP_01298","CHLMDKPE_03499","GGAGLIPG_03302","PNCEMHLO_03317","HIIAAINO_03132","MLIKFGOG_03236","LBMBHCAM_03012","PBGIHMLN_03425","FBLKDDDN_00252","GGMENCCH_02272","KNKCDEML_01708","GJADGJJH_02272","NBEGIDKM_00971","MAEDJJLC_00957","ANLDKFMB_00950","ILDLLNBB_01287","MDOHONPH_01015","CBGFCALD_01230","LBEKBKDJ_00957","BAIOBGAL_00262","OEMCECNB_02242","KALPBOHI_02519","CNFKGNAC_01597","ANBHONDD_02193","NDEKODAF_00770","DPBEEGGP_00251","ADHCADEO_01707","AMNKGELI_01135","PFJGCJHP_01030","NDDDIKEP_01204","ONGGBCKC_01908","PMNODAEP_02875","BOLLBKIO_01178","IHJLPFIO_01712","HIIAAINO_02013","MGKDKBAB_01257","MLIKFGOG_02013","LOFCOCJN_01257","LBMBHCAM_00930","PFAGPCGD_01597","KLCBHBCF_01409","OFBONHDF_01500","PBGIHMLN_01613","HGOBCNMM_02533","FBLKDDDN_03275","GGMENCCH_03298","KNKCDEML_03198","GJADGJJH_03281","NBEGIDKM_02875","MAEDJJLC_03188","ANLDKFMB_02962","ALMPNPLM_03303","MDOHONPH_01372","LBEKBKDJ_03309","MJNIIBMG_03092","HIKCDMLK_03822","CNFKGNAC_01603","NDEKODAF_02186","ADHCADEO_03389","MHIPNNDE_01711","AMNKGELI_03440","PFJGCJHP_03138","NDDDIKEP_02980","ONGGBCKC_02998","PMNODAEP_03072","BOLLBKIO_03059","GGAGLIPG_02871","IHJLPFIO_03105","PNCEMHLO_02988","HIIAAINO_03402","MGKDKBAB_03128","MLIKFGOG_03170","LOFCOCJN_03128","LBMBHCAM_03226","PFAGPCGD_02781","KLCBHBCF_03098","OFBONHDF_03076","PBGIHMLN_03364","HGOBCNMM_02202","FBLKDDDN_02207","GGMENCCH_01137","KNKCDEML_00179","GJADGJJH_00179","NBEGIDKM_01984","MAEDJJLC_02345","ANLDKFMB_01494","ILDLLNBB_00698","MDOHONPH_00756","CBGFCALD_00178","LBEKBKDJ_02366","MJNIIBMG_00179","OEMCECNB_01106","CNFKGNAC_02369","NDEKODAF_02018","MHIPNNDE_02337","AMNKGELI_02702","PFJGCJHP_02934","NDDDIKEP_00111","ONGGBCKC_00209","PMNODAEP_00921","BOLLBKIO_00237","GGAGLIPG_00180","IHJLPFIO_00237","PNCEMHLO_00541","HIIAAINO_00183","MGKDKBAB_00921","MLIKFGOG_00183","LOFCOCJN_00921","LBMBHCAM_02718","PFAGPCGD_00122","KLCBHBCF_00237","OFBONHDF_00237","PBGIHMLN_00420","HGOBCNMM_02205","FBLKDDDN_02210","GGMENCCH_01140","KNKCDEML_00176","GJADGJJH_00176","NBEGIDKM_01981","MAEDJJLC_02348","ANLDKFMB_01497","ILDLLNBB_00695","MDOHONPH_00759","CBGFCALD_00175","LBEKBKDJ_02363","MJNIIBMG_00176","OEMCECNB_01109","CNFKGNAC_02372","NDEKODAF_02015","MHIPNNDE_02340","AMNKGELI_02699","PFJGCJHP_02937","NDDDIKEP_00108","ONGGBCKC_00206","PMNODAEP_00924","BOLLBKIO_00234","GGAGLIPG_00177","IHJLPFIO_00234","PNCEMHLO_00544","HIIAAINO_00180","MGKDKBAB_00924","MLIKFGOG_00180","LOFCOCJN_00924","LBMBHCAM_02715","PFAGPCGD_00119","KLCBHBCF_00234","OFBONHDF_00234","PBGIHMLN_00423","HGOBCNMM_02204","FBLKDDDN_02209","GGMENCCH_01139","KNKCDEML_00177","GJADGJJH_00177","NBEGIDKM_01982","MAEDJJLC_02347","ANLDKFMB_01496","ILDLLNBB_00696","MDOHONPH_00758","CBGFCALD_00176","LBEKBKDJ_02364","MJNIIBMG_00177","OEMCECNB_01108","CNFKGNAC_02371","NDEKODAF_02016","MHIPNNDE_02339","AMNKGELI_02700","PFJGCJHP_02936","NDDDIKEP_00109","ONGGBCKC_00207","PMNODAEP_00923","BOLLBKIO_00235","GGAGLIPG_00178","IHJLPFIO_00235","PNCEMHLO_00543","HIIAAINO_00181","MGKDKBAB_00923","MLIKFGOG_00181","LOFCOCJN_00923","LBMBHCAM_02716","PFAGPCGD_00120","KLCBHBCF_00235","OFBONHDF_00235","PBGIHMLN_00422","HGOBCNMM_02201","FBLKDDDN_02206","GGMENCCH_01136","KNKCDEML_00180","GJADGJJH_00180","NBEGIDKM_01985","MAEDJJLC_02344","ANLDKFMB_01493","ILDLLNBB_00699","MDOHONPH_00755","CBGFCALD_00179","LBEKBKDJ_02367","MJNIIBMG_00180","OEMCECNB_01105","CNFKGNAC_02368","NDEKODAF_02019","MHIPNNDE_02336","AMNKGELI_02703","PFJGCJHP_02933","NDDDIKEP_00112","ONGGBCKC_00210","PMNODAEP_00920","BOLLBKIO_00238","GGAGLIPG_00181","IHJLPFIO_00238","PNCEMHLO_00540","HIIAAINO_00184","MGKDKBAB_00920","MLIKFGOG_00184","LOFCOCJN_00920","LBMBHCAM_02719","PFAGPCGD_00123","KLCBHBCF_00238","OFBONHDF_00238","PBGIHMLN_00419","HGOBCNMM_02203","FBLKDDDN_02208","GGMENCCH_01138","KNKCDEML_00178","GJADGJJH_00178","NBEGIDKM_01983","MAEDJJLC_02346","ANLDKFMB_01495","ILDLLNBB_00697","MDOHONPH_00757","CBGFCALD_00177","LBEKBKDJ_02365","MJNIIBMG_00178","OEMCECNB_01107","CNFKGNAC_02370","NDEKODAF_02017","MHIPNNDE_02338","AMNKGELI_02701","PFJGCJHP_02935","NDDDIKEP_00110","ONGGBCKC_00208","PMNODAEP_00922","BOLLBKIO_00236","GGAGLIPG_00179","IHJLPFIO_00236","PNCEMHLO_00542","HIIAAINO_00182","MGKDKBAB_00922","MLIKFGOG_00182","LOFCOCJN_00922","LBMBHCAM_02717","PFAGPCGD_00121","KLCBHBCF_00236","OFBONHDF_00236","PBGIHMLN_00421","HGOBCNMM_02200","FBLKDDDN_02205","GGMENCCH_01135","KNKCDEML_00181","GJADGJJH_00181","NBEGIDKM_01986","MAEDJJLC_02343","ANLDKFMB_01492","ILDLLNBB_00700","MDOHONPH_00754","CBGFCALD_00180","LBEKBKDJ_02368","MJNIIBMG_00181","OEMCECNB_01104","CNFKGNAC_02367","NDEKODAF_02020","MHIPNNDE_02335","AMNKGELI_02704","PFJGCJHP_02932","NDDDIKEP_00113","ONGGBCKC_00211","PMNODAEP_00919","BOLLBKIO_00239","GGAGLIPG_00182","IHJLPFIO_00239","PNCEMHLO_00539","HIIAAINO_00185","MGKDKBAB_00919","MLIKFGOG_00185","LOFCOCJN_00919","LBMBHCAM_02720","PFAGPCGD_00124","KLCBHBCF_00239","OFBONHDF_00239","PBGIHMLN_00418","HGOBCNMM_02382","FBLKDDDN_01569","GGMENCCH_00305","KNKCDEML_03904","GJADGJJH_01011","NBEGIDKM_01416","MAEDJJLC_01513","PPAMLKML_00855","ALMPNPLM_00736","ILDLLNBB_00333","MDOHONPH_00329","ICFJOHJH_00887","CBGFCALD_01670","LBEKBKDJ_01778","MJNIIBMG_00954","BAIOBGAL_01509","OEMCECNB_00291","KALPBOHI_00724","HIKCDMLK_01280","CNFKGNAC_02558","AFDIGNOC_00809","ANBHONDD_02689","NDEKODAF_02381","DPBEEGGP_02127","DHOHKKHL_01372","ADHCADEO_02402","MHIPNNDE_03453","AMNKGELI_03060","PFJGCJHP_03274","CHLMDKPE_02172","HIIAAINO_00964","MLIKFGOG_00964","LBMBHCAM_01692","PBGIHMLN_01287","FBLKDDDN_00253","GGMENCCH_02271","KNKCDEML_01709","GJADGJJH_02271","NBEGIDKM_00970","MAEDJJLC_00956","ANLDKFMB_00949","ILDLLNBB_01286","MDOHONPH_01016","CBGFCALD_01229","LBEKBKDJ_00956","BAIOBGAL_00263","OEMCECNB_02241","CNFKGNAC_01598","ANBHONDD_02192","NDEKODAF_00769","DPBEEGGP_00252","AMNKGELI_01136","PFJGCJHP_01029","NDDDIKEP_01205","ONGGBCKC_01909","PMNODAEP_02876","BOLLBKIO_01179","IHJLPFIO_01711","HIIAAINO_02012","MGKDKBAB_01258","MLIKFGOG_02012","LOFCOCJN_01258","LBMBHCAM_00929","PFAGPCGD_01596","KLCBHBCF_01410","OFBONHDF_01499","PBGIHMLN_01612","FBLKDDDN_00248","GGMENCCH_02276","KNKCDEML_01704","GJADGJJH_02276","NBEGIDKM_00975","MAEDJJLC_00961","ANLDKFMB_00954","ILDLLNBB_01291","MDOHONPH_01011","CBGFCALD_01234","LBEKBKDJ_00961","BAIOBGAL_00254","OEMCECNB_02246","KALPBOHI_02523","CNFKGNAC_01593","ANBHONDD_02197","NDEKODAF_00774","DPBEEGGP_00247","AMNKGELI_01131","PFJGCJHP_01034","NDDDIKEP_01200","ONGGBCKC_01904","PMNODAEP_02871","BOLLBKIO_01174","IHJLPFIO_01716","HIIAAINO_02017","MGKDKBAB_01253","MLIKFGOG_02017","LOFCOCJN_01253","LBMBHCAM_00934","PFAGPCGD_01601","KLCBHBCF_01405","OFBONHDF_01504","FBLKDDDN_00255","GGMENCCH_02269","KNKCDEML_01711","GJADGJJH_02269","NBEGIDKM_00968","MAEDJJLC_00954","ANLDKFMB_00947","ILDLLNBB_01284","MDOHONPH_01018","CBGFCALD_01227","LBEKBKDJ_00954","BAIOBGAL_00265","OEMCECNB_02239","CNFKGNAC_01600","ANBHONDD_02190","NDEKODAF_00767","DPBEEGGP_00254","AMNKGELI_01138","PFJGCJHP_01027","NDDDIKEP_01207","ONGGBCKC_01911","PMNODAEP_02878","BOLLBKIO_01181","IHJLPFIO_01709","HIIAAINO_02010","MGKDKBAB_01260","MLIKFGOG_02010","LOFCOCJN_01260","LBMBHCAM_00927","PFAGPCGD_01594","KLCBHBCF_01412","OFBONHDF_01497","PBGIHMLN_01610","HGOBCNMM_03180","FBLKDDDN_03504","GGMENCCH_03502","KNKCDEML_03471","GJADGJJH_03509","NBEGIDKM_03580","MAEDJJLC_03406","PPAMLKML_03059","ANLDKFMB_02497","ALMPNPLM_03359","ILDLLNBB_03233","MDOHONPH_03178","ICFJOHJH_03339","CBGFCALD_03621","LBEKBKDJ_03366","MJNIIBMG_03425","BAIOBGAL_03184","OEMCECNB_03409","KALPBOHI_03268","CNFKGNAC_03226","ANBHONDD_03556","DPBEEGGP_03375","ADHCADEO_02331","MHIPNNDE_03073","AMNKGELI_01246","PFJGCJHP_03660","CHLMDKPE_03502","GGAGLIPG_03305","PNCEMHLO_03314","HIIAAINO_03135","MLIKFGOG_03239","LBMBHCAM_03015","PBGIHMLN_03428","FBLKDDDN_00256","GGMENCCH_02268","KNKCDEML_01712","GJADGJJH_02268","NBEGIDKM_00967","MAEDJJLC_00953","ANLDKFMB_00946","ILDLLNBB_01283","MDOHONPH_01019","CBGFCALD_01226","LBEKBKDJ_00953","BAIOBGAL_00266","OEMCECNB_02238","CNFKGNAC_01601","ANBHONDD_02189","NDEKODAF_00766","DPBEEGGP_00255","AMNKGELI_01139","PFJGCJHP_01026","NDDDIKEP_01208","ONGGBCKC_01912","PMNODAEP_02879","BOLLBKIO_01182","IHJLPFIO_01708","HIIAAINO_02009","MGKDKBAB_01261","MLIKFGOG_02009","LOFCOCJN_01261","LBMBHCAM_00926","PFAGPCGD_01593","KLCBHBCF_01413","OFBONHDF_01496","PBGIHMLN_01609","HGOBCNMM_03179","FBLKDDDN_01874","GGMENCCH_03203","KNKCDEML_03768","KNKCDEML_03867","GJADGJJH_03272","NBEGIDKM_03366","MAEDJJLC_03167","PPAMLKML_03060","ANLDKFMB_02498","ALMPNPLM_03360","ILDLLNBB_03234","MDOHONPH_03179","ICFJOHJH_03338","CBGFCALD_03526","LBEKBKDJ_03273","MJNIIBMG_03121","BAIOBGAL_03185","OEMCECNB_03413","KALPBOHI_03269","CNFKGNAC_03225","ANBHONDD_03557","DPBEEGGP_03374","ADHCADEO_02330","MHIPNNDE_03074","AMNKGELI_01247","CHLMDKPE_03503","GGAGLIPG_03306","PNCEMHLO_03313","HIIAAINO_03136","MLIKFGOG_03240","LBMBHCAM_03016","PBGIHMLN_03429","KNKCDEML_03815","PPAMLKML_00253","ALMPNPLM_01698","ICFJOHJH_01382","CBGFCALD_01068","BAIOBGAL_00434","OEMCECNB_02080","KALPBOHI_01251","CNFKGNAC_00656","AFDIGNOC_02584","ANBHONDD_00168","NDEKODAF_00082","DPBEEGGP_00424","AMNKGELI_00781","PFJGCJHP_00270","CHLMDKPE_00216","NDDDIKEP_01366","ONGGBCKC_00833","PMNODAEP_01548","BOLLBKIO_01340","GGAGLIPG_01529","IHJLPFIO_01550","PNCEMHLO_01473","HIIAAINO_01851","MGKDKBAB_01419","MLIKFGOG_01851"
,"LOFCOCJN_01419","LBMBHCAM_00768","PFAGPCGD_01435","KLCBHBCF_01571","OFBONHDF_01338","PBGIHMLN_03011","HGOBCNMM_02754","FBLKDDDN_01666","GGMENCCH_00208","KNKCDEML_03591","GJADGJJH_01108","NBEGIDKM_01319","MAEDJJLC_01416","PPAMLKML_01893","ANLDKFMB_03050","ALMPNPLM_02287","ILDLLNBB_00236","MDOHONPH_00232","ICFJOHJH_00983","CBGFCALD_01766","LBEKBKDJ_01875","MJNIIBMG_01051","BAIOBGAL_01605","OEMCECNB_00195","KALPBOHI_00822","HIKCDMLK_01376","CNFKGNAC_02591","ANBHONDD_02766","NDEKODAF_01093","DPBEEGGP_00931","DHOHKKHL_01275","ADHCADEO_02273","MHIPNNDE_02924","AMNKGELI_02448","PFJGCJHP_02389","CHLMDKPE_02432","GGAGLIPG_00914","PNCEMHLO_00935","HGOBCNMM_03181","FBLKDDDN_03505","GGMENCCH_03503","KNKCDEML_03470","GJADGJJH_03508","NBEGIDKM_03579","MAEDJJLC_03405","PPAMLKML_03058","8_refound_81_stop","ALMPNPLM_03358","ILDLLNBB_03232","MDOHONPH_03177","ICFJOHJH_03340","CBGFCALD_03579","LBEKBKDJ_03367","MJNIIBMG_03424","OEMCECNB_03407","KALPBOHI_03267","CNFKGNAC_03227","ANBHONDD_03520","DPBEEGGP_03376","26_refound_259","MHIPNNDE_03072","AMNKGELI_01244","PFJGCJHP_01300","CHLMDKPE_03501","GGAGLIPG_03304","PNCEMHLO_03315","HIIAAINO_03134","MLIKFGOG_03238","LBMBHCAM_03014","PBGIHMLN_03427","HGOBCNMM_02759","FBLKDDDN_01663","GGMENCCH_00211","KNKCDEML_02243","KNKCDEML_03588","KNKCDEML_03769","GJADGJJH_01105","NBEGIDKM_01322","MAEDJJLC_01419","PPAMLKML_00950","ANLDKFMB_03045","ALMPNPLM_02285","ILDLLNBB_00239","MDOHONPH_00235","ICFJOHJH_00981","CBGFCALD_01764","LBEKBKDJ_01872","MJNIIBMG_01048","BAIOBGAL_01603","OEMCECNB_00197","KALPBOHI_00819","HIKCDMLK_01374","CNFKGNAC_02597","ANBHONDD_02763","NDEKODAF_01089","DPBEEGGP_00928","DHOHKKHL_01278","ADHCADEO_02271","MHIPNNDE_02920","AMNKGELI_02451","PFJGCJHP_02392","CHLMDKPE_02437","GGAGLIPG_00917","PNCEMHLO_00938","FBLKDDDN_00160","GGMENCCH_02364","KNKCDEML_01616","GJADGJJH_02364","NBEGIDKM_02758","MAEDJJLC_01049","ANLDKFMB_01075","CBGFCALD_01317","LBEKBKDJ_01049","MJNIIBMG_02274","BAIOBGAL_00160","OEMCECNB_02495","CNFKGNAC_01486","ANBHONDD_02281","NDEKODAF_00855","DPBEEGGP_00163","ADHCADEO_01113","AMNKGELI_01019","PFJGCJHP_01116","NDDDIKEP_01112","ONGGBCKC_01816","PMNODAEP_02859","BOLLBKIO_01086","IHJLPFIO_02187","HIIAAINO_02815","MGKDKBAB_01165","MLIKFGOG_02876","LOFCOCJN_01165","LBMBHCAM_02993","PFAGPCGD_01689","KLCBHBCF_01317","OFBONHDF_01592","HGOBCNMM_02760","FBLKDDDN_01662","GGMENCCH_00212","KNKCDEML_02242","GJADGJJH_01104","NBEGIDKM_01323","MAEDJJLC_01420","PPAMLKML_00949","ANLDKFMB_03044","ALMPNPLM_02284","ILDLLNBB_00240","MDOHONPH_00236","ICFJOHJH_00980","CBGFCALD_01763","LBEKBKDJ_01871","MJNIIBMG_01047","BAIOBGAL_01602","OEMCECNB_00198","KALPBOHI_00818","HIKCDMLK_01373","CNFKGNAC_02598","ANBHONDD_02762","NDEKODAF_01088","DPBEEGGP_00927","DHOHKKHL_01279","ADHCADEO_02270","MHIPNNDE_02919","AMNKGELI_02452","PFJGCJHP_02393","CHLMDKPE_02438","GGAGLIPG_00918","PNCEMHLO_00939","HGOBCNMM_02761","FBLKDDDN_01661","GGMENCCH_00213","KNKCDEML_02241","GJADGJJH_01103","NBEGIDKM_01324","MAEDJJLC_01421","PPAMLKML_00948","ANLDKFMB_03043","ALMPNPLM_02283","ILDLLNBB_00241","MDOHONPH_00237","ICFJOHJH_00979","CBGFCALD_01762","LBEKBKDJ_01870","MJNIIBMG_01046","BAIOBGAL_01601","OEMCECNB_00199","KALPBOHI_00817","HIKCDMLK_01372","CNFKGNAC_02599","ANBHONDD_02761","NDEKODAF_01087","DPBEEGGP_00926","DHOHKKHL_01280","ADHCADEO_02269","MHIPNNDE_02918","AMNKGELI_02453","PFJGCJHP_02394","CHLMDKPE_02439","GGAGLIPG_00919","PNCEMHLO_00940","HGOBCNMM_02762","FBLKDDDN_01660","GGMENCCH_00214","KNKCDEML_02240","GJADGJJH_01102","NBEGIDKM_01325","MAEDJJLC_01422","PPAMLKML_00947","ANLDKFMB_03042","ALMPNPLM_02282","ILDLLNBB_00242","MDOHONPH_00238","ICFJOHJH_00978","CBGFCALD_01761","LBEKBKDJ_01869","MJNIIBMG_01045","BAIOBGAL_01600","OEMCECNB_00200","KALPBOHI_00816","HIKCDMLK_01371","CNFKGNAC_02600","ANBHONDD_02760","NDEKODAF_01086","DPBEEGGP_00925","DHOHKKHL_01281","ADHCADEO_02268","MHIPNNDE_02917","AMNKGELI_02454","PFJGCJHP_02395","CHLMDKPE_02440","GGAGLIPG_00920","PNCEMHLO_00941","FBLKDDDN_00161","GGMENCCH_02363","KNKCDEML_01617","GJADGJJH_02363","NBEGIDKM_02759","MAEDJJLC_01048","ANLDKFMB_01074","CBGFCALD_01316","LBEKBKDJ_01048","MJNIIBMG_02275","BAIOBGAL_00161","OEMCECNB_02494","CNFKGNAC_01487","ANBHONDD_02280","NDEKODAF_00854","DPBEEGGP_00164","AMNKGELI_01020","PFJGCJHP_01115","NDDDIKEP_01113","ONGGBCKC_01817","PMNODAEP_02858","BOLLBKIO_01087","IHJLPFIO_02188","HIIAAINO_02814","MGKDKBAB_01166","MLIKFGOG_02877","LOFCOCJN_01166","LBMBHCAM_02994","PFAGPCGD_01688","KLCBHBCF_01318","OFBONHDF_01591","HGOBCNMM_03216","FBLKDDDN_01909","GGMENCCH_03238","KNKCDEML_03308","GJADGJJH_03237","NBEGIDKM_03646","MAEDJJLC_03132","PPAMLKML_02813","ANLDKFMB_03236","ALMPNPLM_03289","ILDLLNBB_03259","MDOHONPH_03223","ICFJOHJH_02576","CBGFCALD_03405","LBEKBKDJ_03238","BAIOBGAL_03221","OEMCECNB_03601","KALPBOHI_03399","19_refound_174","CNFKGNAC_01733","AFDIGNOC_03372","ANBHONDD_03222","NDEKODAF_03017","DPBEEGGP_03149","25_refound_236","ADHCADEO_02966","MHIPNNDE_03025","AMNKGELI_01309","GGAGLIPG_02972","PNCEMHLO_03101","PBGIHMLN_03007","HGOBCNMM_03200","5_refound_48","ANLDKFMB_03251","ILDLLNBB_03326","MDOHONPH_03337","16_refound_147","HIKCDMLK_03860","CNFKGNAC_01718","AFDIGNOC_03357","ANBHONDD_03481","NDEKODAF_03002","DPBEEGGP_03134","DHOHKKHL_03777","ADHCADEO_02984","MHIPNNDE_03009","NDDDIKEP_03274","ONGGBCKC_03440","PMNODAEP_03494","BOLLBKIO_03291","GGAGLIPG_02955","IHJLPFIO_03263","PNCEMHLO_03118","HIIAAINO_03384","MGKDKBAB_03520","MLIKFGOG_03428","LOFCOCJN_03520","LBMBHCAM_03081","PFAGPCGD_03189","KLCBHBCF_03546","OFBONHDF_03169","PBGIHMLN_02993","FBLKDDDN_02712","GGMENCCH_02774","KNKCDEML_02730","GJADGJJH_02850","NBEGIDKM_02479","MAEDJJLC_02774","ANLDKFMB_02195","CBGFCALD_02645","LBEKBKDJ_02955","MJNIIBMG_02713","OEMCECNB_02444","CNFKGNAC_02112","ANBHONDD_01979","NDEKODAF_01508","DPBEEGGP_02766","MHIPNNDE_00485","AMNKGELI_00349","NDDDIKEP_02145","ONGGBCKC_01612","PMNODAEP_01842","BOLLBKIO_02119","IHJLPFIO_02671","HIIAAINO_02508","MGKDKBAB_01856","MLIKFGOG_02409","LOFCOCJN_01856","LBMBHCAM_02248","PFAGPCGD_02994","KLCBHBCF_02350","OFBONHDF_02269","PBGIHMLN_01978","FBLKDDDN_02713","GGMENCCH_02773","KNKCDEML_02729","GJADGJJH_02851","NBEGIDKM_02478","MAEDJJLC_02773","ANLDKFMB_02196","CBGFCALD_02646","LBEKBKDJ_02956","MJNIIBMG_02714","OEMCECNB_02445","CNFKGNAC_02113","ANBHONDD_01978","NDEKODAF_01507","DPBEEGGP_02767","MHIPNNDE_00484","AMNKGELI_00348","NDDDIKEP_02146","ONGGBCKC_01613","PMNODAEP_01843","BOLLBKIO_02120","IHJLPFIO_02670","HIIAAINO_02507","MGKDKBAB_01855","MLIKFGOG_02410","LOFCOCJN_01855","LBMBHCAM_02247","PFAGPCGD_02995","KLCBHBCF_02351","OFBONHDF_02268","PBGIHMLN_01979","FBLKDDDN_02710","GGMENCCH_02776","KNKCDEML_02732","GJADGJJH_02848","NBEGIDKM_02481","MAEDJJLC_02776","ANLDKFMB_02193","CBGFCALD_02643","LBEKBKDJ_02953","MJNIIBMG_02711","OEMCECNB_02442","CNFKGNAC_02110","ANBHONDD_01981","NDEKODAF_01510","DPBEEGGP_02764","MHIPNNDE_00487","AMNKGELI_00351","NDDDIKEP_02143","ONGGBCKC_01610","PMNODAEP_01840","BOLLBKIO_02117","IHJLPFIO_02673","HIIAAINO_02510","MGKDKBAB_01858","MLIKFGOG_02407","LOFCOCJN_01858","LBMBHCAM_02250","PFAGPCGD_02992","KLCBHBCF_02348","OFBONHDF_02271","PBGIHMLN_01976","FBLKDDDN_02708","GGMENCCH_02778","KNKCDEML_02734","GJADGJJH_02846","NBEGIDKM_02483","MAEDJJLC_02778","ANLDKFMB_02191","CBGFCALD_02641","LBEKBKDJ_02951","MJNIIBMG_02709","OEMCECNB_02440","CNFKGNAC_02108","ANBHONDD_01983","NDEKODAF_01512","DPBEEGGP_02762","MHIPNNDE_00489","AMNKGELI_00353","NDDDIKEP_02141","ONGGBCKC_01608","PMNODAEP_01838","BOLLBKIO_02115","IHJLPFIO_02675","HIIAAINO_02512","MGKDKBAB_01860","MLIKFGOG_02405","LOFCOCJN_01860","LBMBHCAM_02252","PFAGPCGD_02990","KLCBHBCF_02346","OFBONHDF_02273","PBGIHMLN_01974","FBLKDDDN_02711","GGMENCCH_02775","KNKCDEML_02731","GJADGJJH_02849","NBEGIDKM_02480","MAEDJJLC_02775","ANLDKFMB_02194","CBGFCALD_02644","LBEKBKDJ_02954","MJNIIBMG_02712","OEMCECNB_02443","CNFKGNAC_02111","ANBHONDD_01980","NDEKODAF_01509","DPBEEGGP_02765","MHIPNNDE_00486","AMNKGELI_00350","NDDDIKEP_02144","ONGGBCKC_01611","PMNODAEP_01841","BOLLBKIO_02118","IHJLPFIO_02672","HIIAAINO_02509","MGKDKBAB_01857","MLIKFGOG_02408","LOFCOCJN_01857","LBMBHCAM_02249","PFAGPCGD_02993","KLCBHBCF_02349","OFBONHDF_02270","PBGIHMLN_01977","FBLKDDDN_02709","GGMENCCH_02777","KNKCDEML_02733","GJADGJJH_02847","NBEGIDKM_02482","MAEDJJLC_02777","ANLDKFMB_02192","CBGFCALD_02642","LBEKBKDJ_02952","MJNIIBMG_02710","OEMCECNB_02441","CNFKGNAC_02109","ANBHONDD_01982","NDEKODAF_01511","DPBEEGGP_02763","MHIPNNDE_00488","AMNKGELI_00352","NDDDIKEP_02142","ONGGBCKC_01609","PMNODAEP_01839","BOLLBKIO_02116","IHJLPFIO_02674","HIIAAINO_02511","MGKDKBAB_01859","MLIKFGOG_02406","LOFCOCJN_01859","LBMBHCAM_02251","PFAGPCGD_02991","KLCBHBCF_02347","OFBONHDF_02272","PBGIHMLN_01975","FBLKDDDN_02707","GGMENCCH_02779","KNKCDEML_02735","GJADGJJH_02845","NBEGIDKM_02484","MAEDJJLC_02779","ANLDKFMB_02190","CBGFCALD_02640","LBEKBKDJ_02950","MJNIIBMG_02708","OEMCECNB_02439","CNFKGNAC_02107","ANBHONDD_01984","NDEKODAF_01513","DPBEEGGP_02761","MHIPNNDE_00490","AMNKGELI_00354","NDDDIKEP_02140","ONGGBCKC_01607","PMNODAEP_01837","BOLLBKIO_02114","IHJLPFIO_02676","HIIAAINO_02513","MGKDKBAB_01861","MLIKFGOG_02404","LOFCOCJN_01861","LBMBHCAM_02253","PFAGPCGD_02989","KLCBHBCF_02345","OFBONHDF_02274","PBGIHMLN_01973","FBLKDDDN_00162","GGMENCCH_02362","KNKCDEML_01618","GJADGJJH_02362","NBEGIDKM_02760","MAEDJJLC_01047","ANLDKFMB_01073","CBGFCALD_01315","LBEKBKDJ_01047","MJNIIBMG_02276","BAIOBGAL_00162","OEMCECNB_02493","CNFKGNAC_01488","ANBHONDD_02279","NDEKODAF_00853","DPBEEGGP_00165","AMNKGELI_01021","PFJGCJHP_01114","NDDDIKEP_01114","ONGGBCKC_01818","PMNODAEP_02857","BOLLBKIO_01088","IHJLPFIO_02189","HIIAAINO_02813","MGKDKBAB_01167","MLIKFGOG_02878","LOFCOCJN_01167","LBMBHCAM_02995","PFAGPCGD_01687","KLCBHBCF_01319","OFBONHDF_01590","FBLKDDDN_00172","GGMENCCH_02352","KNKCDEML_01628","GJADGJJH_02352","NBEGIDKM_01053","MAEDJJLC_01037","CBGFCALD_01310","LBEKBKDJ_01037","MJNIIBMG_02286","BAIOBGAL_00178","OEMCECNB_02322","CNFKGNAC_01517","ANBHONDD_02273","NDEKODAF_00850"
,"DPBEEGGP_00171","AMNKGELI_01044","PFJGCJHP_01110","NDDDIKEP_01118","ONGGBCKC_01822","PMNODAEP_02853","BOLLBKIO_01092","IHJLPFIO_02193","HIIAAINO_02093","MGKDKBAB_01171","MLIKFGOG_02093","LOFCOCJN_01171","LBMBHCAM_01010","PFAGPCGD_01683","KLCBHBCF_01323","OFBONHDF_01586","PPAMLKML_03549","ILDLLNBB_03130","MDOHONPH_03305","ICFJOHJH_03486","BAIOBGAL_03468","OEMCECNB_02857","KALPBOHI_03368","HIKCDMLK_03512","AFDIGNOC_03346","ANBHONDD_03551","DPBEEGGP_03463","DHOHKKHL_03577","ADHCADEO_03455","PFJGCJHP_03371","CHLMDKPE_03451","NDDDIKEP_03483","ONGGBCKC_03467","PMNODAEP_03509","BOLLBKIO_03303","IHJLPFIO_03512","HIIAAINO_02981","MGKDKBAB_03492","MLIKFGOG_02892","LOFCOCJN_03492","LBMBHCAM_03093","PFAGPCGD_03201","KLCBHBCF_03528","OFBONHDF_03157","PBGIHMLN_02981","HGOBCNMM_00193","KNKCDEML_03237","7_refound_72","ANLDKFMB_02794","ILDLLNBB_00004","11_refound_105","ICFJOHJH_03133","CBGFCALD_02824","MJNIIBMG_03030","OEMCECNB_02728","KALPBOHI_03246","AFDIGNOC_00315","AFDIGNOC_03677","AFDIGNOC_03445","DHOHKKHL_03443","ADHCADEO_03267","MHIPNNDE_01002","PFJGCJHP_02256","CHLMDKPE_02817","NDDDIKEP_03591","ONGGBCKC_03586","PMNODAEP_03604","IHJLPFIO_03603","HIIAAINO_02401","39_refound_340","MLIKFGOG_03052","41_refound_348","LBMBHCAM_01812","PFAGPCGD_02846","KLCBHBCF_03163","OFBONHDF_03580","HGOBCNMM_03213","FBLKDDDN_01912","GGMENCCH_03241","KNKCDEML_03305","GJADGJJH_03234","NBEGIDKM_03553","MAEDJJLC_03129","ANLDKFMB_03239","ALMPNPLM_03292","ILDLLNBB_03262","MDOHONPH_03226","ICFJOHJH_03047","CBGFCALD_03408","LBEKBKDJ_03235","BAIOBGAL_03224","OEMCECNB_03604","HIKCDMLK_03621","CNFKGNAC_01730","AFDIGNOC_03369","ANBHONDD_03219","NDEKODAF_03014","DPBEEGGP_03146","DHOHKKHL_03699","ADHCADEO_02970","MHIPNNDE_03022","PFJGCJHP_03558","GGAGLIPG_02969","PNCEMHLO_03104","PBGIHMLN_03004","HGOBCNMM_03220","FBLKDDDN_01905","GGMENCCH_03234","KNKCDEML_03312","GJADGJJH_03241","NBEGIDKM_03642","MAEDJJLC_03136","ANLDKFMB_03232","ALMPNPLM_03413","ILDLLNBB_03253","MDOHONPH_03217","CBGFCALD_03401","LBEKBKDJ_03242","BAIOBGAL_03218","OEMCECNB_03597","HIKCDMLK_03627","CNFKGNAC_01738","AFDIGNOC_03374","ANBHONDD_02965","NDEKODAF_03021","DPBEEGGP_03299","DHOHKKHL_03693","ADHCADEO_02962","MHIPNNDE_03029","AMNKGELI_01305","PFJGCJHP_03603","GGAGLIPG_02976","PNCEMHLO_03097","PBGIHMLN_03651","FBLKDDDN_00254","GGMENCCH_02270","KNKCDEML_01710","GJADGJJH_02270","NBEGIDKM_00969","MAEDJJLC_00955","ANLDKFMB_00948","ILDLLNBB_01285","MDOHONPH_01017","CBGFCALD_01228","LBEKBKDJ_00955","BAIOBGAL_00264","OEMCECNB_02240","CNFKGNAC_01599","PFJGCJHP_01028","NDDDIKEP_01206","ONGGBCKC_01910","PMNODAEP_02877","BOLLBKIO_01180","IHJLPFIO_01710","HIIAAINO_02011","MGKDKBAB_01259","MLIKFGOG_02011","LOFCOCJN_01259","LBMBHCAM_00928","PFAGPCGD_01595","KLCBHBCF_01411","OFBONHDF_01498","PBGIHMLN_01611","HGOBCNMM_02089","FBLKDDDN_01886","GGMENCCH_03215","KNKCDEML_03482","GJADGJJH_03260","NBEGIDKM_03378","MAEDJJLC_03155","PPAMLKML_03073","ANLDKFMB_02529","ILDLLNBB_03246","MDOHONPH_03191","LBEKBKDJ_03261","MJNIIBMG_02379","BAIOBGAL_03196","KALPBOHI_03280","HIKCDMLK_02645","AFDIGNOC_03400","ANBHONDD_02986","DHOHKKHL_03090","ADHCADEO_03281","MHIPNNDE_03086","28_refound_279","CHLMDKPE_03202","GGAGLIPG_03318","GGAGLIPG_03567","PNCEMHLO_03301","PNCEMHLO_03578","HIIAAINO_03148","MLIKFGOG_03252","LBMBHCAM_03028","PBGIHMLN_03705","PBGIHMLN_03654","HGOBCNMM_01552","PPAMLKML_00003","ALMPNPLM_01926","CBGFCALD_00839","BAIOBGAL_00724","OEMCECNB_01851","KALPBOHI_03549","HIKCDMLK_03109","CNFKGNAC_00884","AFDIGNOC_02356","NDEKODAF_00370","DHOHKKHL_02333","ADHCADEO_00997","MHIPNNDE_02139","AMNKGELI_01491","CHLMDKPE_00443","NDDDIKEP_01593","ONGGBCKC_01060","PMNODAEP_01321","BOLLBKIO_01567","GGAGLIPG_01299","IHJLPFIO_01323","PNCEMHLO_01703","MGKDKBAB_01646","LOFCOCJN_01646","PFAGPCGD_01208","KLCBHBCF_01798","OFBONHDF_01111","PBGIHMLN_01099","3_refound_26","PPAMLKML_03562","ALMPNPLM_03744","ILDLLNBB_03135","MDOHONPH_03310","ICFJOHJH_03515","KALPBOHI_03373","HIKCDMLK_03507","AFDIGNOC_03351","ANBHONDD_03595","DPBEEGGP_03478","DHOHKKHL_03582","ADHCADEO_03460","PFJGCJHP_03366","CHLMDKPE_03456","NDDDIKEP_03478","ONGGBCKC_03472","PMNODAEP_03504","BOLLBKIO_03298","IHJLPFIO_03507","HIIAAINO_02976","MGKDKBAB_03487","MLIKFGOG_02887","LOFCOCJN_03487","LBMBHCAM_03088","PFAGPCGD_03196","KLCBHBCF_03533","OFBONHDF_03162","PBGIHMLN_02986","HGOBCNMM_03201","ANLDKFMB_03250","ILDLLNBB_03327","MDOHONPH_03336","HIKCDMLK_03859","CNFKGNAC_01719","AFDIGNOC_03358","ANBHONDD_03482","NDEKODAF_03003","DPBEEGGP_03135","DHOHKKHL_03778","ADHCADEO_02983","MHIPNNDE_03010","NDDDIKEP_03273","ONGGBCKC_03441","PMNODAEP_03493","BOLLBKIO_03290","GGAGLIPG_02956","IHJLPFIO_03264","PNCEMHLO_03117","HIIAAINO_03383","MGKDKBAB_03519","MLIKFGOG_03427","LOFCOCJN_03519","LBMBHCAM_03080","PFAGPCGD_03188","KLCBHBCF_03545","OFBONHDF_03170","PBGIHMLN_02994","PPAMLKML_03559","ANLDKFMB_03253","ALMPNPLM_03506","ICFJOHJH_03504","KALPBOHI_03396","HIKCDMLK_03503","CNFKGNAC_01716","AFDIGNOC_03355","ANBHONDD_03478","NDEKODAF_03000","DPBEEGGP_03131","DHOHKKHL_03586","ADHCADEO_02986","PFJGCJHP_03362","CHLMDKPE_03484","NDDDIKEP_03277","ONGGBCKC_03437","PMNODAEP_03497","BOLLBKIO_03294","IHJLPFIO_03260","HIIAAINO_03387","MGKDKBAB_03523","MLIKFGOG_03431","LOFCOCJN_03523","LBMBHCAM_03084","PFAGPCGD_03192","KLCBHBCF_03549","OFBONHDF_03166","PBGIHMLN_02990","KNKCDEML_03813","PPAMLKML_00255","ALMPNPLM_01696","ICFJOHJH_01380","CBGFCALD_01070","BAIOBGAL_00432","OEMCECNB_02082","KALPBOHI_01253","CNFKGNAC_00654","AFDIGNOC_02586","ANBHONDD_00166","DPBEEGGP_00422","CHLMDKPE_00214","NDDDIKEP_01364","ONGGBCKC_00831","PMNODAEP_01550","BOLLBKIO_01338","GGAGLIPG_01531","IHJLPFIO_01552","PNCEMHLO_01471","HIIAAINO_01853","MGKDKBAB_01417","MLIKFGOG_01853","LOFCOCJN_01417","LBMBHCAM_00770","PFAGPCGD_01437","KLCBHBCF_01569","OFBONHDF_01340","PBGIHMLN_01391","KNKCDEML_03814","PPAMLKML_00254","ALMPNPLM_01697","ICFJOHJH_01381","CBGFCALD_01069","BAIOBGAL_00433","OEMCECNB_02081","KALPBOHI_01252","CNFKGNAC_00655","AFDIGNOC_02585","ANBHONDD_00167","DPBEEGGP_00423","CHLMDKPE_00215","NDDDIKEP_01365","ONGGBCKC_00832","PMNODAEP_01549","BOLLBKIO_01339","GGAGLIPG_01530","IHJLPFIO_01551","PNCEMHLO_01472","HIIAAINO_01852","MGKDKBAB_01418","MLIKFGOG_01852","LOFCOCJN_01418","LBMBHCAM_00769","PFAGPCGD_01436","KLCBHBCF_01570","OFBONHDF_01339","PBGIHMLN_01390","HGOBCNMM_03518","FBLKDDDN_03330","GGMENCCH_03433","KNKCDEML_03316","GJADGJJH_03377","MAEDJJLC_03338","PPAMLKML_03100","ANLDKFMB_03209","ANLDKFMB_03381","ALMPNPLM_03487","ILDLLNBB_00028","MDOHONPH_00024","ICFJOHJH_03169","CBGFCALD_01974","LBEKBKDJ_03415","MJNIIBMG_02583","BAIOBGAL_03476","OEMCECNB_03196","KALPBOHI_02935","HIKCDMLK_00585","AFDIGNOC_00457","DHOHKKHL_01070","ADHCADEO_03428","MHIPNNDE_01643","PFJGCJHP_00185","CHLMDKPE_00970","GGAGLIPG_03179","PNCEMHLO_00727","PBGIHMLN_03559","HGOBCNMM_03579","FBLKDDDN_03323","GGMENCCH_03440","KNKCDEML_03323","GJADGJJH_03384","NBEGIDKM_01115","MAEDJJLC_03331","PPAMLKML_03094","ANLDKFMB_03374","ALMPNPLM_03492","ILDLLNBB_00033","MDOHONPH_00029","ICFJOHJH_03164","CBGFCALD_01968","LBEKBKDJ_03408","MJNIIBMG_02577","BAIOBGAL_01807","OEMCECNB_03191","KALPBOHI_02941","HIKCDMLK_00591","AFDIGNOC_00466","DHOHKKHL_01076","ADHCADEO_02173","MHIPNNDE_01636","PFJGCJHP_00179","CHLMDKPE_00965","GGAGLIPG_03186","PNCEMHLO_00734","HGOBCNMM_00906","FBLKDDDN_01862","GGMENCCH_00012","KNKCDEML_03619","GJADGJJH_01304","NBEGIDKM_01124","MAEDJJLC_01220","PPAMLKML_01697","ANLDKFMB_03426","ALMPNPLM_02480","ILDLLNBB_00040","MDOHONPH_00036","ICFJOHJH_01176","CBGFCALD_01962","LBEKBKDJ_02072","MJNIIBMG_02566","BAIOBGAL_01800","OEMCECNB_03184","KALPBOHI_01019","HIKCDMLK_00597","AFDIGNOC_00482","DHOHKKHL_01082","ADHCADEO_02164","MHIPNNDE_01630","PFJGCJHP_00172","CHLMDKPE_02234","GGAGLIPG_00720","PNCEMHLO_00741","HGOBCNMM_00903","FBLKDDDN_01866","GGMENCCH_00008","KNKCDEML_03662","GJADGJJH_01308","NBEGIDKM_01121","MAEDJJLC_01216","PPAMLKML_03091","ANLDKFMB_03430","ALMPNPLM_02487","ILDLLNBB_00036","MDOHONPH_00032","ICFJOHJH_03379","CBGFCALD_01965","LBEKBKDJ_02076","MJNIIBMG_02569","BAIOBGAL_01804","OEMCECNB_03188","KALPBOHI_02945","HIKCDMLK_00594","AFDIGNOC_00479","DHOHKKHL_01079","ADHCADEO_02167","MHIPNNDE_01633","PFJGCJHP_00176","CHLMDKPE_00963","GGAGLIPG_03189","PNCEMHLO_00737","HGOBCNMM_00907","FBLKDDDN_01861","GGMENCCH_00013","KNKCDEML_03620","GJADGJJH_01303","NBEGIDKM_01125","MAEDJJLC_01221","PPAMLKML_01698","ANLDKFMB_03425","ALMPNPLM_02479","ILDLLNBB_00041","MDOHONPH_00037","ICFJOHJH_01175","CBGFCALD_01961","LBEKBKDJ_02071","MJNIIBMG_02565","BAIOBGAL_01799","OEMCECNB_03652","KALPBOHI_01018","HIKCDMLK_00598","AFDIGNOC_00483","DHOHKKHL_01083","ADHCADEO_02163","MHIPNNDE_01629","PFJGCJHP_00171","CHLMDKPE_02235","GGAGLIPG_00721","PNCEMHLO_00742","HGOBCNMM_00904","FBLKDDDN_01865","GGMENCCH_00009","3_refound_34","GJADGJJH_01307","NBEGIDKM_01122","MAEDJJLC_01217","PPAMLKML_03090","ANLDKFMB_03429","ALMPNPLM_02486","ILDLLNBB_00037","MDOHONPH_00033","ICFJOHJH_03378","CBGFCALD_01964","LBEKBKDJ_02075","MJNIIBMG_02568","BAIOBGAL_01803","OEMCECNB_03187","KALPBOHI_02946","HIKCDMLK_00595","AFDIGNOC_00480","DHOHKKHL_01080","ADHCADEO_02166","MHIPNNDE_01632","PFJGCJHP_00175","CHLMDKPE_00962","GGAGLIPG_03190","PNCEMHLO_00738","HGOBCNMM_03251","FBLKDDDN_03239","GGMENCCH_03117","KNKCDEML_02788","GJADGJJH_03116","NBEGIDKM_02837","MAEDJJLC_03096","PPAMLKML_02887","ANLDKFMB_03440","ALMPNPLM_03522","ICFJOHJH_03090","CBGFCALD_02814","LBEKBKDJ_03117","MJNIIBMG_03457","OEMCECNB_03331","KALPBOHI_03297","HIKCDMLK_03554","CNFKGNAC_01697","AFDIGNOC_03643","ANBHONDD_03323","NDEKODAF_02269","DPBEEGGP_03115","DHOHKKHL_03626","ADHCADEO_03698","MHIPNNDE_03006","CHLMDKPE_03059","GGAGLIPG_03202","PNCEMHLO_03437","HGOBCNMM_03215","FBLKDDDN_01910","GGMENCCH_03239","KNKCDEML_03307","GJADGJJH_03236","NBEGIDKM_03647","MAEDJJLC_03131","ANLDKFMB_03237","ALMPNPLM_03290","ILDLLNBB_03260","MDOHONPH_03224","CBGFCALD_03406","LBEKBKDJ_03237","BAIOBGAL_03222","OEMCECNB_03602","HIKCDMLK_03623","CNFKGNAC_01732","AFDIGNOC_03371","ANBHONDD_03221","NDEKODAF_03016","DPBEEGGP_03148","DHOHKKHL_03697"
,"ADHCADEO_02967","MHIPNNDE_03024","AMNKGELI_01310","GGAGLIPG_02971","PNCEMHLO_03102","PBGIHMLN_03006","PPAMLKML_03547","ALMPNPLM_03688","ILDLLNBB_03132","MDOHONPH_03307","ICFJOHJH_03484","KALPBOHI_03370","HIKCDMLK_03510","AFDIGNOC_03348","ANBHONDD_03609","DPBEEGGP_03461","DHOHKKHL_03579","ADHCADEO_03457","PFJGCJHP_03369","CHLMDKPE_03453","NDDDIKEP_03481","ONGGBCKC_03469","PMNODAEP_03507","BOLLBKIO_03301","IHJLPFIO_03510","HIIAAINO_02979","MGKDKBAB_03490","MLIKFGOG_02890","LOFCOCJN_03490","LBMBHCAM_03091","PFAGPCGD_03199","KLCBHBCF_03530","OFBONHDF_03159","PBGIHMLN_02983","HGOBCNMM_03214","FBLKDDDN_01911","GGMENCCH_03240","KNKCDEML_03306","GJADGJJH_03235","NBEGIDKM_03554","MAEDJJLC_03130","ANLDKFMB_03238","ALMPNPLM_03291","ILDLLNBB_03261","MDOHONPH_03225","CBGFCALD_03407","LBEKBKDJ_03236","BAIOBGAL_03223","OEMCECNB_03603","HIKCDMLK_03622","CNFKGNAC_01731","AFDIGNOC_03370","ANBHONDD_03220","NDEKODAF_03015","DPBEEGGP_03147","DHOHKKHL_03698","ADHCADEO_02969","MHIPNNDE_03023","PFJGCJHP_03557","GGAGLIPG_02970","PNCEMHLO_03103","PBGIHMLN_03005","KNKCDEML_03964","PPAMLKML_03563","ILDLLNBB_03134","MDOHONPH_03309","ICFJOHJH_03516","KALPBOHI_03372","HIKCDMLK_03508","AFDIGNOC_03350","ANBHONDD_03594","DPBEEGGP_03477","DHOHKKHL_03581","ADHCADEO_03459","PFJGCJHP_03367","CHLMDKPE_03455","NDDDIKEP_03479","ONGGBCKC_03471","PMNODAEP_03505","BOLLBKIO_03299","IHJLPFIO_03508","HIIAAINO_02977","MGKDKBAB_03488","MLIKFGOG_02888","LOFCOCJN_03488","LBMBHCAM_03089","PFAGPCGD_03197","KLCBHBCF_03532","OFBONHDF_03161","PBGIHMLN_02985","HGOBCNMM_03217","FBLKDDDN_01908","GGMENCCH_03237","KNKCDEML_03309","GJADGJJH_03238","NBEGIDKM_03645","MAEDJJLC_03133","ANLDKFMB_03235","ALMPNPLM_03416","ILDLLNBB_03258","MDOHONPH_03222","CBGFCALD_03404","LBEKBKDJ_03239","BAIOBGAL_03220","OEMCECNB_03600","HIKCDMLK_03624","CNFKGNAC_01734","AFDIGNOC_03373","ANBHONDD_03223","NDEKODAF_03018","DPBEEGGP_03150","DHOHKKHL_03696","ADHCADEO_02965","MHIPNNDE_03026","AMNKGELI_01308","GGAGLIPG_02973","PNCEMHLO_03100","PBGIHMLN_03008","PPAMLKML_03564","ALMPNPLM_03689","ILDLLNBB_03133","MDOHONPH_03308","ICFJOHJH_03517","KALPBOHI_03371","HIKCDMLK_03509","AFDIGNOC_03349","ANBHONDD_03550","DPBEEGGP_03460","DHOHKKHL_03580","ADHCADEO_03458","PFJGCJHP_03368","CHLMDKPE_03454","NDDDIKEP_03480","ONGGBCKC_03470","PMNODAEP_03506","BOLLBKIO_03300","IHJLPFIO_03509","HIIAAINO_02978","MGKDKBAB_03489","MLIKFGOG_02889","LOFCOCJN_03489","LBMBHCAM_03090","PFAGPCGD_03198","KLCBHBCF_03531","OFBONHDF_03160","PBGIHMLN_02984","HGOBCNMM_03575","FBLKDDDN_03327","GGMENCCH_03436","KNKCDEML_03319","GJADGJJH_03380","NBEGIDKM_01110","MAEDJJLC_03335","PPAMLKML_03098","ANLDKFMB_03378","ALMPNPLM_03488","ILDLLNBB_00029","MDOHONPH_00025","ICFJOHJH_03168","CBGFCALD_01972","LBEKBKDJ_03412","MJNIIBMG_02582","BAIOBGAL_01811","OEMCECNB_03195","KALPBOHI_02937","HIKCDMLK_00587","AFDIGNOC_00462","DHOHKKHL_01072","ADHCADEO_02177","MHIPNNDE_01640","PFJGCJHP_00183","CHLMDKPE_00969","GGAGLIPG_03182","PNCEMHLO_00730","HGOBCNMM_02288","FBLKDDDN_03259","GGMENCCH_03314","KNKCDEML_02972","GJADGJJH_03297","NBEGIDKM_02859","MAEDJJLC_03204","ICFJOHJH_03073","LBEKBKDJ_03293","MJNIIBMG_03108","KALPBOHI_03474","HIKCDMLK_03617","DHOHKKHL_03676","ADHCADEO_03763","MHIPNNDE_01808","AMNKGELI_01644","CHLMDKPE_03381","NDDDIKEP_03188","ONGGBCKC_03120","PMNODAEP_03194","BOLLBKIO_03222","IHJLPFIO_03173","MGKDKBAB_03220","LOFCOCJN_03220","PFAGPCGD_03120","KLCBHBCF_03247","OFBONHDF_03278","PPAMLKML_03561","ALMPNPLM_03745","ILDLLNBB_03136","MDOHONPH_03311","ICFJOHJH_03514","HIKCDMLK_03506","AFDIGNOC_03352","ANBHONDD_03602","DPBEEGGP_03483","DHOHKKHL_03583","ADHCADEO_03461","PFJGCJHP_02277","CHLMDKPE_03457","NDDDIKEP_03477","ONGGBCKC_03473","PMNODAEP_03503","BOLLBKIO_03297","IHJLPFIO_03506","HIIAAINO_02975","MGKDKBAB_03486","MLIKFGOG_02886","LOFCOCJN_03486","LBMBHCAM_03087","PFAGPCGD_03195","KLCBHBCF_03534","OFBONHDF_03163","PBGIHMLN_02987","PPAMLKML_03560","ALMPNPLM_03505","MDOHONPH_03291","ICFJOHJH_03505","KALPBOHI_03397","HIKCDMLK_03504","AFDIGNOC_03354","ANBHONDD_02907","DPBEEGGP_03187","DHOHKKHL_03585","ADHCADEO_02987","PFJGCJHP_03363","30_refound_301","NDDDIKEP_03278","ONGGBCKC_03436","PMNODAEP_03498","BOLLBKIO_03295","IHJLPFIO_03259","HIIAAINO_03388","MGKDKBAB_03524","MLIKFGOG_03432","LOFCOCJN_03524","LBMBHCAM_03085","PFAGPCGD_03193","KLCBHBCF_03550","OFBONHDF_03165","PBGIHMLN_02989","HGOBCNMM_03202","ANLDKFMB_03249","ILDLLNBB_03328","MDOHONPH_03335","CNFKGNAC_01720","AFDIGNOC_03359","ANBHONDD_03483","23_refound_214_stop","DPBEEGGP_03136","ADHCADEO_02982","MHIPNNDE_03011","NDDDIKEP_03272","ONGGBCKC_03442","PMNODAEP_03492","BOLLBKIO_03289","GGAGLIPG_02957","IHJLPFIO_03265","PNCEMHLO_03116","HIIAAINO_03382","MGKDKBAB_03518","MLIKFGOG_03426","LOFCOCJN_03518","LBMBHCAM_03079","PFAGPCGD_03187","KLCBHBCF_03544","OFBONHDF_03171","PBGIHMLN_02995","HGOBCNMM_03577","FBLKDDDN_03325","GGMENCCH_03438","KNKCDEML_03321","GJADGJJH_03382","MAEDJJLC_03333","PPAMLKML_03096","ANLDKFMB_03376","ALMPNPLM_03490","ILDLLNBB_00031","MDOHONPH_00027","ICFJOHJH_03166","CBGFCALD_01970","LBEKBKDJ_03410","BAIOBGAL_01809","OEMCECNB_03193","KALPBOHI_02939","HIKCDMLK_00589","AFDIGNOC_00464","DHOHKKHL_01074","ADHCADEO_02175","MHIPNNDE_01638","PFJGCJHP_00181","CHLMDKPE_00967","GGAGLIPG_03184","PNCEMHLO_00732","HGOBCNMM_03576","FBLKDDDN_03326","GGMENCCH_03437","KNKCDEML_03320","GJADGJJH_03381","MAEDJJLC_03334","PPAMLKML_03097","ANLDKFMB_03377","ALMPNPLM_03489","ILDLLNBB_00030","MDOHONPH_00026","ICFJOHJH_03167","CBGFCALD_01971","LBEKBKDJ_03411","BAIOBGAL_01810","OEMCECNB_03194","KALPBOHI_02938","HIKCDMLK_00588","AFDIGNOC_00463","DHOHKKHL_01073","ADHCADEO_02176","MHIPNNDE_01639","PFJGCJHP_00182","CHLMDKPE_00968","GGAGLIPG_03183","PNCEMHLO_00731","HGOBCNMM_03578","FBLKDDDN_03324","GGMENCCH_03439","KNKCDEML_03322","GJADGJJH_03383","MAEDJJLC_03332","PPAMLKML_03095","ANLDKFMB_03375","ALMPNPLM_03491","ILDLLNBB_00032","MDOHONPH_00028","ICFJOHJH_03165","CBGFCALD_01969","LBEKBKDJ_03409","BAIOBGAL_01808","OEMCECNB_03192","KALPBOHI_02940","HIKCDMLK_00590","AFDIGNOC_00465","DHOHKKHL_01075","ADHCADEO_02174","MHIPNNDE_01637","PFJGCJHP_00180","CHLMDKPE_00966","GGAGLIPG_03185","PNCEMHLO_00733","ALMPNPLM_03504","MDOHONPH_03292","ICFJOHJH_03512","KALPBOHI_03538","HIKCDMLK_03505","AFDIGNOC_03353","ANBHONDD_03611","DPBEEGGP_03489","DHOHKKHL_03584","ADHCADEO_02988","PFJGCJHP_03364","CHLMDKPE_03485","NDDDIKEP_03279","ONGGBCKC_03435","PMNODAEP_03499","BOLLBKIO_03296","IHJLPFIO_03258","HIIAAINO_03389","MGKDKBAB_03525","MLIKFGOG_03433","LOFCOCJN_03525","LBMBHCAM_03086","PFAGPCGD_03194","KLCBHBCF_03551","OFBONHDF_03164","PBGIHMLN_02988","HGOBCNMM_00905","FBLKDDDN_01864","FBLKDDDN_01863","GGMENCCH_00010","GGMENCCH_00011","KNKCDEML_03617","KNKCDEML_03618","GJADGJJH_01305","GJADGJJH_01306","NBEGIDKM_01123","MAEDJJLC_01219","MAEDJJLC_01218","PPAMLKML_01696","ANLDKFMB_03428","ANLDKFMB_03427","ALMPNPLM_02481","ILDLLNBB_00038","ILDLLNBB_00039","MDOHONPH_00034","MDOHONPH_00035","ICFJOHJH_01177","LBEKBKDJ_02074","LBEKBKDJ_02073","MJNIIBMG_02567","BAIOBGAL_01802","BAIOBGAL_01801","OEMCECNB_03185","OEMCECNB_03186","KALPBOHI_01020","HIKCDMLK_00596","AFDIGNOC_00481","DHOHKKHL_01081","ADHCADEO_02165","MHIPNNDE_01631","PFJGCJHP_00174","PFJGCJHP_00173","PNCEMHLO_00740","PNCEMHLO_00739","HGOBCNMM_02751","FBLKDDDN_01671","GGMENCCH_00203","KNKCDEML_02427","GJADGJJH_01113","NBEGIDKM_01314","MAEDJJLC_01411","PPAMLKML_01890","ANLDKFMB_02242","ILDLLNBB_00233","MDOHONPH_00229","CBGFCALD_01769","LBEKBKDJ_01880","MJNIIBMG_01056","BAIOBGAL_01608","OEMCECNB_00192","KALPBOHI_00825","HIKCDMLK_01379","DHOHKKHL_01272","ADHCADEO_02276","MHIPNNDE_02929","PFJGCJHP_02384","CHLMDKPE_02429","GGAGLIPG_00911","PNCEMHLO_00932","HGOBCNMM_02750","FBLKDDDN_01672","GGMENCCH_00202","KNKCDEML_02428","GJADGJJH_01114","NBEGIDKM_01313","MAEDJJLC_01410","PPAMLKML_01889","ANLDKFMB_02243","ILDLLNBB_00232","MDOHONPH_00228","CBGFCALD_01770","LBEKBKDJ_01881","MJNIIBMG_01057","BAIOBGAL_01609","OEMCECNB_00191","KALPBOHI_00826","HIKCDMLK_01380","DHOHKKHL_01271","ADHCADEO_02277","MHIPNNDE_02930","PFJGCJHP_02383","CHLMDKPE_02428","GGAGLIPG_00910","PNCEMHLO_00931","HGOBCNMM_03212","FBLKDDDN_01914","GGMENCCH_03243","KNKCDEML_03303","GJADGJJH_03232","NBEGIDKM_03551","MAEDJJLC_03127","ANLDKFMB_03240","ILDLLNBB_03263","MDOHONPH_03227","CBGFCALD_03410","LBEKBKDJ_03233","BAIOBGAL_03225","OEMCECNB_03606","HIKCDMLK_03619","CNFKGNAC_01729","AFDIGNOC_03368","ANBHONDD_03218","DPBEEGGP_03145","DHOHKKHL_03701","ADHCADEO_02971","MHIPNNDE_03021","PFJGCJHP_03559","GGAGLIPG_02967","PNCEMHLO_03106","PPAMLKML_02949","ANLDKFMB_02941","CBGFCALD_02864","OEMCECNB_02688","CNFKGNAC_01644","ANBHONDD_03188","NDEKODAF_02091","DPBEEGGP_03258","AMNKGELI_02968","PFJGCJHP_01200","CHLMDKPE_03172","NDDDIKEP_03402","ONGGBCKC_03413","PMNODAEP_03436","BOLLBKIO_03491","IHJLPFIO_03487","HIIAAINO_03259","MGKDKBAB_03439","MLIKFGOG_03338","LOFCOCJN_03439","LBMBHCAM_03188","PFAGPCGD_03389","KLCBHBCF_03476","OFBONHDF_03497","PBGIHMLN_02720","PPAMLKML_02950","ANLDKFMB_02942","CBGFCALD_02865","OEMCECNB_02687","CNFKGNAC_01643","ANBHONDD_03189","NDEKODAF_02092","DPBEEGGP_03259","AMNKGELI_02967","PFJGCJHP_01199","CHLMDKPE_03173","NDDDIKEP_03403","ONGGBCKC_03412","PMNODAEP_03437","BOLLBKIO_03490","IHJLPFIO_03486","HIIAAINO_03260","MGKDKBAB_03440","MLIKFGOG_03339","LOFCOCJN_03440","LBMBHCAM_03189","PFAGPCGD_03388","KLCBHBCF_03475","OFBONHDF_03498","PBGIHMLN_02719","PPAMLKML_02948","ANLDKFMB_02940","CBGFCALD_02863","OEMCECNB_02689","CNFKGNAC_01645","ANBHONDD_03187","NDEKODAF_02090","DPBEEGGP_03257","AMNKGELI_02969","PFJGCJHP_01201","CHLMDKPE_03171","NDDDIKEP_03401","ONGGBCKC_03414","PMNODAEP_03435","BOLLBKIO_03492","IHJLPFIO_03488","HIIAAINO_03258","MGKDKBAB_03438","MLIKFGOG_03337","LOFCOCJN_03438","LBMBHCAM_03187","PFAGPCGD_03390","KLCBHBCF_03477","OFBONHDF_03496","PBGIHMLN_02721","KNKCDEML_03230","CBGFCALD_03279","BAIOBGAL_03469","OEMCECNB_03374","HIKCDMLK_03501","AFDIGNOC_03356","ANBHONDD_03479","DPBEEGGP_03132","DHOHKKHL_03588","ADHCADEO_02985","NDDDIKEP_03276","ONGGBCKC_03438","PMNODAEP_03496","BOLLBKIO_03293"
,"IHJLPFIO_03261","HIIAAINO_03386","MGKDKBAB_03522","MLIKFGOG_03430","LOFCOCJN_03522","LBMBHCAM_03083","PFAGPCGD_03191","KLCBHBCF_03548","OFBONHDF_03167","PBGIHMLN_02991","HGOBCNMM_03218","FBLKDDDN_01907","GGMENCCH_03236","KNKCDEML_03310","GJADGJJH_03239","NBEGIDKM_03644","MAEDJJLC_03134","ANLDKFMB_03233","ALMPNPLM_03415","ILDLLNBB_03256","MDOHONPH_03220","CBGFCALD_03403","LBEKBKDJ_03240","OEMCECNB_03599","HIKCDMLK_03625","CNFKGNAC_01736","NDEKODAF_03019","DHOHKKHL_03695","ADHCADEO_02964","MHIPNNDE_03027","AMNKGELI_01307","GGAGLIPG_02974","PNCEMHLO_03099","PBGIHMLN_03009","HGOBCNMM_03219","FBLKDDDN_01906","GGMENCCH_03235","KNKCDEML_03311","GJADGJJH_03240","NBEGIDKM_03643","MAEDJJLC_03135","8_refound_75","ALMPNPLM_03414","ILDLLNBB_03255","MDOHONPH_03219","CBGFCALD_03402","LBEKBKDJ_03241","OEMCECNB_03598","HIKCDMLK_03626","CNFKGNAC_01737","NDEKODAF_03020","DHOHKKHL_03694","ADHCADEO_02963","MHIPNNDE_03028","AMNKGELI_01306","GGAGLIPG_02975","PNCEMHLO_03098","PBGIHMLN_03010","FBLKDDDN_03534","GGMENCCH_03537","KNKCDEML_03531","GJADGJJH_03535","NBEGIDKM_00507","MAEDJJLC_03257","ICFJOHJH_03076","CBGFCALD_03411","LBEKBKDJ_03541","OEMCECNB_03654","KALPBOHI_03263","HIKCDMLK_03591","AFDIGNOC_00412","ANBHONDD_01723","NDEKODAF_00569","DPBEEGGP_01148","DHOHKKHL_02330","PFJGCJHP_03560","CHLMDKPE_03045","HIIAAINO_02440","MLIKFGOG_03206","LBMBHCAM_03408","PBGIHMLN_02644","HGOBCNMM_03580","FBLKDDDN_03322","GGMENCCH_03441","KNKCDEML_03324","GJADGJJH_03385","MAEDJJLC_03330","PPAMLKML_03093","ANLDKFMB_03373","ILDLLNBB_00034","MDOHONPH_00030","ICFJOHJH_03163","CBGFCALD_01967","LBEKBKDJ_03407","BAIOBGAL_01806","OEMCECNB_03190","KALPBOHI_02942","AFDIGNOC_00467","ADHCADEO_02172","MHIPNNDE_01635","PFJGCJHP_00178","CHLMDKPE_00964","GGAGLIPG_03187","PNCEMHLO_00735","FBLKDDDN_03240","GGMENCCH_03333","KNKCDEML_03819","GJADGJJH_03316","NBEGIDKM_02840","MAEDJJLC_03223","ILDLLNBB_03116","MDOHONPH_03196","CBGFCALD_03182","LBEKBKDJ_03274","MJNIIBMG_03460","OEMCECNB_03034","NDDDIKEP_03499","ONGGBCKC_03451","PMNODAEP_03525","BOLLBKIO_03319","IHJLPFIO_03528","MGKDKBAB_03508","LOFCOCJN_03508","PFAGPCGD_03217","KLCBHBCF_03512","OFBONHDF_03141","PBGIHMLN_02965","ILDLLNBB_03131","MDOHONPH_03306","HIKCDMLK_03511","AFDIGNOC_03347","ANBHONDD_03607","DPBEEGGP_03462","DHOHKKHL_03578","ADHCADEO_03456","PFJGCJHP_03370","NDDDIKEP_03482","ONGGBCKC_03468","PMNODAEP_03508","BOLLBKIO_03302","IHJLPFIO_03511","HIIAAINO_02980","MGKDKBAB_03491","MLIKFGOG_02891","LOFCOCJN_03491","LBMBHCAM_03092","PFAGPCGD_03200","KLCBHBCF_03529","OFBONHDF_03158","PBGIHMLN_02982","FBLKDDDN_03532","GGMENCCH_03533","KNKCDEML_03906","GJADGJJH_03533","NBEGIDKM_02838","MAEDJJLC_03430","PPAMLKML_02988","ILDLLNBB_03118","MDOHONPH_03293","ICFJOHJH_03221","CBGFCALD_03183","LBEKBKDJ_03532","MJNIIBMG_03458","BAIOBGAL_03505","OEMCECNB_03033","KALPBOHI_03531","AFDIGNOC_03654","ANBHONDD_03314","NDEKODAF_02260","DPBEEGGP_03106","CHLMDKPE_03158","PBGIHMLN_03473","PPAMLKML_02954","CBGFCALD_02869","OEMCECNB_02683","CNFKGNAC_01639","NDEKODAF_02096","AMNKGELI_03159","PFJGCJHP_01195","CHLMDKPE_03177","NDDDIKEP_03407","ONGGBCKC_03408","PMNODAEP_03441","BOLLBKIO_03486","IHJLPFIO_03482","HIIAAINO_03264","MGKDKBAB_03444","MLIKFGOG_03343","LOFCOCJN_03444","LBMBHCAM_03193","PFAGPCGD_03384","KLCBHBCF_03471","OFBONHDF_03502","PBGIHMLN_02715","PPAMLKML_02962","CBGFCALD_02877","OEMCECNB_02675","CNFKGNAC_01631","NDEKODAF_02104","AMNKGELI_03151","PFJGCJHP_01187","CHLMDKPE_03185","NDDDIKEP_03415","ONGGBCKC_03400","PMNODAEP_03449","BOLLBKIO_03478","IHJLPFIO_03474","HIIAAINO_03272","MGKDKBAB_03452","MLIKFGOG_03351","LOFCOCJN_03452","LBMBHCAM_03201","PFAGPCGD_03376","KLCBHBCF_03463","OFBONHDF_03510","PBGIHMLN_02706","PBGIHMLN_02707","PPAMLKML_02958","CBGFCALD_02873","OEMCECNB_02679","CNFKGNAC_01635","NDEKODAF_02100","AMNKGELI_03155","PFJGCJHP_01191","CHLMDKPE_03181","NDDDIKEP_03411","ONGGBCKC_03404","PMNODAEP_03445","BOLLBKIO_03482","IHJLPFIO_03478","HIIAAINO_03268","MGKDKBAB_03448","MLIKFGOG_03347","LOFCOCJN_03448","LBMBHCAM_03197","PFAGPCGD_03380","KLCBHBCF_03467","OFBONHDF_03506","PBGIHMLN_02711","HGOBCNMM_01144","ANLDKFMB_02634","ILDLLNBB_01498","MDOHONPH_01954","ICFJOHJH_00538","MHIPNNDE_00160","AMNKGELI_00022","NDDDIKEP_00485","ONGGBCKC_00583","PMNODAEP_00547","BOLLBKIO_00611","GGAGLIPG_00537","IHJLPFIO_00611","PNCEMHLO_00184","HIIAAINO_00543","MGKDKBAB_00547","MLIKFGOG_00543","LOFCOCJN_00547","LBMBHCAM_01271","PFAGPCGD_00496","KLCBHBCF_00611","OFBONHDF_00611","HGOBCNMM_01143","ANLDKFMB_02635","ILDLLNBB_01499","MDOHONPH_01953","ICFJOHJH_00539","MHIPNNDE_00161","AMNKGELI_00021","NDDDIKEP_00486","ONGGBCKC_00584","PMNODAEP_00546","BOLLBKIO_00612","GGAGLIPG_00538","IHJLPFIO_00612","PNCEMHLO_00183","HIIAAINO_00544","MGKDKBAB_00546","MLIKFGOG_00544","LOFCOCJN_00546","LBMBHCAM_01272","PFAGPCGD_00497","KLCBHBCF_00612","OFBONHDF_00612","PPAMLKML_02951","CBGFCALD_02866","OEMCECNB_02686","CNFKGNAC_01642","NDEKODAF_02093","AMNKGELI_02966","PFJGCJHP_01198","CHLMDKPE_03174","NDDDIKEP_03404","ONGGBCKC_03411","PMNODAEP_03438","BOLLBKIO_03489","IHJLPFIO_03485","HIIAAINO_03261","MGKDKBAB_03441","MLIKFGOG_03340","LOFCOCJN_03441","LBMBHCAM_03190","PFAGPCGD_03387","KLCBHBCF_03474","OFBONHDF_03499","PBGIHMLN_02718","PPAMLKML_02956","CBGFCALD_02871","OEMCECNB_02681","CNFKGNAC_01637","NDEKODAF_02098","AMNKGELI_03157","PFJGCJHP_01193","CHLMDKPE_03179","NDDDIKEP_03409","ONGGBCKC_03406","PMNODAEP_03443","BOLLBKIO_03484","IHJLPFIO_03480","HIIAAINO_03266","MGKDKBAB_03446","MLIKFGOG_03345","LOFCOCJN_03446","LBMBHCAM_03195","PFAGPCGD_03382","KLCBHBCF_03469","OFBONHDF_03504","PBGIHMLN_02713","HGOBCNMM_01147","ANLDKFMB_02631","ILDLLNBB_01495","MDOHONPH_01957","ICFJOHJH_00535","MHIPNNDE_00157","AMNKGELI_00025","NDDDIKEP_00482","ONGGBCKC_00580","PMNODAEP_00550","BOLLBKIO_00608","GGAGLIPG_00534","IHJLPFIO_00608","PNCEMHLO_00187","HIIAAINO_00540","MGKDKBAB_00550","MLIKFGOG_00540","LOFCOCJN_00550","LBMBHCAM_01268","PFAGPCGD_00493","KLCBHBCF_00608","OFBONHDF_00608","HGOBCNMM_01145","ANLDKFMB_02633","ILDLLNBB_01497","MDOHONPH_01955","ICFJOHJH_00537","MHIPNNDE_00159","AMNKGELI_00023","NDDDIKEP_00484","ONGGBCKC_00582","PMNODAEP_00548","BOLLBKIO_00610","GGAGLIPG_00536","IHJLPFIO_00610","PNCEMHLO_00185","HIIAAINO_00542","MGKDKBAB_00548","MLIKFGOG_00542","LOFCOCJN_00548","LBMBHCAM_01270","PFAGPCGD_00495","KLCBHBCF_00610","OFBONHDF_00610","PPAMLKML_02953","CBGFCALD_02868","OEMCECNB_02684","CNFKGNAC_01640","NDEKODAF_02095","AMNKGELI_03160","PFJGCJHP_01196","CHLMDKPE_03176","NDDDIKEP_03406","ONGGBCKC_03409","PMNODAEP_03440","BOLLBKIO_03487","IHJLPFIO_03483","HIIAAINO_03263","MGKDKBAB_03443","MLIKFGOG_03342","LOFCOCJN_03443","LBMBHCAM_03192","PFAGPCGD_03385","KLCBHBCF_03472","OFBONHDF_03501","PBGIHMLN_02716","PPAMLKML_02955","CBGFCALD_02870","OEMCECNB_02682","CNFKGNAC_01638","NDEKODAF_02097","AMNKGELI_03158","PFJGCJHP_01194","CHLMDKPE_03178","NDDDIKEP_03408","ONGGBCKC_03407","PMNODAEP_03442","BOLLBKIO_03485","IHJLPFIO_03481","HIIAAINO_03265","MGKDKBAB_03445","MLIKFGOG_03344","LOFCOCJN_03445","LBMBHCAM_03194","PFAGPCGD_03383","KLCBHBCF_03470","OFBONHDF_03503","PBGIHMLN_02714","PPAMLKML_02957","CBGFCALD_02872","OEMCECNB_02680","CNFKGNAC_01636","NDEKODAF_02099","AMNKGELI_03156","PFJGCJHP_01192","CHLMDKPE_03180","NDDDIKEP_03410","ONGGBCKC_03405","PMNODAEP_03444","BOLLBKIO_03483","IHJLPFIO_03479","HIIAAINO_03267","MGKDKBAB_03447","MLIKFGOG_03346","LOFCOCJN_03447","LBMBHCAM_03196","PFAGPCGD_03381","KLCBHBCF_03468","OFBONHDF_03505","PBGIHMLN_02712","PPAMLKML_02961","CBGFCALD_02876","OEMCECNB_02676","CNFKGNAC_01632","NDEKODAF_02103","AMNKGELI_03152","PFJGCJHP_01188","CHLMDKPE_03184","NDDDIKEP_03414","ONGGBCKC_03401","PMNODAEP_03448","BOLLBKIO_03479","IHJLPFIO_03475","HIIAAINO_03271","MGKDKBAB_03451","MLIKFGOG_03350","LOFCOCJN_03451","LBMBHCAM_03200","PFAGPCGD_03377","KLCBHBCF_03464","OFBONHDF_03509","PBGIHMLN_02708","PPAMLKML_02960","CBGFCALD_02875","OEMCECNB_02677","CNFKGNAC_01633","NDEKODAF_02102","AMNKGELI_03153","PFJGCJHP_01189","CHLMDKPE_03183","NDDDIKEP_03413","ONGGBCKC_03402","PMNODAEP_03447","BOLLBKIO_03480","IHJLPFIO_03476","HIIAAINO_03270","MGKDKBAB_03450","MLIKFGOG_03349","LOFCOCJN_03450","LBMBHCAM_03199","PFAGPCGD_03378","KLCBHBCF_03465","OFBONHDF_03508","PBGIHMLN_02709","HGOBCNMM_01146","ANLDKFMB_02632","ILDLLNBB_01496","MDOHONPH_01956","ICFJOHJH_00536","MHIPNNDE_00158","AMNKGELI_00024","NDDDIKEP_00483","ONGGBCKC_00581","PMNODAEP_00549","BOLLBKIO_00609","GGAGLIPG_00535","IHJLPFIO_00609","PNCEMHLO_00186","HIIAAINO_00541","MGKDKBAB_00549","MLIKFGOG_00541","LOFCOCJN_00549","LBMBHCAM_01269","PFAGPCGD_00494","KLCBHBCF_00609","OFBONHDF_00609","PPAMLKML_02952","CBGFCALD_02867","OEMCECNB_02685","CNFKGNAC_01641","NDEKODAF_02094","AMNKGELI_03161","PFJGCJHP_01197","CHLMDKPE_03175","NDDDIKEP_03405","ONGGBCKC_03410","PMNODAEP_03439","BOLLBKIO_03488","IHJLPFIO_03484","HIIAAINO_03262","MGKDKBAB_03442","MLIKFGOG_03341","LOFCOCJN_03442","LBMBHCAM_03191","PFAGPCGD_03386","KLCBHBCF_03473","OFBONHDF_03500","PBGIHMLN_02717","PPAMLKML_02959","CBGFCALD_02874","OEMCECNB_02678","CNFKGNAC_01634","NDEKODAF_02101","AMNKGELI_03154","PFJGCJHP_01190","CHLMDKPE_03182","NDDDIKEP_03412","ONGGBCKC_03403","PMNODAEP_03446","BOLLBKIO_03481","IHJLPFIO_03477","HIIAAINO_03269","MGKDKBAB_03449","MLIKFGOG_03348","LOFCOCJN_03449","LBMBHCAM_03198","PFAGPCGD_03379","KLCBHBCF_03466","OFBONHDF_03507","PBGIHMLN_02710","FBLKDDDN_03258","GGMENCCH_03315","KNKCDEML_02973","GJADGJJH_03298","NBEGIDKM_02858","MAEDJJLC_03205","ALMPNPLM_03476","ILDLLNBB_03001","MDOHONPH_03066","ICFJOHJH_02592","LBEKBKDJ_03292","MJNIIBMG_03110","KALPBOHI_02888","HIKCDMLK_03046","AFDIGNOC_03162","ANBHONDD_03402","DPBEEGGP_03274","ADHCADEO_03384","CHLMDKPE_03506","GGAGLIPG_03590","PNCEMHLO_03598","HGOBCNMM_03517","FBLKDDDN_03329","GGMENCCH_03434","KNKCDEML_03317","GJADGJJH_03378","MAEDJJLC_03337","PPAMLKML_03099","ANLDKFMB_03380","CBGFCALD_01973","LBEKBKDJ_03414","BAIOBGAL_03477","KALPBOHI_02936","HIKCDMLK_00586","AFDIGNOC_02903","DHOHKKHL_01071","ADHCADEO_03429","MHIPNNDE_01642","PFJGCJHP_00184","GGAGLIPG_03180"
,"PNCEMHLO_00728","PBGIHMLN_03708","FBLKDDDN_03429","GGMENCCH_03409","KNKCDEML_03737","GJADGJJH_03423","NBEGIDKM_03193","MAEDJJLC_03240","CBGFCALD_03437","LBEKBKDJ_03454","MJNIIBMG_03147","BAIOBGAL_03359","OEMCECNB_03468","HIKCDMLK_02667","AFDIGNOC_03422","ANBHONDD_03426","DPBEEGGP_02364","DHOHKKHL_03068","ADHCADEO_02318","MHIPNNDE_03394","PFJGCJHP_02005","GGAGLIPG_03129","PNCEMHLO_02851","FBLKDDDN_03430","GGMENCCH_03408","KNKCDEML_03738","GJADGJJH_03424","NBEGIDKM_03192","MAEDJJLC_03241","CBGFCALD_03438","LBEKBKDJ_03453","MJNIIBMG_03148","BAIOBGAL_03360","OEMCECNB_03469","HIKCDMLK_02668","AFDIGNOC_03423","ANBHONDD_03425","DPBEEGGP_02365","DHOHKKHL_03067","ADHCADEO_02319","MHIPNNDE_03393","PFJGCJHP_02004","GGAGLIPG_03130","PNCEMHLO_02852","KNKCDEML_03838","PPAMLKML_00853","ANLDKFMB_01896","ALMPNPLM_00734","KALPBOHI_00722","NDDDIKEP_00906","ONGGBCKC_02591","PMNODAEP_00126","BOLLBKIO_02394","GGAGLIPG_01012","IHJLPFIO_01032","PNCEMHLO_01033","HIIAAINO_00962","MGKDKBAB_00126","MLIKFGOG_00962","LOFCOCJN_00126","LBMBHCAM_01690","PFAGPCGD_00917","KLCBHBCF_01032","OFBONHDF_02137","PBGIHMLN_01285","FBLKDDDN_00173","GGMENCCH_02351","KNKCDEML_01629","GJADGJJH_02351","NBEGIDKM_01052","MAEDJJLC_01036","ANLDKFMB_01029","CBGFCALD_01309","LBEKBKDJ_01036","MJNIIBMG_02287","BAIOBGAL_00179","OEMCECNB_02321","CNFKGNAC_01518","ANBHONDD_02272","NDEKODAF_00849","DPBEEGGP_00172","AMNKGELI_01045","PFJGCJHP_01109","HIIAAINO_02092","MLIKFGOG_02092","LBMBHCAM_01009","FBLKDDDN_03427","GGMENCCH_03411","KNKCDEML_03735","GJADGJJH_03421","NBEGIDKM_03195","MAEDJJLC_03238","CBGFCALD_03435","LBEKBKDJ_03456","MJNIIBMG_03145","BAIOBGAL_03357","OEMCECNB_03466","HIKCDMLK_02665","AFDIGNOC_03420","ANBHONDD_03428","DPBEEGGP_02362","DHOHKKHL_03070","ADHCADEO_02316","MHIPNNDE_03396","PFJGCJHP_02007","GGAGLIPG_03127","PNCEMHLO_02849","HGOBCNMM_02088","FBLKDDDN_01887","GGMENCCH_03216","KNKCDEML_03596","GJADGJJH_03259","NBEGIDKM_03379","MAEDJJLC_03154","ANLDKFMB_02533","ALMPNPLM_03365","CBGFCALD_03598","LBEKBKDJ_03260","BAIOBGAL_03197","OEMCECNB_03582","CNFKGNAC_03067","NDEKODAF_03085","MHIPNNDE_03087","AMNKGELI_01282","PFJGCJHP_03589","GGAGLIPG_02997","PNCEMHLO_03076","PBGIHMLN_03656","FBLKDDDN_03428","GGMENCCH_03410","KNKCDEML_03736","GJADGJJH_03422","NBEGIDKM_03194","MAEDJJLC_03239","CBGFCALD_03436","LBEKBKDJ_03455","MJNIIBMG_03146","BAIOBGAL_03358","OEMCECNB_03467","HIKCDMLK_02666","AFDIGNOC_03421","ANBHONDD_03427","DPBEEGGP_02363","DHOHKKHL_03069","ADHCADEO_02317","MHIPNNDE_03395","PFJGCJHP_02006","GGAGLIPG_03128","PNCEMHLO_02850","FBLKDDDN_03425","GGMENCCH_03413","KNKCDEML_03733","GJADGJJH_03419","NBEGIDKM_03197","MAEDJJLC_03236","CBGFCALD_03433","LBEKBKDJ_03458","MJNIIBMG_03143","BAIOBGAL_03355","OEMCECNB_03464","HIKCDMLK_02663","AFDIGNOC_03418","ANBHONDD_03430","DPBEEGGP_02360","DHOHKKHL_03072","ADHCADEO_02314","MHIPNNDE_03398","PFJGCJHP_02009","GGAGLIPG_03125","PNCEMHLO_02847","FBLKDDDN_03426","GGMENCCH_03412","KNKCDEML_03734","GJADGJJH_03420","NBEGIDKM_03196","MAEDJJLC_03237","CBGFCALD_03434","LBEKBKDJ_03457","MJNIIBMG_03144","BAIOBGAL_03356","OEMCECNB_03465","HIKCDMLK_02664","AFDIGNOC_03419","ANBHONDD_03429","DPBEEGGP_02361","DHOHKKHL_03071","ADHCADEO_02315","MHIPNNDE_03397","PFJGCJHP_02008","GGAGLIPG_03126","PNCEMHLO_02848","FBLKDDDN_03424","GGMENCCH_03414","KNKCDEML_03732","GJADGJJH_03418","NBEGIDKM_03198","MAEDJJLC_03235","CBGFCALD_03432","LBEKBKDJ_03459","MJNIIBMG_03142","BAIOBGAL_03354","OEMCECNB_03463","HIKCDMLK_02662","AFDIGNOC_03417","ANBHONDD_03431","DPBEEGGP_02359","DHOHKKHL_03073","ADHCADEO_02313","MHIPNNDE_03399","PFJGCJHP_02010","GGAGLIPG_03124","PNCEMHLO_02846","HGOBCNMM_03604","FBLKDDDN_01878","GGMENCCH_03207","KNKCDEML_03917","GJADGJJH_03268","NBEGIDKM_03370","MAEDJJLC_03163","PPAMLKML_01486","ANLDKFMB_03401","CBGFCALD_03530","LBEKBKDJ_03269","MJNIIBMG_02575","OEMCECNB_03417","AFDIGNOC_00375","CHLMDKPE_03520","GGAGLIPG_03647","PNCEMHLO_03653","HIIAAINO_02370","MLIKFGOG_03083","LBMBHCAM_01843","ALMPNPLM_01595","ILDLLNBB_02343","MDOHONPH_01637","CBGFCALD_02376","OEMCECNB_01433","KALPBOHI_00415","NDEKODAF_01380","NDDDIKEP_03549","ONGGBCKC_02347","PMNODAEP_02099","BOLLBKIO_02537","GGAGLIPG_02536","IHJLPFIO_02484","PNCEMHLO_02287","MGKDKBAB_02579","LOFCOCJN_02579","PFAGPCGD_03454","KLCBHBCF_02732","OFBONHDF_02733","ALMPNPLM_01596","ILDLLNBB_02342","MDOHONPH_01636","CBGFCALD_02375","OEMCECNB_01434","KALPBOHI_00414","NDEKODAF_01379","NDDDIKEP_03548","ONGGBCKC_02346","PMNODAEP_02098","BOLLBKIO_02536","GGAGLIPG_02537","IHJLPFIO_02483","PNCEMHLO_02286","MGKDKBAB_02580","LOFCOCJN_02580","PFAGPCGD_03453","KLCBHBCF_02733","OFBONHDF_02734","ALMPNPLM_01594","ILDLLNBB_02344","MDOHONPH_01638","CBGFCALD_02377","OEMCECNB_01432","KALPBOHI_00416","NDEKODAF_01381","NDDDIKEP_03550","ONGGBCKC_02348","PMNODAEP_02100","BOLLBKIO_02538","GGAGLIPG_02535","IHJLPFIO_02485","PNCEMHLO_02288","MGKDKBAB_02578","LOFCOCJN_02578","PFAGPCGD_03455","KLCBHBCF_02731","OFBONHDF_02732","ALMPNPLM_01591","ILDLLNBB_02346","MDOHONPH_01640","CBGFCALD_02381","OEMCECNB_01428","KALPBOHI_00418","NDEKODAF_01383","NDDDIKEP_03553","ONGGBCKC_02351","PMNODAEP_02103","BOLLBKIO_02541","GGAGLIPG_02532","IHJLPFIO_02488","PNCEMHLO_02291","MGKDKBAB_02575","LOFCOCJN_02575","PFAGPCGD_03458","KLCBHBCF_02728","OFBONHDF_02729","ALMPNPLM_01590","ILDLLNBB_02347","MDOHONPH_01641","CBGFCALD_02382","OEMCECNB_01427","KALPBOHI_00419","NDEKODAF_01384","NDDDIKEP_03554","ONGGBCKC_02352","PMNODAEP_02104","BOLLBKIO_02542","GGAGLIPG_02531","IHJLPFIO_02489","PNCEMHLO_02292","MGKDKBAB_02574","LOFCOCJN_02574","PFAGPCGD_03459","KLCBHBCF_02727","OFBONHDF_02728","ALMPNPLM_01599","ILDLLNBB_01983","MDOHONPH_01630","CBGFCALD_02371","OEMCECNB_01438","KALPBOHI_00410","NDEKODAF_01375","NDDDIKEP_03588","ONGGBCKC_02343","PMNODAEP_02095","BOLLBKIO_02533","GGAGLIPG_03364","IHJLPFIO_02480","PNCEMHLO_03386","MGKDKBAB_02583","LOFCOCJN_02583","PFAGPCGD_03426","KLCBHBCF_02736","OFBONHDF_02737","FBLKDDDN_03247","GGMENCCH_03326","KNKCDEML_02984","GJADGJJH_03309","NBEGIDKM_02847","MAEDJJLC_03216","CBGFCALD_03317","LBEKBKDJ_03281","MJNIIBMG_03301","OEMCECNB_03270","KALPBOHI_02774","HIKCDMLK_03713","AFDIGNOC_03150","ANBHONDD_03576","DPBEEGGP_03286","DHOHKKHL_03749","MHIPNNDE_02790","GGAGLIPG_03228","PNCEMHLO_03455","ALMPNPLM_01588","ILDLLNBB_02351","MDOHONPH_01645","CBGFCALD_02384","OEMCECNB_01425","KALPBOHI_00423","NDEKODAF_01386","NDDDIKEP_03555","ONGGBCKC_02353","PMNODAEP_02105","BOLLBKIO_02543","GGAGLIPG_02530","IHJLPFIO_02490","PNCEMHLO_02293","MGKDKBAB_02573","LOFCOCJN_02573","PFAGPCGD_03460","KLCBHBCF_02726","OFBONHDF_02727","HGOBCNMM_00755","KNKCDEML_03745","ANLDKFMB_01647","ILDLLNBB_01673","MDOHONPH_03326","ICFJOHJH_03479","BAIOBGAL_02741","HIKCDMLK_03281","CNFKGNAC_00141","AFDIGNOC_01061","NDEKODAF_03353","DHOHKKHL_00778","ADHCADEO_00223","MHIPNNDE_00337","PFJGCJHP_02700","CHLMDKPE_02636","GGAGLIPG_00714","PNCEMHLO_00006","PBGIHMLN_02559","HGOBCNMM_00756","KNKCDEML_03746","ANLDKFMB_01648","ILDLLNBB_01674","MDOHONPH_02938","ICFJOHJH_03478","BAIOBGAL_02742","HIKCDMLK_03280","CNFKGNAC_00140","AFDIGNOC_01060","NDEKODAF_03095","DHOHKKHL_00777","ADHCADEO_00224","MHIPNNDE_00338","PFJGCJHP_02701","CHLMDKPE_02635","GGAGLIPG_00715","PNCEMHLO_00005","PBGIHMLN_02560","ALMPNPLM_01600","ILDLLNBB_01982","MDOHONPH_01629","CBGFCALD_02370","OEMCECNB_01439","KALPBOHI_00409","NDEKODAF_01374","NDDDIKEP_03587","ONGGBCKC_02342","PMNODAEP_02094","BOLLBKIO_02532","GGAGLIPG_03365","IHJLPFIO_02479","PNCEMHLO_03385","MGKDKBAB_02584","LOFCOCJN_02584","PFAGPCGD_03427","KLCBHBCF_02737","OFBONHDF_02738","HGOBCNMM_00753","KNKCDEML_03743","ANLDKFMB_01645","ILDLLNBB_01671","MDOHONPH_03328","ICFJOHJH_03481","BAIOBGAL_02739","HIKCDMLK_03283","CNFKGNAC_00143","AFDIGNOC_01063","NDEKODAF_03355","DHOHKKHL_00780","ADHCADEO_00221","MHIPNNDE_00335","PFJGCJHP_02698","CHLMDKPE_02638","GGAGLIPG_00712","PNCEMHLO_00008","PBGIHMLN_02557","HGOBCNMM_00754","KNKCDEML_03744","ANLDKFMB_01646","ILDLLNBB_01672","MDOHONPH_03327","ICFJOHJH_03480","BAIOBGAL_02740","HIKCDMLK_03282","CNFKGNAC_00142","AFDIGNOC_01062","NDEKODAF_03354","DHOHKKHL_00779","ADHCADEO_00222","MHIPNNDE_00336","PFJGCJHP_02699","CHLMDKPE_02637","GGAGLIPG_00713","PNCEMHLO_00007","PBGIHMLN_02558","HGOBCNMM_00752","KNKCDEML_03742","ANLDKFMB_01644","ILDLLNBB_01670","MDOHONPH_03329","ICFJOHJH_03482","BAIOBGAL_02738","HIKCDMLK_03284","CNFKGNAC_00144","AFDIGNOC_01064","NDEKODAF_03356","DHOHKKHL_00781","ADHCADEO_00220","MHIPNNDE_00334","PFJGCJHP_02697","CHLMDKPE_02639","GGAGLIPG_00711","PNCEMHLO_00009","PBGIHMLN_02556","FBLKDDDN_03253","GGMENCCH_03320","KNKCDEML_02978","GJADGJJH_03303","NBEGIDKM_02853","MAEDJJLC_03210","PPAMLKML_03515","ANLDKFMB_02847","ALMPNPLM_03545","CBGFCALD_03586","LBEKBKDJ_03287","MJNIIBMG_03115","HIKCDMLK_03707","AFDIGNOC_03155","ANBHONDD_03570","DPBEEGGP_03280","ADHCADEO_03707","PFJGCJHP_03546","FBLKDDDN_03328","GGMENCCH_03435","KNKCDEML_03318","GJADGJJH_03379","MAEDJJLC_03336","PPAMLKML_03396","ANLDKFMB_03379","ALMPNPLM_03610","LBEKBKDJ_03413","KALPBOHI_03515","HIKCDMLK_03782","AFDIGNOC_03334","ANBHONDD_03332","DPBEEGGP_02314","MHIPNNDE_01641","CHLMDKPE_03427","GGAGLIPG_03181","PNCEMHLO_00729","FBLKDDDN_03254","GGMENCCH_03319","KNKCDEML_02977","GJADGJJH_03302","NBEGIDKM_02854","MAEDJJLC_03209","ANLDKFMB_02845","ANLDKFMB_02844","ALMPNPLM_03547","LBEKBKDJ_03288","MJNIIBMG_03114","HIKCDMLK_03041","AFDIGNOC_03157","ANBHONDD_03397","DPBEEGGP_03279","ADHCADEO_03744","PFJGCJHP_03621","GGAGLIPG_03224","PNCEMHLO_03451","FBLKDDDN_03245","GGMENCCH_03328","KNKCDEML_02986","GJADGJJH_03311","NBEGIDKM_02845","MAEDJJLC_03218","ALMPNPLM_03566","ALMPNPLM_03567","LBEKBKDJ_03279","MJNIIBMG_03492","OEMCECNB_03272","CNFKGNAC_03248","AFDIGNOC_03148","ANBHONDD_02863","DPBEEGGP_03287","ADHCADEO_03615","PFJGCJHP_03540","GGAGLIPG_03229","PNCEMHLO_03456","ANLDKFMB_01239","ALMPNPLM_01257","ANBHONDD_01245","NDEKODAF_02900","DPBEEGGP_01383","NDDDIKEP_00366","ONGGBCKC_00464","PMNODAEP_00666","BOLLBKIO_00492","IHJLPFIO_00492","HIIAAINO_00440","MGKDKBAB_00666","MLIKFGOG_00440","LOFCOCJN_00666","LBMBHCAM_01168"
,"PFAGPCGD_00377","KLCBHBCF_00492","OFBONHDF_00492","HGOBCNMM_02094","KNKCDEML_03349","PPAMLKML_03463","ALMPNPLM_02888","ICFJOHJH_03375","MJNIIBMG_02373","HIKCDMLK_03360","AFDIGNOC_02899","DHOHKKHL_03216","ADHCADEO_03468","MHIPNNDE_01284","CHLMDKPE_03039","GGAGLIPG_03553","PNCEMHLO_03576","HIIAAINO_02153","MLIKFGOG_02580","LBMBHCAM_02060","PBGIHMLN_03467","HGOBCNMM_03526","PPAMLKML_03438","ALMPNPLM_03430","ILDLLNBB_02986","MDOHONPH_03081","ICFJOHJH_03426","KALPBOHI_02995","HIKCDMLK_02638","CNFKGNAC_01758","AFDIGNOC_03393","ANBHONDD_02979","NDEKODAF_03092","DPBEEGGP_03313","DHOHKKHL_03097","ADHCADEO_03288","MHIPNNDE_03050","PFJGCJHP_03531","CHLMDKPE_03465","HGOBCNMM_03527","PPAMLKML_03437","ALMPNPLM_03431","ILDLLNBB_02985","MDOHONPH_03082","ICFJOHJH_03425","KALPBOHI_02994","HIKCDMLK_02639","CNFKGNAC_01759","AFDIGNOC_03394","ANBHONDD_02980","NDEKODAF_03091","DPBEEGGP_03314","DHOHKKHL_03096","ADHCADEO_03287","MHIPNNDE_03051","PFJGCJHP_03532","CHLMDKPE_03464","HGOBCNMM_03525","PPAMLKML_03439","ALMPNPLM_03429","ILDLLNBB_02987","MDOHONPH_03080","ICFJOHJH_03427","KALPBOHI_02996","HIKCDMLK_02637","CNFKGNAC_01757","AFDIGNOC_03392","ANBHONDD_02978","NDEKODAF_03093","DPBEEGGP_03312","DHOHKKHL_03098","ADHCADEO_03289","MHIPNNDE_03049","PFJGCJHP_03620","CHLMDKPE_03408","FBLKDDDN_03261","GGMENCCH_03312","KNKCDEML_02970","GJADGJJH_03295","NBEGIDKM_02861","MAEDJJLC_03202","ANLDKFMB_03333","MDOHONPH_01387","CBGFCALD_03290","LBEKBKDJ_03295","MJNIIBMG_03106","OEMCECNB_03386","CNFKGNAC_02910","AFDIGNOC_03618","HIIAAINO_03107","MLIKFGOG_03018","LBMBHCAM_03165","FBLKDDDN_03255","GGMENCCH_03318","KNKCDEML_02976","GJADGJJH_03301","NBEGIDKM_02855","MAEDJJLC_03208","ANLDKFMB_02843","ALMPNPLM_03548","LBEKBKDJ_03289","MJNIIBMG_03113","HIKCDMLK_03042","AFDIGNOC_03158","ANBHONDD_03398","DPBEEGGP_03278","ADHCADEO_03388","GGAGLIPG_03223","PNCEMHLO_03450","HGOBCNMM_02752","KNKCDEML_03901","PPAMLKML_01891","ANLDKFMB_02241","ILDLLNBB_00234","MDOHONPH_00230","CBGFCALD_01768","BAIOBGAL_01607","OEMCECNB_00193","KALPBOHI_00824","HIKCDMLK_01378","DHOHKKHL_01273","ADHCADEO_02275","PFJGCJHP_02385","CHLMDKPE_02430","GGAGLIPG_00912","PNCEMHLO_00933","FBLKDDDN_03531","GGMENCCH_03532","KNKCDEML_03907","GJADGJJH_03532","NBEGIDKM_02839","MAEDJJLC_03429","PPAMLKML_02989","ILDLLNBB_03117","MDOHONPH_03195","ICFJOHJH_03220","CBGFCALD_03610","LBEKBKDJ_03533","MJNIIBMG_03459","OEMCECNB_03645","KALPBOHI_03532","AFDIGNOC_03655","CHLMDKPE_03157","FBLKDDDN_03248","GGMENCCH_03325","KNKCDEML_02983","GJADGJJH_03308","NBEGIDKM_02848","MAEDJJLC_03215","PPAMLKML_03018","LBEKBKDJ_03282","MJNIIBMG_03120","HIKCDMLK_03712","ANBHONDD_03575","DPBEEGGP_03285","PFJGCJHP_03542","HIIAAINO_03324","MLIKFGOG_03409","LBMBHCAM_03311","FBLKDDDN_03251","GGMENCCH_03322","KNKCDEML_02980","GJADGJJH_03305","NBEGIDKM_02851","MAEDJJLC_03212","ANLDKFMB_02849","ALMPNPLM_03544","LBEKBKDJ_03285","MJNIIBMG_03117","HIKCDMLK_03709","AFDIGNOC_03153","ANBHONDD_03572","DPBEEGGP_03282","ADHCADEO_03708","PFJGCJHP_03545","FBLKDDDN_03249","GGMENCCH_03324","KNKCDEML_02982","GJADGJJH_03307","NBEGIDKM_02849","MAEDJJLC_03214","ANLDKFMB_02851","ALMPNPLM_03542","LBEKBKDJ_03283","MJNIIBMG_03119","HIKCDMLK_03711","AFDIGNOC_03151","ANBHONDD_03574","DPBEEGGP_03284","ADHCADEO_03710","PFJGCJHP_03543","CNFKGNAC_01235","NDEKODAF_00619","AMNKGELI_00615","NDDDIKEP_01861","ONGGBCKC_01328","PMNODAEP_02494","BOLLBKIO_01835","IHJLPFIO_01823","HIIAAINO_02776","MGKDKBAB_02140","MLIKFGOG_02140","LOFCOCJN_02140","LBMBHCAM_02516","PFAGPCGD_02115","KLCBHBCF_02066","OFBONHDF_01854","HGOBCNMM_00135","FBLKDDDN_00413","GGMENCCH_02111","KNKCDEML_01122","GJADGJJH_02111","NBEGIDKM_00739","MAEDJJLC_00796","ANLDKFMB_00778","ILDLLNBB_01126","MDOHONPH_01176","LBEKBKDJ_00796","MJNIIBMG_01874","HIKCDMLK_00379","DHOHKKHL_02129","ADHCADEO_03041","MHIPNNDE_01060","FBLKDDDN_03250","GGMENCCH_03323","KNKCDEML_02981","GJADGJJH_03306","NBEGIDKM_02850","MAEDJJLC_03213","ANLDKFMB_02850","ALMPNPLM_03543","LBEKBKDJ_03284","MJNIIBMG_03118","HIKCDMLK_03710","AFDIGNOC_03152","ANBHONDD_03573","DPBEEGGP_03283","ADHCADEO_03709","PFJGCJHP_03544","CNFKGNAC_01236","NDEKODAF_00620","AMNKGELI_00614","NDDDIKEP_01863","ONGGBCKC_01330","PMNODAEP_02496","BOLLBKIO_01837","IHJLPFIO_01825","HIIAAINO_02775","MGKDKBAB_02138","MLIKFGOG_02142","LOFCOCJN_02138","LBMBHCAM_02515","PFAGPCGD_02113","KLCBHBCF_02068","OFBONHDF_01856","HGOBCNMM_02095","KNKCDEML_03350","PPAMLKML_03464","ALMPNPLM_02887","ICFJOHJH_03376","MJNIIBMG_02372","AFDIGNOC_00687","MHIPNNDE_01285","CHLMDKPE_03038","GGAGLIPG_03552","PNCEMHLO_03577","HIIAAINO_02154","MLIKFGOG_02581","LBMBHCAM_02059","PBGIHMLN_03466","FBLKDDDN_03256","GGMENCCH_03317","KNKCDEML_02975","GJADGJJH_03300","NBEGIDKM_02856","MAEDJJLC_03207","LBEKBKDJ_03290","MJNIIBMG_03112","HIKCDMLK_03043","AFDIGNOC_03159","ANBHONDD_03399","DPBEEGGP_03277","ADHCADEO_03387","GGAGLIPG_03588","PNCEMHLO_03600","HGOBCNMM_00630","FBLKDDDN_03363","GGMENCCH_00676","KNKCDEML_01335","GJADGJJH_00640","NBEGIDKM_02184","MAEDJJLC_01884","LBEKBKDJ_01407","BAIOBGAL_00167","CNFKGNAC_00267","ANBHONDD_01433","NDEKODAF_03243","NDEKODAF_03242","CHLMDKPE_02759","PBGIHMLN_02847","NDDDIKEP_02624","ONGGBCKC_02801","PMNODAEP_02560","BOLLBKIO_02783","IHJLPFIO_02941","HIIAAINO_02369","MGKDKBAB_02756","MLIKFGOG_03084","LOFCOCJN_02756","LBMBHCAM_01844","PFAGPCGD_02555","KLCBHBCF_02853","OFBONHDF_02758","PBGIHMLN_02777","NDDDIKEP_02623","ONGGBCKC_02800","PMNODAEP_02561","BOLLBKIO_02782","IHJLPFIO_02940","HIIAAINO_02368","MGKDKBAB_02757","MLIKFGOG_03085","LOFCOCJN_02757","LBMBHCAM_01845","PFAGPCGD_02556","KLCBHBCF_02852","OFBONHDF_02759","PBGIHMLN_02778","FBLKDDDN_03257","GGMENCCH_03316","KNKCDEML_02974","GJADGJJH_03299","NBEGIDKM_02857","MAEDJJLC_03206","ILDLLNBB_03000","MDOHONPH_03067","LBEKBKDJ_03291","MJNIIBMG_03111","CHLMDKPE_03507","GGAGLIPG_03589","PNCEMHLO_03599","NDDDIKEP_01976","ONGGBCKC_01443","PMNODAEP_01673","BOLLBKIO_01950","IHJLPFIO_01938","HIIAAINO_02671","MGKDKBAB_02025","MLIKFGOG_02246","LOFCOCJN_02025","LBMBHCAM_02411","PFAGPCGD_02000","KLCBHBCF_02181","OFBONHDF_01969","NDDDIKEP_01975","ONGGBCKC_01442","PMNODAEP_01672","BOLLBKIO_01949","IHJLPFIO_01937","HIIAAINO_02672","MGKDKBAB_02026","MLIKFGOG_02245","LOFCOCJN_02026","LBMBHCAM_02412","PFAGPCGD_02001","KLCBHBCF_02180","OFBONHDF_01968","NDDDIKEP_01973","ONGGBCKC_01440","PMNODAEP_01670","BOLLBKIO_01947","IHJLPFIO_01935","HIIAAINO_02673","MGKDKBAB_02028","MLIKFGOG_02244","LOFCOCJN_02028","LBMBHCAM_02413","PFAGPCGD_02003","KLCBHBCF_02178","OFBONHDF_01966","FBLKDDDN_03252","GGMENCCH_03321","KNKCDEML_02979","GJADGJJH_03304","NBEGIDKM_02852","MAEDJJLC_03211","ANLDKFMB_02848","LBEKBKDJ_03286","MJNIIBMG_03116","HIKCDMLK_03708","AFDIGNOC_03154","ANBHONDD_03571","DPBEEGGP_03281","NDDDIKEP_02625","ONGGBCKC_02802","PMNODAEP_02559","BOLLBKIO_02784","IHJLPFIO_02942","HIIAAINO_02371","MGKDKBAB_02755","MLIKFGOG_03082","LOFCOCJN_02755","LBMBHCAM_01842","PFAGPCGD_02554","KLCBHBCF_02854","OFBONHDF_02757","KNKCDEML_03781","NDDDIKEP_02191","ONGGBCKC_02136","PMNODAEP_01888","BOLLBKIO_02165","GGAGLIPG_01955","IHJLPFIO_02465","PNCEMHLO_01981","MGKDKBAB_02243","LOFCOCJN_02243","PFAGPCGD_02218","KLCBHBCF_02588","OFBONHDF_02589","HGOBCNMM_02104","KNKCDEML_03354","PPAMLKML_02689","ALMPNPLM_02877","ICFJOHJH_03533","MJNIIBMG_02364","HIKCDMLK_02042","AFDIGNOC_02913","DHOHKKHL_02568","MHIPNNDE_01295","CHLMDKPE_03034","GGAGLIPG_02720","PNCEMHLO_02746","NDDDIKEP_01971","ONGGBCKC_01438","PMNODAEP_01668","BOLLBKIO_01945","IHJLPFIO_01933","HIIAAINO_02674","MGKDKBAB_02030","MLIKFGOG_02243","LOFCOCJN_02030","LBMBHCAM_02414","PFAGPCGD_02005","KLCBHBCF_02176","OFBONHDF_01964","HGOBCNMM_02103","FBLKDDDN_03093","GGMENCCH_02937","KNKCDEML_03353","GJADGJJH_02937","ALMPNPLM_02879","ALMPNPLM_02878","ICFJOHJH_03532","LBEKBKDJ_02126","AFDIGNOC_02912","MHIPNNDE_01294","CHLMDKPE_03035","GGAGLIPG_02721","PNCEMHLO_02747","FBLKDDDN_03242","GGMENCCH_03331","KNKCDEML_02989","GJADGJJH_03314","NBEGIDKM_02842","MAEDJJLC_03221","LBEKBKDJ_03276","MJNIIBMG_03489","HIKCDMLK_03890","ANBHONDD_02865","DPBEEGGP_03289","DHOHKKHL_03797","ADHCADEO_03782","HGOBCNMM_00706","KNKCDEML_03727","BAIOBGAL_02692","CNFKGNAC_00189","ANBHONDD_01513","NDEKODAF_02705","DPBEEGGP_02533","ADHCADEO_00175","MHIPNNDE_00288","AMNKGELI_02206","GGAGLIPG_00665","PNCEMHLO_00056","PNCEMHLO_00055","PBGIHMLN_02922","HGOBCNMM_00708","KNKCDEML_03729","BAIOBGAL_02694","CNFKGNAC_00187","ANBHONDD_01515","NDEKODAF_02703","DPBEEGGP_02535","ADHCADEO_00177","MHIPNNDE_00290","AMNKGELI_02208","GGAGLIPG_00667","PNCEMHLO_00053","PBGIHMLN_02924","HGOBCNMM_03369","FBLKDDDN_03503","KNKCDEML_03472","GJADGJJH_03510","MAEDJJLC_03407","ALMPNPLM_02022","LBEKBKDJ_03365","MJNIIBMG_03426","OEMCECNB_03410","AFDIGNOC_03462","GGAGLIPG_02873","PNCEMHLO_02990","HIIAAINO_02420","HGOBCNMM_00704","KNKCDEML_03725","BAIOBGAL_02690","CNFKGNAC_00191","ANBHONDD_01511","NDEKODAF_02707","DPBEEGGP_02531","ADHCADEO_00173","MHIPNNDE_00286","AMNKGELI_02204","GGAGLIPG_00663","PNCEMHLO_00058","PBGIHMLN_02920","HGOBCNMM_00707","KNKCDEML_03728","BAIOBGAL_02693","CNFKGNAC_00188","ANBHONDD_01514","NDEKODAF_02704","DPBEEGGP_02534","ADHCADEO_00176","MHIPNNDE_00289","AMNKGELI_02207","GGAGLIPG_00666","PNCEMHLO_00054","PBGIHMLN_02923","HGOBCNMM_00705","KNKCDEML_03726","BAIOBGAL_02691","CNFKGNAC_00190","ANBHONDD_01512","NDEKODAF_02706","DPBEEGGP_02532","ADHCADEO_00174","MHIPNNDE_00287","AMNKGELI_02205","GGAGLIPG_00664","PNCEMHLO_00057","PBGIHMLN_02921","FBLKDDDN_03241","GGMENCCH_03332","KNKCDEML_02990","GJADGJJH_03315","NBEGIDKM_02841","MAEDJJLC_03222","LBEKBKDJ_03275","HIKCDMLK_03869","AFDIGNOC_03145","ANBHONDD_02866","DPBEEGGP_03290","DHOHKKHL_03784","FBLKDDDN_00733","GGMENCCH_01791","KNKCDEML_00802","GJADGJJH_01791","NBEGIDKM_00418","MAEDJJLC_00476","LBEKBKDJ_00476","MJNIIBMG_01554","CNFKGNAC_00973","ANBHONDD_00487","NDEKODAF_00460","DPBEEGGP_00743","PPAMLKML_01444","ALMPNPLM_00203","ICFJOHJH_00183","BAIOBGAL_02071","KALPBOHI_01855","HIKCDMLK_01795","AFDIGNOC_01604","ANBHONDD_00972","DPBEEGGP_01656","DHOHKKHL_01628","ADHCADEO_01499","ADHCADEO_01500"
,"CHLMDKPE_01532","HGOBCNMM_03370","FBLKDDDN_03502","KNKCDEML_03473","GJADGJJH_03511","MAEDJJLC_03408","LBEKBKDJ_03364","MJNIIBMG_03427","OEMCECNB_03411","AFDIGNOC_03463","GGAGLIPG_02874","PNCEMHLO_02991","HIIAAINO_02419","FBLKDDDN_00732","GGMENCCH_01792","KNKCDEML_00803","GJADGJJH_01792","NBEGIDKM_00419","MAEDJJLC_00477","LBEKBKDJ_00477","MJNIIBMG_01555","CNFKGNAC_00972","ANBHONDD_00486","NDEKODAF_00459","DPBEEGGP_00742","PPAMLKML_01439","ALMPNPLM_00198","ICFJOHJH_00178","BAIOBGAL_02076","KALPBOHI_01850","HIKCDMLK_01790","AFDIGNOC_01609","ANBHONDD_00967","DPBEEGGP_01661","DHOHKKHL_01633","ADHCADEO_01505","CHLMDKPE_01527","PPAMLKML_01443","ALMPNPLM_00202","ICFJOHJH_00182","BAIOBGAL_02072","KALPBOHI_01854","HIKCDMLK_01794","AFDIGNOC_01605","ANBHONDD_00971","DPBEEGGP_01657","DHOHKKHL_01629","ADHCADEO_01501","CHLMDKPE_01531","PPAMLKML_01440","ALMPNPLM_00199","ICFJOHJH_00179","BAIOBGAL_02075","KALPBOHI_01851","HIKCDMLK_01791","AFDIGNOC_01608","ANBHONDD_00968","DPBEEGGP_01660","DHOHKKHL_01632","ADHCADEO_01504","CHLMDKPE_01528","PPAMLKML_01441","ALMPNPLM_00200","ICFJOHJH_00180","BAIOBGAL_02074","KALPBOHI_01852","HIKCDMLK_01792","AFDIGNOC_01607","ANBHONDD_00969","DPBEEGGP_01659","DHOHKKHL_01631","ADHCADEO_01503","CHLMDKPE_01529","PPAMLKML_01442","ALMPNPLM_00201","ICFJOHJH_00181","BAIOBGAL_02073","KALPBOHI_01853","HIKCDMLK_01793","AFDIGNOC_01606","ANBHONDD_00970","DPBEEGGP_01658","DHOHKKHL_01630","ADHCADEO_01502","CHLMDKPE_01530","PPAMLKML_01445","ALMPNPLM_00204","ICFJOHJH_00184","BAIOBGAL_02070","KALPBOHI_01856","HIKCDMLK_01796","AFDIGNOC_01603","ANBHONDD_00973","DPBEEGGP_01655","DHOHKKHL_01627","ADHCADEO_01498","CHLMDKPE_01533","HGOBCNMM_02102","ALMPNPLM_02880","MJNIIBMG_02365","HIKCDMLK_03369","AFDIGNOC_02910","DHOHKKHL_03781","ADHCADEO_03670","MHIPNNDE_01293","CHLMDKPE_03036","GGAGLIPG_02722","PNCEMHLO_02748","NDDDIKEP_01862","ONGGBCKC_01329","PMNODAEP_02495","BOLLBKIO_01836","IHJLPFIO_01824","MGKDKBAB_02139","MLIKFGOG_02141","LOFCOCJN_02139","PFAGPCGD_02114","KLCBHBCF_02067","OFBONHDF_01855","HGOBCNMM_02092","KNKCDEML_03346","PPAMLKML_03461","ALMPNPLM_02889","12_refound_110","MJNIIBMG_02376","MHIPNNDE_01281","CHLMDKPE_03041","GGAGLIPG_03555","PNCEMHLO_03574","PBGIHMLN_03709","FBLKDDDN_03243","GGMENCCH_03330","KNKCDEML_02988","GJADGJJH_03313","NBEGIDKM_02843","MAEDJJLC_03220","LBEKBKDJ_03277","MJNIIBMG_03490","AFDIGNOC_03147","ANBHONDD_02864","DPBEEGGP_03288","HGOBCNMM_02093","KNKCDEML_03347","PPAMLKML_03462","ICFJOHJH_03370","MJNIIBMG_02375","MHIPNNDE_01282","CHLMDKPE_03040","GGAGLIPG_03554","PNCEMHLO_03575","PBGIHMLN_03619","FBLKDDDN_03244","GGMENCCH_03329","KNKCDEML_02987","GJADGJJH_03312","NBEGIDKM_02844","MAEDJJLC_03219","ALMPNPLM_03539","LBEKBKDJ_03278","MJNIIBMG_03491","OEMCECNB_03273","HGOBCNMM_02091","KNKCDEML_03918","PPAMLKML_03460","ALMPNPLM_02890","MJNIIBMG_02377","MHIPNNDE_01280","CHLMDKPE_03042","GGAGLIPG_03556","PNCEMHLO_03573","PBGIHMLN_03707","HGOBCNMM_02090","KNKCDEML_03919","PPAMLKML_03459","ALMPNPLM_02891","MJNIIBMG_02378","MHIPNNDE_01279","CHLMDKPE_03043","GGAGLIPG_03557","PNCEMHLO_03572","PBGIHMLN_03706","HGOBCNMM_02085","HIKCDMLK_02642","CNFKGNAC_01762","AFDIGNOC_03397","ANBHONDD_02983","NDEKODAF_03088","DPBEEGGP_03317","DHOHKKHL_03093","ADHCADEO_03284","MHIPNNDE_03090","HGOBCNMM_03528","HIKCDMLK_02640","CNFKGNAC_01760","AFDIGNOC_03395","ANBHONDD_02981","NDEKODAF_03090","DPBEEGGP_03315","DHOHKKHL_03095","ADHCADEO_03286","MHIPNNDE_03052","NDDDIKEP_01974","ONGGBCKC_01441","PMNODAEP_01671","BOLLBKIO_01948","IHJLPFIO_01936","MGKDKBAB_02027","LOFCOCJN_02027","PFAGPCGD_02002","KLCBHBCF_02179","OFBONHDF_01967","FBLKDDDN_03246","GGMENCCH_03327","KNKCDEML_02985","GJADGJJH_03310","NBEGIDKM_02846","MAEDJJLC_03217","LBEKBKDJ_03280","MJNIIBMG_03493","19_refound_171","AFDIGNOC_03149","NDDDIKEP_00421","ONGGBCKC_00519","PMNODAEP_00611","BOLLBKIO_00547","IHJLPFIO_00547","MGKDKBAB_00611","LOFCOCJN_00611","PFAGPCGD_00432","KLCBHBCF_00547","OFBONHDF_00547","NDDDIKEP_00416","ONGGBCKC_00514","PMNODAEP_00616","BOLLBKIO_00542","IHJLPFIO_00542","MGKDKBAB_00616","LOFCOCJN_00616","PFAGPCGD_00427","KLCBHBCF_00542","OFBONHDF_00542","NDDDIKEP_01980","ONGGBCKC_01447","PMNODAEP_01677","BOLLBKIO_01954","IHJLPFIO_01942","MGKDKBAB_02021","LOFCOCJN_02021","PFAGPCGD_01996","KLCBHBCF_02185","OFBONHDF_01973","NDDDIKEP_00419","ONGGBCKC_00517","PMNODAEP_00613","BOLLBKIO_00545","IHJLPFIO_00545","MGKDKBAB_00613","LOFCOCJN_00613","PFAGPCGD_00430","KLCBHBCF_00545","OFBONHDF_00545","NDDDIKEP_00410","ONGGBCKC_00508","PMNODAEP_00622","BOLLBKIO_00536","IHJLPFIO_00536","MGKDKBAB_00622","LOFCOCJN_00622","PFAGPCGD_00421","KLCBHBCF_00536","OFBONHDF_00536","NDDDIKEP_00418","ONGGBCKC_00516","PMNODAEP_00614","BOLLBKIO_00544","IHJLPFIO_00544","MGKDKBAB_00614","LOFCOCJN_00614","PFAGPCGD_00429","KLCBHBCF_00544","OFBONHDF_00544","NDDDIKEP_00415","ONGGBCKC_00513","PMNODAEP_00617","BOLLBKIO_00541","IHJLPFIO_00541","MGKDKBAB_00617","LOFCOCJN_00617","PFAGPCGD_00426","KLCBHBCF_00541","OFBONHDF_00541","HGOBCNMM_02084","HIKCDMLK_02641","CNFKGNAC_01761","AFDIGNOC_03396","ANBHONDD_02982","NDEKODAF_03089","DPBEEGGP_03316","DHOHKKHL_03094","ADHCADEO_03285","MHIPNNDE_03091","NDDDIKEP_01972","ONGGBCKC_01439","PMNODAEP_01669","BOLLBKIO_01946","IHJLPFIO_01934","MGKDKBAB_02029","LOFCOCJN_02029","PFAGPCGD_02004","KLCBHBCF_02177","OFBONHDF_01965","NDDDIKEP_01978","ONGGBCKC_01445","PMNODAEP_01675","BOLLBKIO_01952","IHJLPFIO_01940","MGKDKBAB_02023","LOFCOCJN_02023","PFAGPCGD_01998","KLCBHBCF_02183","OFBONHDF_01971","NDDDIKEP_00414","ONGGBCKC_00512","PMNODAEP_00618","BOLLBKIO_00540","IHJLPFIO_00540","MGKDKBAB_00618","LOFCOCJN_00618","PFAGPCGD_00425","KLCBHBCF_00540","OFBONHDF_00540","NDDDIKEP_00417","ONGGBCKC_00515","PMNODAEP_00615","BOLLBKIO_00543","IHJLPFIO_00543","MGKDKBAB_00615","LOFCOCJN_00615","PFAGPCGD_00428","KLCBHBCF_00543","OFBONHDF_00543","NDDDIKEP_00408","ONGGBCKC_00506","PMNODAEP_00624","BOLLBKIO_00534","IHJLPFIO_00534","MGKDKBAB_00624","LOFCOCJN_00624","PFAGPCGD_00419","KLCBHBCF_00534","OFBONHDF_00534","NDDDIKEP_00412","ONGGBCKC_00510","PMNODAEP_00620","BOLLBKIO_00538","IHJLPFIO_00538","MGKDKBAB_00620","LOFCOCJN_00620","PFAGPCGD_00423","KLCBHBCF_00538","OFBONHDF_00538","NDDDIKEP_01979","ONGGBCKC_01446","PMNODAEP_01676","BOLLBKIO_01953","IHJLPFIO_01941","MGKDKBAB_02022","LOFCOCJN_02022","PFAGPCGD_01997","KLCBHBCF_02184","OFBONHDF_01972","NDDDIKEP_00411","ONGGBCKC_00509","PMNODAEP_00621","BOLLBKIO_00537","IHJLPFIO_00537","MGKDKBAB_00621","LOFCOCJN_00621","PFAGPCGD_00422","KLCBHBCF_00537","OFBONHDF_00537","NDDDIKEP_00409","ONGGBCKC_00507","PMNODAEP_00623","BOLLBKIO_00535","IHJLPFIO_00535","MGKDKBAB_00623","LOFCOCJN_00623","PFAGPCGD_00420","KLCBHBCF_00535","OFBONHDF_00535","FBLKDDDN_03260","GGMENCCH_03313","KNKCDEML_02971","GJADGJJH_03296","NBEGIDKM_02860","MAEDJJLC_03203","ICFJOHJH_03072","LBEKBKDJ_03294","MJNIIBMG_03107","CHLMDKPE_03382","NDDDIKEP_01977","ONGGBCKC_01444","PMNODAEP_01674","BOLLBKIO_01951","IHJLPFIO_01939","MGKDKBAB_02024","LOFCOCJN_02024","PFAGPCGD_01999","KLCBHBCF_02182","OFBONHDF_01970","NDDDIKEP_00420","ONGGBCKC_00518","PMNODAEP_00612","BOLLBKIO_00546","IHJLPFIO_00546","MGKDKBAB_00612","LOFCOCJN_00612","PFAGPCGD_00431","KLCBHBCF_00546","OFBONHDF_00546","NDDDIKEP_00413","ONGGBCKC_00511","PMNODAEP_00619","BOLLBKIO_00539","IHJLPFIO_00539","MGKDKBAB_00619","LOFCOCJN_00619","PFAGPCGD_00424","KLCBHBCF_00539","OFBONHDF_00539","HGOBCNMM_02097","ANLDKFMB_02830","ALMPNPLM_02885","MJNIIBMG_02370","HIKCDMLK_03363","AFDIGNOC_02902","DHOHKKHL_03213","ADHCADEO_03465","MHIPNNDE_01288","HGOBCNMM_02096","ANLDKFMB_02829","ALMPNPLM_02886","MJNIIBMG_02371","HIKCDMLK_03362","AFDIGNOC_02901","DHOHKKHL_03214","ADHCADEO_03466","MHIPNNDE_01287","ICFJOHJH_02807","CBGFCALD_02166","OEMCECNB_01643","KALPBOHI_00204","HIKCDMLK_01131","CNFKGNAC_02854","AFDIGNOC_02139","DHOHKKHL_00182","ADHCADEO_00661","ICFJOHJH_02806","CBGFCALD_02165","OEMCECNB_01644","KALPBOHI_00203","HIKCDMLK_01132","CNFKGNAC_02855","AFDIGNOC_02140","DHOHKKHL_00181","ADHCADEO_00660","HGOBCNMM_02100","ANLDKFMB_02831","ALMPNPLM_02882","MJNIIBMG_02367","HIKCDMLK_03364","AFDIGNOC_02906","DHOHKKHL_03212","ADHCADEO_03464","MHIPNNDE_01291","ICFJOHJH_01805","CBGFCALD_02167","OEMCECNB_01642","KALPBOHI_00205","HIKCDMLK_01130","CNFKGNAC_02853","AFDIGNOC_02138","DHOHKKHL_00183","ADHCADEO_00662","ICFJOHJH_02805","CBGFCALD_02164","OEMCECNB_01645","KALPBOHI_00202","HIKCDMLK_01133","CNFKGNAC_02856","AFDIGNOC_02141","DHOHKKHL_00180","ADHCADEO_00659","KNKCDEML_03432","HIKCDMLK_02669","AFDIGNOC_03424","ANBHONDD_03622","DPBEEGGP_02366","DHOHKKHL_03066","27_refound_270_stop","46_refound_364","FBLKDDDN_01524","GGMENCCH_00350","KNKCDEML_01713","GJADGJJH_00966","NBEGIDKM_01461","MAEDJJLC_01558","LBEKBKDJ_01733","MJNIIBMG_00909","FBLKDDDN_00642","GGMENCCH_01882","KNKCDEML_00893","GJADGJJH_01882","NBEGIDKM_00510","MAEDJJLC_00567","LBEKBKDJ_00567","MJNIIBMG_01645","FBLKDDDN_01522","GGMENCCH_00352","KNKCDEML_01715","GJADGJJH_00964","NBEGIDKM_01463","MAEDJJLC_01560","LBEKBKDJ_01731","MJNIIBMG_00907","FBLKDDDN_00643","GGMENCCH_01881","KNKCDEML_00892","GJADGJJH_01881","NBEGIDKM_00509","MAEDJJLC_00566","LBEKBKDJ_00566","MJNIIBMG_01644","FBLKDDDN_01523","GGMENCCH_00351","KNKCDEML_01714","GJADGJJH_00965","NBEGIDKM_01462","MAEDJJLC_01559","LBEKBKDJ_01732","MJNIIBMG_00908","ALMPNPLM_01598","ILDLLNBB_01984","MDOHONPH_01631","KALPBOHI_00411","33_refound_311","39_refound_339","41_refound_347","HGOBCNMM_00133","ILDLLNBB_01128","MDOHONPH_01174","NDEKODAF_00079","ADHCADEO_03180","MHIPNNDE_01062","AMNKGELI_00778","CNFKGNAC_00461","ADHCADEO_02364","MHIPNNDE_00057","CHLMDKPE_01785","GGAGLIPG_00434","PNCEMHLO_00287","PBGIHMLN_00167","ANLDKFMB_02434","ILDLLNBB_02858","MDOHONPH_02886","ANBHONDD_03268","DPBEEGGP_03434","AMNKGELI_01176","PFJGCJHP_01236","PPAMLKML_01515","BAIOBGAL_02281","KALPBOHI_02215","HIKCDMLK_00250","AFDIGNOC_01323","DHOHKKHL_01040","ADHCADEO_02531","PPAMLKML_01509","BAIOBGAL_02275","KALPBOHI_02221","HIKCDMLK_00256","AFDIGNOC_01329","DHOHKKHL_01046","ADHCADEO_02537","PPAMLKML_01502","BAIOBGAL_02268","KALPBOHI_02228"
,"HIKCDMLK_00263","AFDIGNOC_01336","DHOHKKHL_01053","ADHCADEO_02351","PPAMLKML_01504","BAIOBGAL_02270","KALPBOHI_02226","HIKCDMLK_00261","AFDIGNOC_01334","DHOHKKHL_01051","ADHCADEO_02349","PPAMLKML_01511","BAIOBGAL_02277","KALPBOHI_02219","HIKCDMLK_00254","AFDIGNOC_01327","DHOHKKHL_01044","ADHCADEO_02535","PPAMLKML_00854","ALMPNPLM_00735","KALPBOHI_00723","HIIAAINO_00963","MLIKFGOG_00963","LBMBHCAM_01691","PBGIHMLN_01286","ALMPNPLM_00943","KALPBOHI_01475","HIKCDMLK_02435","AFDIGNOC_00296","DHOHKKHL_02671","ADHCADEO_02192","CHLMDKPE_01106","ANLDKFMB_03354","CNFKGNAC_03219","ANBHONDD_03261","NDEKODAF_02935","DPBEEGGP_03441","AMNKGELI_01170","PFJGCJHP_01230","ALMPNPLM_00946","KALPBOHI_01472","HIKCDMLK_02432","AFDIGNOC_00299","DHOHKKHL_02668","ADHCADEO_02195","CHLMDKPE_01103","ANLDKFMB_03358","CNFKGNAC_03215","ANBHONDD_03257","NDEKODAF_02939","DPBEEGGP_03445","AMNKGELI_01166","PFJGCJHP_01226","ANLDKFMB_02439","ILDLLNBB_02863","MDOHONPH_02891","ANBHONDD_03272","DPBEEGGP_03430","AMNKGELI_01181","PFJGCJHP_01241","ANLDKFMB_02433","ILDLLNBB_02857","MDOHONPH_02885","ANBHONDD_03267","DPBEEGGP_03435","AMNKGELI_01175","PFJGCJHP_01235","PPAMLKML_01503","BAIOBGAL_02269","KALPBOHI_02227","HIKCDMLK_00262","AFDIGNOC_01335","DHOHKKHL_01052","ADHCADEO_02350","PPAMLKML_01514","BAIOBGAL_02280","KALPBOHI_02216","HIKCDMLK_00251","AFDIGNOC_01324","DHOHKKHL_01041","ADHCADEO_02532","ANLDKFMB_02438","ILDLLNBB_02862","MDOHONPH_02890","ANBHONDD_03271","DPBEEGGP_03431","AMNKGELI_01180","PFJGCJHP_01240","ALMPNPLM_00945","KALPBOHI_01473","HIKCDMLK_02433","AFDIGNOC_00298","DHOHKKHL_02669","ADHCADEO_02194","CHLMDKPE_01104","ANLDKFMB_03359","CNFKGNAC_03214","ANBHONDD_03256","NDEKODAF_02940","DPBEEGGP_03446","AMNKGELI_01165","PFJGCJHP_01225","PPAMLKML_01507","BAIOBGAL_02273","KALPBOHI_02223","HIKCDMLK_00258","AFDIGNOC_01331","DHOHKKHL_01048","ADHCADEO_02346","PPAMLKML_01506","BAIOBGAL_02272","KALPBOHI_02224","HIKCDMLK_00259","AFDIGNOC_01332","DHOHKKHL_01049","ADHCADEO_02347","ANLDKFMB_03357","CNFKGNAC_03216","ANBHONDD_03258","NDEKODAF_02938","DPBEEGGP_03444","AMNKGELI_01167","PFJGCJHP_01227","ANLDKFMB_03356","CNFKGNAC_03217","ANBHONDD_03259","NDEKODAF_02937","DPBEEGGP_03443","AMNKGELI_01168","PFJGCJHP_01228","ANLDKFMB_02437","ILDLLNBB_02861","MDOHONPH_02889","ANBHONDD_03270","DPBEEGGP_03432","AMNKGELI_01179","PFJGCJHP_01239","PPAMLKML_01508","BAIOBGAL_02274","KALPBOHI_02222","HIKCDMLK_00257","AFDIGNOC_01330","DHOHKKHL_01047","ADHCADEO_02345","ANLDKFMB_02440","ILDLLNBB_02864","MDOHONPH_02892","ANBHONDD_03273","DPBEEGGP_03429","AMNKGELI_01182","PFJGCJHP_01242","ANLDKFMB_02436","ILDLLNBB_02860","MDOHONPH_02888","ANBHONDD_03269","DPBEEGGP_03433","AMNKGELI_01178","PFJGCJHP_01238","PPAMLKML_01510","BAIOBGAL_02276","KALPBOHI_02220","HIKCDMLK_00255","AFDIGNOC_01328","DHOHKKHL_01045","ADHCADEO_02536","PPAMLKML_01505","BAIOBGAL_02271","KALPBOHI_02225","HIKCDMLK_00260","AFDIGNOC_01333","DHOHKKHL_01050","ADHCADEO_02348","ALMPNPLM_00944","KALPBOHI_01474","HIKCDMLK_02434","AFDIGNOC_00297","DHOHKKHL_02670","ADHCADEO_02193","CHLMDKPE_01105","PPAMLKML_01513","BAIOBGAL_02279","KALPBOHI_02217","HIKCDMLK_00252","AFDIGNOC_01325","DHOHKKHL_01042","ADHCADEO_02533","ANLDKFMB_03360","CNFKGNAC_03213","ANBHONDD_03255","NDEKODAF_02941","DPBEEGGP_03447","AMNKGELI_01164","PFJGCJHP_01224","ANLDKFMB_03355","CNFKGNAC_03218","ANBHONDD_03260","NDEKODAF_02936","DPBEEGGP_03442","AMNKGELI_01169","PFJGCJHP_01229","PPAMLKML_01512","BAIOBGAL_02278","KALPBOHI_02218","HIKCDMLK_00253","AFDIGNOC_01326","DHOHKKHL_01043","ADHCADEO_02534","ALMPNPLM_00947","KALPBOHI_01471","HIKCDMLK_02431","AFDIGNOC_00300","DHOHKKHL_02667","ADHCADEO_02196","CHLMDKPE_01102","KNKCDEML_03206","CBGFCALD_03374","HIKCDMLK_02658","AFDIGNOC_03413","DHOHKKHL_03077","ADHCADEO_02309","CBGFCALD_00421","BAIOBGAL_01825","OEMCECNB_00824","KALPBOHI_02250","HIKCDMLK_02794","DHOHKKHL_02810","CBGFCALD_00420","BAIOBGAL_01826","OEMCECNB_00825","KALPBOHI_02251","HIKCDMLK_02795","DHOHKKHL_02811","ALMPNPLM_01593","ILDLLNBB_02345","MDOHONPH_01639","KALPBOHI_00417","GGAGLIPG_02534","PNCEMHLO_02289","ANLDKFMB_00912","ALMPNPLM_03010","ICFJOHJH_01250","BAIOBGAL_00300","CHLMDKPE_00082","PBGIHMLN_01575","ANLDKFMB_02484","ILDLLNBB_02906","MDOHONPH_02934","ANBHONDD_03275","ANBHONDD_03589","DPBEEGGP_03472","DPBEEGGP_03427","PFJGCJHP_01287","ANLDKFMB_00902","ALMPNPLM_03238","ICFJOHJH_01258","BAIOBGAL_00309","CHLMDKPE_00091","PBGIHMLN_01566","ANLDKFMB_02483","ILDLLNBB_02905","MDOHONPH_02933","ANBHONDD_03274","DPBEEGGP_03428","PFJGCJHP_01286","ANLDKFMB_02860","BAIOBGAL_02185","CHLMDKPE_01418","HIIAAINO_00071","MLIKFGOG_00071","LBMBHCAM_02606","FBLKDDDN_01054","GGMENCCH_01470","KNKCDEML_00554","GJADGJJH_01470","MAEDJJLC_00155","LBEKBKDJ_00155","FBLKDDDN_01047","GGMENCCH_01477","KNKCDEML_00561","GJADGJJH_01477","MAEDJJLC_00162","LBEKBKDJ_00162","FBLKDDDN_01049","GGMENCCH_01475","KNKCDEML_00559","GJADGJJH_01475","MAEDJJLC_00160","LBEKBKDJ_00160","FBLKDDDN_01051","GGMENCCH_01473","KNKCDEML_00557","GJADGJJH_01473","MAEDJJLC_00158","LBEKBKDJ_00158","FBLKDDDN_01048","GGMENCCH_01476","KNKCDEML_00560","GJADGJJH_01476","MAEDJJLC_00161","LBEKBKDJ_00161","FBLKDDDN_01052","GGMENCCH_01472","KNKCDEML_00556","GJADGJJH_01472","MAEDJJLC_00157","LBEKBKDJ_00157","ANLDKFMB_02485","ILDLLNBB_02907","MDOHONPH_02935","ANBHONDD_03588","DPBEEGGP_03473","PFJGCJHP_01288","ANLDKFMB_02859","BAIOBGAL_02184","CHLMDKPE_01419","HIIAAINO_00072","MLIKFGOG_00072","LBMBHCAM_02607","FBLKDDDN_01058","GGMENCCH_01466","KNKCDEML_00550","GJADGJJH_01466","MAEDJJLC_00151","LBEKBKDJ_00151","FBLKDDDN_01053","GGMENCCH_01471","KNKCDEML_00555","GJADGJJH_01471","MAEDJJLC_00156","LBEKBKDJ_00156","ANLDKFMB_02861","BAIOBGAL_02186","CHLMDKPE_01417","HIIAAINO_00070","MLIKFGOG_00070","LBMBHCAM_02605","FBLKDDDN_01050","GGMENCCH_01474","KNKCDEML_00558","GJADGJJH_01474","MAEDJJLC_00159","LBEKBKDJ_00159","FBLKDDDN_01059","GGMENCCH_01465","KNKCDEML_00549","GJADGJJH_01465","MAEDJJLC_00150","LBEKBKDJ_00150","FBLKDDDN_01046","GGMENCCH_01478","KNKCDEML_00562","GJADGJJH_01478","MAEDJJLC_00163","LBEKBKDJ_00163","FBLKDDDN_01055","GGMENCCH_01469","KNKCDEML_00553","GJADGJJH_01469","MAEDJJLC_00154","LBEKBKDJ_00154","FBLKDDDN_01057","GGMENCCH_01467","KNKCDEML_00551","GJADGJJH_01467","MAEDJJLC_00152","LBEKBKDJ_00152","FBLKDDDN_01056","GGMENCCH_01468","KNKCDEML_00552","GJADGJJH_01468","MAEDJJLC_00153","LBEKBKDJ_00153","HGOBCNMM_02101","PPAMLKML_03465","ICFJOHJH_03377","MJNIIBMG_02366","MHIPNNDE_01292","ILDLLNBB_02172","MDOHONPH_02088","CNFKGNAC_01516","ANBHONDD_02276","DPBEEGGP_00168","HIKCDMLK_02661","AFDIGNOC_03416","DHOHKKHL_03074","ADHCADEO_02312","PFJGCJHP_02011","HGOBCNMM_02479","ALMPNPLM_03408","ILDLLNBB_01745","MDOHONPH_01392","MHIPNNDE_01996","HGOBCNMM_02099","ALMPNPLM_02883","MJNIIBMG_02368","AFDIGNOC_02905","MHIPNNDE_01290","HGOBCNMM_02098","ALMPNPLM_02884","MJNIIBMG_02369","AFDIGNOC_02904","MHIPNNDE_01289","PPAMLKML_00122","ILDLLNBB_02164","MDOHONPH_02096","CNFKGNAC_01508","PPAMLKML_00120","ILDLLNBB_02162","MDOHONPH_02098","CNFKGNAC_01506","ILDLLNBB_02348","MDOHONPH_01642","KALPBOHI_00420","NDEKODAF_01385","ANLDKFMB_00904","ICFJOHJH_01256","BAIOBGAL_00307","PBGIHMLN_01568","ALMPNPLM_01597","ILDLLNBB_01985","MDOHONPH_01632","KALPBOHI_00412","PPAMLKML_00119","ILDLLNBB_02159","MDOHONPH_02101","CNFKGNAC_01505","PPAMLKML_00116","ILDLLNBB_02156","MDOHONPH_02103","CNFKGNAC_01503","ANBHONDD_02195","NDEKODAF_00772","DPBEEGGP_00249","AMNKGELI_01133","HIKCDMLK_02659","AFDIGNOC_03414","DHOHKKHL_03076","ADHCADEO_02310","PPAMLKML_00117","ILDLLNBB_02157","MDOHONPH_02102","CNFKGNAC_01504","HGOBCNMM_03024","ALMPNPLM_01486","BAIOBGAL_01245","AMNKGELI_03120","HGOBCNMM_02473","ILDLLNBB_02640","MDOHONPH_02703","MHIPNNDE_01990","PPAMLKML_00115","ILDLLNBB_02155","MDOHONPH_02104","CNFKGNAC_01502","HGOBCNMM_03023","ALMPNPLM_01485","BAIOBGAL_01246","AMNKGELI_03121","CBGFCALD_00431","BAIOBGAL_01814","OEMCECNB_00814","KALPBOHI_02240","CBGFCALD_00434","BAIOBGAL_02256","OEMCECNB_00811","KALPBOHI_02239","HIKCDMLK_02660","AFDIGNOC_03415","DHOHKKHL_03075","ADHCADEO_02311","CNFKGNAC_01662","NDEKODAF_02979","AMNKGELI_02989","PBGIHMLN_02741","HIKCDMLK_02265","DHOHKKHL_02636","ADHCADEO_01178","AMNKGELI_00952","ANLDKFMB_01245","ANBHONDD_01239","NDEKODAF_02894","DPBEEGGP_01389","HGOBCNMM_03025","ALMPNPLM_01487","BAIOBGAL_01244","AMNKGELI_03119","ANLDKFMB_01241","ANBHONDD_01243","NDEKODAF_02898","DPBEEGGP_01385","HIKCDMLK_02264","DHOHKKHL_02637","ADHCADEO_01179","AMNKGELI_00951","HIKCDMLK_02266","DHOHKKHL_02635","ADHCADEO_01177","AMNKGELI_00953","HGOBCNMM_00134","ILDLLNBB_01127","MDOHONPH_01175","MHIPNNDE_01061","CBGFCALD_00424","BAIOBGAL_01822","OEMCECNB_00821","KALPBOHI_02247","ANLDKFMB_01246","ANBHONDD_01238","NDEKODAF_02893","DPBEEGGP_01390","HGOBCNMM_03026","ALMPNPLM_01488","BAIOBGAL_01243","AMNKGELI_03118","CBGFCALD_00429","BAIOBGAL_01816","OEMCECNB_00816","KALPBOHI_02242","PPAMLKML_00114","ILDLLNBB_02154","MDOHONPH_02105","CNFKGNAC_01501","ANLDKFMB_01240","ANBHONDD_01244","NDEKODAF_02899","DPBEEGGP_01384","CBGFCALD_00425","BAIOBGAL_01821","OEMCECNB_00820","KALPBOHI_02246","ANLDKFMB_01244","ANBHONDD_01240","NDEKODAF_02895","DPBEEGGP_01388","CBGFCALD_00423","BAIOBGAL_01823","OEMCECNB_00822","KALPBOHI_02248","HGOBCNMM_03027","ALMPNPLM_01489","BAIOBGAL_01242","AMNKGELI_03117","HGOBCNMM_03022","ALMPNPLM_01484","BAIOBGAL_01247","AMNKGELI_03122","ANLDKFMB_01243","ANBHONDD_01241","NDEKODAF_02896","DPBEEGGP_01387","ANLDKFMB_01242","ANBHONDD_01242","NDEKODAF_02897","DPBEEGGP_01386","CBGFCALD_00428","BAIOBGAL_01817","OEMCECNB_00817","KALPBOHI_02243","CBGFCALD_00430","BAIOBGAL_01815","OEMCECNB_00815","KALPBOHI_02241","CBGFCALD_00422","BAIOBGAL_01824","OEMCECNB_00823","KALPBOHI_02249","HIKCDMLK_02263","DHOHKKHL_02638","ADHCADEO_01180","AMNKGELI_00950","ANBHONDD_02191","NDEKODAF_00768","DPBEEGGP_00253","AMNKGELI_01137","CNFKGNAC_01663","NDEKODAF_02978","AMNKGELI_02990","PBGIHMLN_02742","BAIOBGAL_00172","CNFKGNAC_00263","CHLMDKPE_02756","BAIOBGAL_00170","BAIOBGAL_00171","CNFKGNAC_00265","CHLMDKPE_02757","ILDLLNBB_02163","MDOHONPH_02097"
,"CNFKGNAC_01507"
,"ANLDKFMB_00907","ICFJOHJH_01253","PBGIHMLN_01572","HGOBCNMM_02757","ANLDKFMB_03047","CNFKGNAC_02593","ANLDKFMB_00908","ICFJOHJH_01252","PBGIHMLN_01573","ICFJOHJH_00433","CNFKGNAC_00462","ADHCADEO_02365","HGOBCNMM_02756","ANLDKFMB_03048","NDEKODAF_01091","CBGFCALD_03563","BAIOBGAL_00166","NDEKODAF_03241","ANLDKFMB_00910","ALMPNPLM_03011","BAIOBGAL_00302","ANLDKFMB_00903","ICFJOHJH_01257","PBGIHMLN_01567","HGOBCNMM_02105","MHIPNNDE_01296","CHLMDKPE_03033","ILDLLNBB_02152","MDOHONPH_02107","CNFKGNAC_01499","ILDLLNBB_02350","MDOHONPH_01644","KALPBOHI_00422","HIIAAINO_02668","MLIKFGOG_02249","LBMBHCAM_02408","KNKCDEML_03784","GGAGLIPG_01958","PNCEMHLO_01984","CBGFCALD_00432","BAIOBGAL_01813","OEMCECNB_00813","CBGFCALD_00433","BAIOBGAL_01812","OEMCECNB_00812","HIIAAINO_02667","MLIKFGOG_02250","LBMBHCAM_02407","PPAMLKML_03331","GGAGLIPG_02133","PNCEMHLO_02252","KNKCDEML_03783","GGAGLIPG_01957","PNCEMHLO_01983","PPAMLKML_03328","GGAGLIPG_02136","PNCEMHLO_02249","ILDLLNBB_02349","MDOHONPH_01643","KALPBOHI_00421","ILDLLNBB_02153","MDOHONPH_02106","CNFKGNAC_01500","PPAMLKML_03329","GGAGLIPG_02135","PNCEMHLO_02250","CNFKGNAC_01674","NDEKODAF_02967","AMNKGELI_03001","PPAMLKML_03330","GGAGLIPG_02134","PNCEMHLO_02251","CNFKGNAC_01671","NDEKODAF_02970","AMNKGELI_02998","CNFKGNAC_01673","NDEKODAF_02968","AMNKGELI_03000","CNFKGNAC_01670","NDEKODAF_02971","AMNKGELI_02997","ILDLLNBB_02151","MDOHONPH_02108","CNFKGNAC_01498","KNKCDEML_03782","GGAGLIPG_01956","PNCEMHLO_01982","CNFKGNAC_01669","NDEKODAF_02972","AMNKGELI_02996","PPAMLKML_02687","GGAGLIPG_02140","PNCEMHLO_02245","CNFKGNAC_01667","NDEKODAF_02974","AMNKGELI_02994","PPAMLKML_03326","GGAGLIPG_02138","PNCEMHLO_02247","CBGFCALD_00426","OEMCECNB_00819","KALPBOHI_02245","CNFKGNAC_01666","NDEKODAF_02975","AMNKGELI_02993","CNFKGNAC_01664","NDEKODAF_02977","AMNKGELI_02991","CNFKGNAC_01668","NDEKODAF_02973","AMNKGELI_02995","PPAMLKML_03327","GGAGLIPG_02137","PNCEMHLO_02248","PPAMLKML_03325","GGAGLIPG_02139","PNCEMHLO_02246","CNFKGNAC_01665","NDEKODAF_02976","AMNKGELI_02992","HIIAAINO_02670","MLIKFGOG_02247","LBMBHCAM_02410","CNFKGNAC_01672","NDEKODAF_02969","AMNKGELI_02999","PPAMLKML_02686","GGAGLIPG_02141","PNCEMHLO_02244","CBGFCALD_00427","OEMCECNB_00818","KALPBOHI_02244","ILDLLNBB_00035","MDOHONPH_00031","MHIPNNDE_01634","CBGFCALD_03565","BAIOBGAL_00164","NDEKODAF_03239","HIIAAINO_02669","MLIKFGOG_02248","LBMBHCAM_02409","ANLDKFMB_00911","BAIOBGAL_00301","ILDLLNBB_02340","ILDLLNBB_01987","MDOHONPH_01634","HGOBCNMM_02753","CHLMDKPE_02431","HGOBCNMM_02755","ANLDKFMB_03049","HIKCDMLK_01163","DHOHKKHL_00150","HIKCDMLK_01166","DHOHKKHL_00147","AMNKGELI_02986","PBGIHMLN_02738","ANBHONDD_00083","DPBEEGGP_00339","ANBHONDD_01019","DPBEEGGP_01609","ANBHONDD_01013","DPBEEGGP_01615","HGOBCNMM_00199","MHIPNNDE_00996","HIKCDMLK_01167","DHOHKKHL_00146","HIKCDMLK_01173","DHOHKKHL_00140","ANBHONDD_01005","DPBEEGGP_01623","ANBHONDD_01001","DPBEEGGP_01627","ANBHONDD_01017","DPBEEGGP_01611","ANBHONDD_01016","DPBEEGGP_01612","HIKCDMLK_01169","DHOHKKHL_00144","ILDLLNBB_02341","MDOHONPH_01635","ANBHONDD_01020","DPBEEGGP_01608","ANBHONDD_01007","DPBEEGGP_01621","ILDLLNBB_01986","MDOHONPH_01633","HIKCDMLK_01160","DHOHKKHL_00153","HGOBCNMM_02478","MHIPNNDE_01995","ANBHONDD_01002","DPBEEGGP_01626","ANBHONDD_01010","DPBEEGGP_01618","ANBHONDD_01003","DPBEEGGP_01625","HIKCDMLK_01162","DHOHKKHL_00151","HGOBCNMM_02477","MHIPNNDE_01994","ANBHONDD_01006","DPBEEGGP_01622","AMNKGELI_02988","PBGIHMLN_02740","HIKCDMLK_01170","DHOHKKHL_00143","HGOBCNMM_00195","MHIPNNDE_01000","ANBHONDD_01018","DPBEEGGP_01610","ANBHONDD_00082","DPBEEGGP_00338","HIKCDMLK_01161","DHOHKKHL_00152","ANBHONDD_01021","DPBEEGGP_01607","ANBHONDD_01012","DPBEEGGP_01616","HIKCDMLK_01168","DHOHKKHL_00145","ILDLLNBB_01289","MDOHONPH_01013","CNFKGNAC_00464","ADHCADEO_02367","ANBHONDD_00080","DPBEEGGP_00336","HGOBCNMM_02476","MHIPNNDE_01993","ANBHONDD_00073","DPBEEGGP_00329","ANBHONDD_00081","DPBEEGGP_00337","HGOBCNMM_00194","MHIPNNDE_01001","ANBHONDD_02194","DPBEEGGP_00250","HGOBCNMM_02475","MHIPNNDE_01992","HGOBCNMM_00196","MHIPNNDE_00999","ANBHONDD_01008","DPBEEGGP_01620","ANBHONDD_01004","DPBEEGGP_01624","CNFKGNAC_01491","DHOHKKHL_03045","ANBHONDD_01022","DPBEEGGP_01606","HGOBCNMM_00197","MHIPNNDE_00998","ANBHONDD_00075","DPBEEGGP_00331","ANBHONDD_01015","DPBEEGGP_01613","CNFKGNAC_00468","ADHCADEO_02371","ANBHONDD_01011","DPBEEGGP_01617","ILDLLNBB_01288","MDOHONPH_01014","HGOBCNMM_02474","MHIPNNDE_01991","CNFKGNAC_00465","ADHCADEO_02368","ANBHONDD_00079","DPBEEGGP_00335","HIKCDMLK_01172","DHOHKKHL_00141","HIKCDMLK_01171","DHOHKKHL_00142","HGOBCNMM_00200","MHIPNNDE_00995","ANBHONDD_01009","DPBEEGGP_01619","HIKCDMLK_01159","DHOHKKHL_00154","ANBHONDD_00074","DPBEEGGP_00330","HIKCDMLK_01165","DHOHKKHL_00148","HIKCDMLK_01164","DHOHKKHL_00149","CNFKGNAC_00466","ADHCADEO_02369","CNFKGNAC_00463","ADHCADEO_02366","AMNKGELI_02987","PBGIHMLN_02739","CNFKGNAC_00467","ADHCADEO_02370","HGOBCNMM_00198","MHIPNNDE_00997","HIKCDMLK_01158","DHOHKKHL_00155","ANBHONDD_01014","DPBEEGGP_01614","HIKCDMLK_01157","DHOHKKHL_00156","ANBHONDD_00076","DPBEEGGP_00332","HGOBCNMM_02758","ANLDKFMB_03046","ANBHONDD_00078","DPBEEGGP_00334","ANBHONDD_00077","DPBEEGGP_00333","AFDIGNOC_01410","AFDIGNOC_01409","ADHCADEO_01978","ADHCADEO_01981","AMNKGELI_02274","AFDIGNOC_01857","CNFKGNAC_01510","AFDIGNOC_02324","AMNKGELI_02273","AFDIGNOC_01856","AMNKGELI_01088","ADHCADEO_01968","ADHCADEO_01984","ADHCADEO_01470","CNFKGNAC_01511","PFJGCJHP_00944","ADHCADEO_01451","ADHCADEO_01457","PFJGCJHP_00940","ADHCADEO_01456","AFDIGNOC_02322","AMNKGELI_01083","AFDIGNOC_01861","PFJGCJHP_00943","AMNKGELI_01080","ADHCADEO_01966","ADHCADEO_01965","CNFKGNAC_01496","ANLDKFMB_00905","ADHCADEO_01467","ADHCADEO_01962","AMNKGELI_02275","ADHCADEO_01461","ADHCADEO_01977","ADHCADEO_01967","ADHCADEO_01455","AFDIGNOC_01859","ADHCADEO_01964","ADHCADEO_01974","ADHCADEO_01469","ADHCADEO_01465","ANLDKFMB_00906","AMNKGELI_02277","ADHCADEO_01985","CNFKGNAC_01509","ADHCADEO_01975","ADHCADEO_01980","ADHCADEO_01976","ADHCADEO_01982","ADHCADEO_01970","AMNKGELI_01082","ADHCADEO_01454","ADHCADEO_01406","ADHCADEO_01468","ADHCADEO_01983","ADHCADEO_01960","ADHCADEO_01409","PFJGCJHP_00942","ADHCADEO_01458","ADHCADEO_01452","AMNKGELI_01081","CNFKGNAC_01490","ALMPNPLM_02021","AFDIGNOC_02321","BAIOBGAL_02853","BAIOBGAL_00169","AMNKGELI_01087","CNFKGNAC_01515","ADHCADEO_01407","BAIOBGAL_00173","PFJGCJHP_00949","ADHCADEO_01453","BAIOBGAL_02852","AFDIGNOC_02326","AMNKGELI_02272","ANLDKFMB_00603","PFJGCJHP_00947","PFJGCJHP_00941","AFDIGNOC_02323","PFJGCJHP_00945","ADHCADEO_01466","BAIOBGAL_00176","BAIOBGAL_00175","ADHCADEO_01464","AMNKGELI_01084","CNFKGNAC_01512","ADHCADEO_01459","ADHCADEO_01979","PFJGCJHP_00948","ADHCADEO_01971","AFDIGNOC_01862","CNFKGNAC_01492","BAIOBGAL_00174","BAIOBGAL_00168","ADHCADEO_01972","AFDIGNOC_01858","AMNKGELI_01085","CNFKGNAC_01513","CNFKGNAC_01495","AMNKGELI_01079","ADHCADEO_01408","CNFKGNAC_01493","ADHCADEO_01961","AFDIGNOC_02325","AMNKGELI_01086","PFJGCJHP_00946","AMNKGELI_02279","ADHCADEO_01463","AMNKGELI_02276","ADHCADEO_01973","AMNKGELI_02271","AFDIGNOC_01863","ANLDKFMB_00909","PFJGCJHP_00939","ANLDKFMB_00604","BAIOBGAL_00165","ADHCADEO_01462","BAIOBGAL_00177","BAIOBGAL_02854","AMNKGELI_02278","ADHCADEO_01460","AFDIGNOC_01860","ADHCADEO_01969","CNFKGNAC_01497","CNFKGNAC_01514","PFJGCJHP_00952","ADHCADEO_01963","CNFKGNAC_01494","PFJGCJHP_00950","PFJGCJHP_00951"]

8_refound_74_stop
12_refound_106_stop
8_refound_81_stop
5_refound_48
7_refound_72
3_refound_26
3_refound_34
23_refound_214_stop
8_refound_75
27_refound_270_stop
'11_refound_105',
 '12_refound_110',
 '16_refound_147',
 '19_refound_171',
 '19_refound_174',
 '25_refound_236',
 '26_refound_259',
 '28_refound_279',
 '30_refound_301',
 '33_refound_311',
 '39_refound_339',
 '39_refound_340',
 '41_refound_347',
 '41_refound_348',
 '46_refound_364',
 12_refound_110',
 '19_refound_171',
 '25_refound_236',
 '28_refound_279',
 '33_refound_311',
 '39_refound_340',
 '41_refound_348
 19_refound_171',
 '28_refound_279',
 '39_refound_340',
 '28_refound_279'
#these are the bad boys

In [ ]:
inverted

In [ ]:
#get rid of these bad boys for later
for x in all_records:
    if 'refound' in x:
        all_records.remove(x)

In [ ]:
len(all_records)

In [ ]:
all_records

In [ ]:
plt.hist(insert_intergens['fb21'], bins=100)

In [ ]:
overlord = {}
for f in glob.glob("*.gff"):
    with open(f,'r') as ofh:
        genome_list = []
        for line in ofh:
            linea = []
            if line.startswith("##") or "CRISPR" in line:
                continue
            if line.startswith(">"):
                break
            line = line.strip()
            if line.split("\t")[8][3:17] in all_records:
                continue
            else:
                linea.append(line.split("\t")[8][3:17])
                linea.append(line.split("\t")[2])
                linea.append(line.split("\t")[3])
                linea.append(line.split("\t")[4])
                linea.append(line.split("\t")[0])
                genome_list.append(linea)
        overlord[f[:-4]] = genome_list
           # UG_123[line.split("\t")[8][3:17]] = linea
        #linea.append(line.split(""))

In [ ]:
gff_len = {}
#another way to calculate length
for genome in overlord.keys():
    glength = []
    
    for i,j in enumerate(overlord[genome]):
            glength.append(int(j[3]) - int(j[2]))
    gff_len[genome] = glength
        

In [ ]:
all_len_stats = {}
for items, values in gff_len.items():
    saved = []
    
    deets = pd.Series(values).describe()
    saved.append(deets['mean'])
    saved.append(deets['25%'])
    saved.append(deets['75%'])
    saved.append(deets['count'])
    all_len_stats[items] = saved

In [ ]:
all_len_stats

In [ ]:
gff_len = {}
#another way to calculate length
for genome in overlord.keys():
    glength = []
    first_rec = []
    for i,j in enumerate(overlord[genome]):
        if j[4] not in first_rec:
            first_rec.append(j[4])
            continue
        else:
            
            if int(j[0].split("_")[1]) - int(overlord[genome][i-1][0].split("_")[1]) >1:
                continue
            else:
                intergenic.append(int(j[3]) - int(j[2]))
    all_intergens[genome] = intergenic
    

In [ ]:
for u,h in enumerate(overlord['UG_123']):
    

In [ ]:
all_intergens = {}
for genome in overlord.keys():
    intergenic = []
    first_rec = []
    for i,j in enumerate(overlord[genome]):
        if j[4] not in first_rec:
            first_rec.append(j[4])
            continue
        else:
            
            if int(j[0].split("_")[1]) - int(overlord[genome][i-1][0].split("_")[1]) >1:
                continue
            else:
                intergenic.append(int(j[2]) - int(overlord[genome][i-1][3]))
    all_intergens[genome] = intergenic

In [ ]:
all_intergens

In [ ]:
intergens_stats = {}
for items, values in all_intergens.items():
    saved = []
    purged = [ele for ele in values if ele > 0]
    deets = pd.Series(purged).describe()
    saved.append(deets['mean'])
    saved.append(deets['25%'])
    saved.append(deets['75%'])
    saved.append(deets['count'])
    intergens_stats[items] = saved

In [ ]:
for key, value in intergens_stats.items():
    print("{}\t{}\t{}\t{}\t{}".format(key, value[0], value[1], value[2], value[3]), file=open("intergenic_overall_stats.tab",'a'))

In [ ]:
#draft version
    intergenic = []
    first_rec = []
#does strandedness matter? probably not but can do with 1 line
#UG_123_records = list(UG_123.keys())
    for i,j in enumerate(UG_123.keys()):
        if UG_123[j][3] not in UG_123_first_rec: #checks if its the first record of replicon, then adds to dictionary and skips (as there is no previous record)
            UG_123_first_rec.append(UG_123[j][3])
            continue
        else:
        
            if int(j.split("_")[1]) - int(UG_123_records[i-1].split("_")[1]) > 1: #checks if records are consecutive
                continue
            else:
                UG_123_intergenic.append(int(UG_123[j][1]) - int(UG_123[UG_123_records[i-1]][2]))
            
    UG_123_res = [ele for ele in UG_123_intergenic if ele > 0]

In [ ]:
UG_123_res = [ele for ele in UG_123_intergenic if ele > 0]


In [ ]:
deets = pd.Series(UG_123_res).describe()

In [ ]:
deets

## intergenics for insertion elements

In [ ]:
cd /media/tian/Backup_Protected/working/halo_pg/latest_intersect_cog/ffn/20201109_isolates/fasta_fnas/virus/PROKKA_10292021/clusts/hmmers/hmmerout/hits/cripsr/panaroo/reannotate/single_gene/gffs/results/

In [ ]:
overlady

In [ ]:
overlady_eled

In [ ]:
over_elements = {}
for genomes, items in overlady_eled.items():
    
    for recs in items:
        print("{},{},{},{},{},{}".format(recs[0],recs[1],recs[2],recs[3],recs[4],recs[5]), file=open("over_elements.tab",'a'))
        

In [ ]:
overlady_eled = {} #modified by element
for genomes, infos in overlady.items():
    genome_savefile = []
    for rec in infos:
        king = rec
        king.append(all_locus_primes[rec[0]])
        genome_savefile.append(king)
    overlady_eled[genomes] = genome_savefile

In [ ]:
cd /media/tian/Backup_Protected/working/halo_pg/latest_intersect_cog/ffn/20201109_isolates/fasta_fnas/virus/PROKKA_10292021/clusts/hmmers/hmmerout/hits/cripsr/panaroo/reannotate/single_gene/gffs/results

In [ ]:
element_names = []
with open("element_names.txt",'r') as ofh:
    for line in ofh:
        line = line.strip()
        element_names.append(line)
element_set = list(set(element_names))

In [ ]:
len(element_set)

In [ ]:
over_elements ={}
#element_names = []
for elements in element_set:
    element_save = []
    with open("over_elements.tab",'r') as ofh:
        
        for line in ofh:
            #line_save = []
            line = line.strip()
            if line.split(",")[5] == elements:
                line_save = line.split(",")
                element_save.append(line_save)
        over_elements[elements] = element_save
        

In [ ]:
all_locus_primes = {}
for record in all_records:
    with open("insert_only_presence_absence.csv",'r') as ofh:
        for line in ofh:
            if record in line:
                all_locus_primes[record] = inverted[line.split(",")[0]]
        

In [ ]:
overlady = {}
for f in glob.glob("*.gff"):
    with open(f,'r') as ofh:
        genome_list = []
        for line in ofh:
            linea = []
            if line.startswith("##") or "CRISPR" in line:
                continue
            if line.startswith(">"):
                break
            line = line.strip()
            if line.split("\t")[8][3:17] in all_records:
                linea.append(line.split("\t")[8][3:17])
                linea.append(line.split("\t")[2])
                linea.append(line.split("\t")[3])
                linea.append(line.split("\t")[4])
                linea.append(line.split("\t")[0])
                genome_list.append(linea)
            else:
                continue
                
        overlady[f[:-4]] = genome_list

In [ ]:
insert_intergens = {}
for genome in overlady.keys():
    intergenic = []
    first_rec = []
    for i,j in enumerate(overlady[genome]):
        if j[4] not in first_rec:
            first_rec.append(j[4])
            continue
        else:
            
            if int(j[0].split("_")[1]) - int(overlady[genome][i-1][0].split("_")[1]) >1:
                continue
            else:
                intergenic.append(int(j[2]) - int(overlady[genome][i-1][3]))
    insert_intergens[genome] = intergenic

### New stuff

In [ ]:
element_record = [] #says name of the orf in gff file
gene_ids = [] #says what the name of the orf is in panaroo
island_ids = [] #says what island any record belongs to

with open("intergenicdistancesredux/all_recs_list_islands.txt",'r') as ofh:
    for line in ofh:
        element_record.append(line.strip())
with open("intergenicdistancesredux/all_gene_ids_list.txt",'r') as ofh:
    for line in ofh:
        gene_ids.append(line.strip())  
        
with open("intergenicdistancesredux/all_island_ids_list.txt",'r') as ofh:
    for line in ofh:
        island_ids.append(line.strip())

In [ ]:
gene_ids[2761]

In [ ]:
record_keeper = {}
for f in glob.glob("*.gff"):
    with open(f,'r') as ofh:
        genome_list = []
        for line in ofh:
            linea = []
            if line.startswith("##") or "CRISPR" in line:
                continue
            if line.startswith(">"):
                break
            line = line.strip()
            if line.split("\t")[8][3:17] in element_record:
                linea.append(line.split("\t")[8][3:17])
                linea.append(line.split("\t")[2])
                linea.append(line.split("\t")[3])
                linea.append(line.split("\t")[4])
                linea.append(line.split("\t")[0])
                linea.append(island_ids[element_record.index(line.split("\t")[8][3:17])])
                genome_list.append(linea)
            else:
                continue
                
        record_keeper[f[:-4]] = genome_list

In [ ]:
record_keeper['LD3']

In [ ]:
insert_intergens = {}
for genome in record_keeper.keys():
    intergenic = []
    island_hold = []
    first_rec = []
    #distance = 0
    for i,j in enumerate(record_keeper[genome]):
        distance = 0
        if j[4] not in first_rec:
            first_rec.append(j[4])
            continue
        else:
            
            if int(j[0].split("_")[1]) - int(record_keeper[genome][i-1][0].split("_")[1]) >1:
                continue
            else:
                distance += (int(j[2]) - int(record_keeper[genome][i-1][3]))
                if j[5] in insert_intergens.keys():
                    insert_intergens[j[5]].append(distance) 
                else:
                    insert_intergens[j[5]] = [distance]
                
                
                #island_hold.append(j[5])
        
    #insert_intergens[island_ids] = intergenic
    #insert_intergens[genome] = [intergenic, island_hold]
    
    


In [ ]:
insert_intergens

In [ ]:
import pandas as pd

In [ ]:
by_ele_intergens_stats = {}
for items, values in insert_intergens.items():
    saved =[]
    #purged = [ele for ele in values if ele > 0]
    seriez = pd.Series(values).describe()
    saved.append(seriez['mean'])
    saved.append(seriez['25%'])
    saved.append(seriez['75%'])
    saved.append(seriez['count'])
    by_ele_intergens_stats[items] = saved

In [ ]:
by_ele_intergens_stats

In [ ]:
for key, value in by_ele_intergens_stats.items():
    print("{}\t{}\t{}\t{}\t{}".format(key, value[0], value[1], value[2], value[3]), file=open("redux_interstats.tab",'a'))

In [ ]:
#modified for per element use for over_elements
element_intergens = {}
for genome in over_elements.keys():
    intergenic = []
    first_rec = []
    for i,j in enumerate(over_elements[genome]):
        if j[5] not in first_rec: #need this to skip the first one
            first_rec.append(j[5])
            continue
        else:
            
            if int(j[0].split("_")[1]) - int(over_elements[genome][i-1][0].split("_")[1]) >1:
                continue
            else:
                if j[4] == over_elements[genome][i-1][4]:
                    intergenic.append(int(j[2]) - int(over_elements[genome][i-1][3]))
    element_intergens[genome] = intergenic

In [ ]:
over_elements['group_6203_12']

In [ ]:
elle_dict = {}
with open("intergenicdistancesredux/all_element_record_dict.txt",'r') as ofh:
    for line in ofh:
        if line.split("+")[0] in elle_dict.keys():
            elle_dict[line.split("+")[0]].append(line.split("+")[1].strip())
        else:
            elle_dict[line.split("+")[0]] = [line.split("+")[1].strip()]

In [ ]:
for kays, vals in elle_dict.items():
    print(kays + "\t" + str(len(vals)))

In [ ]:
element_intergens['mch_2|0']

In [ ]:
by_ele_intergens_stats = {}
for items, values in element_intergens.items():
    saved =[]
    #purged = [ele for ele in values if ele > 0]
    seriez = pd.Series(values).describe()
    saved.append(seriez['mean'])
    saved.append(seriez['25%'])
    saved.append(seriez['75%'])
    saved.append(seriez['count'])
    by_ele_intergens_stats[items] = saved

In [ ]:
for key, value in by_ele_intergens_stats.items():
    print("{}\t{}\t{}\t{}\t{}".format(key, value[0], value[1], value[2], value[3]), file=open("by_ele_intergens_stats.tab",'a'))

In [ ]:
ins_intergens_stats = {}
for items, values in insert_intergens.items():
    saved = []
    purged = [ele for ele in values if ele > 0]
    deets = pd.Series(purged).describe()
    saved.append(deets['mean'])
    saved.append(deets['25%'])
    saved.append(deets['75%'])
    saved.append(deets['count'])
    ins_intergens_stats[items] = saved

In [ ]:
for key, value in ins_intergens_stats.items():
    print("{}\t{}\t{}\t{}\t{}".format(key, value[0], value[1], value[2], value[3]), file=open("intergenic_overall_stats.tab",'a'))

In [ ]:
from scipy.stats import ks_2samp
ks_2samp(insert_intergens['C191'], all_intergens['C191'])

In [ ]:
insert_intergens

### Find Tns from GFFs
1. allrecords has all the locus tags

In [ ]:
len(element_record)

In [ ]:
overlord = {}
for f in glob.glob("*.gff"):
    with open(f,'r') as ofh:
        genome_list = []
        for line in ofh:
            linea = []
            if line.startswith("##") or "CRISPR" in line:
                continue
            if line.startswith(">"):
                break
            line = line.strip()
            #if line.split("\t")[8][3:17] in all_records:
            #    continue
            linea.append(line.split("\t")[8][3:17])
            if "transposase" in line:
                linea.append('TnE')
            else:
                linea.append(line.split("\t")[2])
            linea.append(line.split("\t")[3])
            linea.append(line.split("\t")[4])
            linea.append(line.split("\t")[0])
            genome_list.append(linea)
        overlord[f[:-4]] = genome_list

In [ ]:
overlord

In [ ]:
for keys, values in overlord.items():
    for i,j in enumerate(values):
        if j[0] in element_record:
            for scroll in henlo:
                try:
                    if values[i+scroll][1] == "TnE":
                        if scroll == 0:
                            print(j[0] + "\t" + j[4]+ "\tinside\t" + island_ids[element_record.index(j[0])] , file=open("TnE_new_annotate3.tab",'a'))
                        else:
                            if values[i+scroll][0] in element_record:
                                print(j[0] + "\t" + j[4] + "\t" + values[i+scroll][0] + "\t" + island_ids[element_record.index(j[0])], file=open("TnE_new_annotate3.tab",'a'))
                            else:
                                print(j[0] + "\t" + j[4] + "\t" + values[i+scroll][0] + "\t" + island_ids[element_record.index(j[0])], file=open("TnE_new_annotate3.tab",'a'))
                except IndexError:
                    print(IndexError, file=open("TnE_new_annotate3.tab",'a'))
                    
                    ###i want to distinguish if the TnE is outside or inside the element

In [ ]:
len(island_ids)

In [ ]:
ltag_eles = {}
for keys,vals in over_elements.items():
    for names in vals:
        ltag_eles[names[0]] = keys
ltag_eles

In [ ]:
henlo = range(-5,5)
for x in henlo:
    print(x)

## same thing but with tRNAs new stuff

In [ ]:
for keys, values in overlord.items():
    for i,j in enumerate(values):
        if j[0] in element_record:
            for scroll in henlo:
                try:
                    if values[i+scroll][1] == "tRNA":
                        if scroll == 0:
                            print(j[0] + "\t" + j[4]+ "\tinside\t" + island_ids[element_record.index(j[0])] , file=open("tRNA_new_annotate.tab",'a'))
                        else:
                            if values[i+scroll][0] in element_record:
                                print(j[0] + "\t" + j[4] + "\t" + values[i+scroll][0] + "\t" + island_ids[element_record.index(j[0])] + "\twarra", file=open("tRNA_new_annotate.tab",'a'))
                            else:
                                print(j[0] + "\t" + j[4] + "\t" + values[i+scroll][0] + "\t" + island_ids[element_record.index(j[0])], file=open("tRNA_new_annotate.tab",'a'))
                except IndexError:
                    print(IndexError, file=open("tRNA_new_annotate.tab",'a'))

In [ ]:
duplicated = []
with open("TnE_annotate.tab",'r') as ofh:
    for line in ofh:
        if "inside" in line:
            print(line.strip(),file=open("TnE_derepped.tab",'a'))
        if "outside" in line:
            if line.split("\t")[2] in duplicated:
                continue
            else:
                duplicated.append(line.split("\t")[2])
                print(line.strip(),file=open("TnE_derepped.tab",'a'))
        

In [ ]:
element: {genome:stats}

In [ ]:
pa_element_tne = {}
for f in over_elements.keys():
    #ele_save =""
    with open("TnE_derepped.tab",'r') as ofh:
        trues = []
        outs = []
        for line in ofh:
            if f in line:
                trues.append(line.split("\t")[0])
            if "outside" in line:
                if ltag_eles[line.split("\t")[0]] == f:
                    outs.append(line.split("\t")[0])
    pa_element_tne[f] = {"true":trues, "out":outs}
                    
            

In [ ]:
for element, various in pa_element_tne.items():
    if len(various['true']) == 0:
        if len(various['out']) == 0:
            print(element +"\t" + "empty",file=open("tne_encodings.tab",'a'))
        if len(various['out']) > 0:
            print(element +"\t" + "out_only",file=open("tne_encodings.tab",'a'))
    if len(various['true']) > 0: #insides
        if len(various['out']) == 0:   
            print(element +"\t" + "inside_only",file=open("tne_encodings.tab",'a'))
        if len(various['out']) > 0:
            print(element +"\t" + "both",file=open("tne_encodings.tab",'a'))

In [ ]:
pa_element_tne

#### Find if the elements are shuffled or not

In [ ]:
#should probably check contiguity
element_shuffles ={}
for elements in over_elements.keys():
    element_record = []
    for record in over_elements[elements]:
        element_record.append(record[0].split("_")[0])
    element_shuffles[elements] = (dict(Counter(element_record)))
    

In [ ]:
for keys, vals in element_shuffles.items():
    try:
        print(keys + "\t" + str(stdev(vals.values())),file=open("std_dev_ele_distros.tab",'a'))
    except:
                    print(keys + "\t0.0",file=open("std_dev_ele_distros.tab",'a'))

In [ ]:
element_shuffles['group_399_12']

In [ ]:
from collections import Counter
from statistics import mean,stdev

## All archaea survey

In [ ]:
cd /media/tian/Backup_Protected/working/halo_pg/latest_intersect_cog/ffn/20201109_isolates/fasta_fnas/virus/PROKKA_10292021/clusts/hmmers/hmmerout/hits/cripsr/panaroo/reannotate/single_gene/gffs/other_archaea/viral_assemblies_genome_gff

In [ ]:
overarchv = {}
for f in glob.glob("*.gff"):
    with open(f,'r') as ofh:
        genome_list = []
        for line in ofh:
            linea = []
            if line.startswith(">"):
                break
            line = line.strip()
            if "cds-" in line:
                    print(line.split("\t")[8].split(";")[0], file=open("id_chek.txt",'a'))
            #else:
                    linea.append(line.split("\t")[8].split(";")[0][7:-2])
                    linea.append(line.split("\t")[2])
                    linea.append(line.split("\t")[3])
                    linea.append(line.split("\t")[4])
                    linea.append(line.split("\t")[0])
                    genome_list.append(linea)
        overarchv[f[:-4]] = genome_list

In [ ]:
overarchv #YP_, NP_, regular

In [ ]:
ins_fams['adhT|6']

In [ ]:
circle = pycircos.Gcircle() 
with open("example_data_chromosome_general.csv") as f:
    f.readline()
    for line in f:
        line   = line.rstrip().split(",") 
        name   = line[0]
        length = int(line[-1]) 
        arc    = pycircos.Garc(arc_id=name, size=length, interspace=3, raxis_range=(950,1000), labelposition=60, label_visible=True)
        circle.add_garc(arc) 

In [ ]:
arc2 = pycircos.Garc(arc_id="two", size=length, interspace=3, raxis_range=(950,1000), labelposition=60, label_visible=True)
#circle.add_garc(arc) 

In [ ]:
circle.set_garcs() 

In [ ]:
circle.figure

In [ ]:
import collections
color_dict   = {"gneg":"#FFFFFF00", "gpos25":"#EEEEEE", "gpos50":"#BBBBBB", "gpos75":"#777777", "gpos100":"#000000", "gvar":"#FFFFFF00", "stalk":"#C01E27", 
               "acen":"#D82322"}

arcdata_dict = collections.defaultdict(dict)
with open("example_data_chromosome_cytoband.csv") as f:
    f.readline()
    for line in f:
        line  = line.rstrip().split(",")
        name  = line[0]     
        start = int(line[1])-1 
        width = int(line[2])-(int(line[1])-1) 
        if name not in arcdata_dict:
            arcdata_dict[name]["positions"] = []
            arcdata_dict[name]["widths"]    = [] 
            arcdata_dict[name]["colors"]    = [] 
        arcdata_dict[name]["positions"].append(start) 
        arcdata_dict[name]["widths"].append(width)
        arcdata_dict[name]["colors"].append(color_dict[line[-1]])

for key in arcdata_dict:
    circle.barplot(key, data=[1]*len(arcdata_dict[key]["positions"]), positions=arcdata_dict[key]["positions"], 
                   width=arcdata_dict[key]["widths"], raxis_range=[950,1000], facecolor=arcdata_dict[key]["colors"]) 

In [ ]:
cd D:\bioinfo\halo_pg\insertscanner

In [ ]:
import collections
circle2 = pycircos.Gcircle() 
with open("ezzemoule_circos.txt") as f:
    f.readline()
    for line in f:
        line   = line.rstrip().split(",") 
        name   = line[0]
        length = int(line[-1]) 
        arc    = pycircos.Garc(arc_id=name, size=length, interspace=3, raxis_range=(950,1000), labelposition=60, label_visible=True)
        circle2.add_garc(arc) 

In [ ]:
#circle2.set_garcs() 
#min width is 700? 
circle2.figure

In [ ]:
import date

In [ ]:
color_dict   = {"gneg":"#FFFFFF00", "gpos25":"#EEEEEE", "gpos50":"#BBBBBB", "gpos75":"#777777", "gpos100":"#000000", "gvar":"#FFFFFF00", "stalk":"#C01E27", 
               "acen":"#D82322"}

In [ ]:
arcdata_dict2 = collections.defaultdict(dict)
with open("ezzemoule_circos_cytoband.txt") as f:
    f.readline()
    for line in f:
        line  = line.rstrip().split(",")
        name  = line[0]     
        start = int(line[1])-1 
        width = int(line[2])-(int(line[1])-1) 
        if name not in arcdata_dict2:
            arcdata_dict2[name]["positions"] = []
            arcdata_dict2[name]["widths"]    = [] 
            arcdata_dict2[name]["colors"]    = [] 
        arcdata_dict2[name]["positions"].append(start) 
        arcdata_dict2[name]["widths"].append(width)
        arcdata_dict2[name]["colors"].append(color_dict[line[-1]])

for key in arcdata_dict2:
    circle2.barplot(key, data=[1]*len(arcdata_dict2[key]["positions"]), positions=arcdata_dict2[key]["positions"], 
                   width=arcdata_dict2[key]["widths"], raxis_range=[950,1000], facecolor=arcdata_dict2[key]["colors"]) 

In [ ]:
records = list(SeqIO.parse("fb21.gbk", "genbank"))

#need to change garc# for every seq record
garc3   = pycircos.Garc(arc_id="king", record=records[2], interspace=0, linewidth=0, 
              facecolor="#FFFFFF00", raxis_range=(0,10), 
              label="Hrr. ezzemoulense fb21", label_visible=True)

In [ ]:
import copy
skews = garc.calc_nnskew(n1="G", n2="C", window_size=1000, step_size=100)  
positive_skews=copy.deepcopy(skews)
positive_skews[skews<0]=0
negative_skews=copy.deepcopy(skews)
negative_skews[skews>=0]=0
circle2.fillplot("chromosome", positive_skews, rlim=(min(skews),max(skews)), base_value=0, raxis_range=(400,700), facecolor="r")
circle2.fillplot("chromosome", negative_skews, rlim=(min(skews),max(skews)), base_value=0, raxis_range=(400,700), facecolor="b")

In [ ]:
#megaplasmid
import copy
skews2 = garc2.calc_nnskew(n1="G", n2="C", window_size=1000, step_size=100)  
positive_skews2=copy.deepcopy(skews2)
positive_skews2[skews2<0]=0
negative_skews2=copy.deepcopy(skews2)
negative_skews2[skews2>=0]=0
circle2.fillplot("megaplasmid", positive_skews2, rlim=(min(skews2),max(skews2)), base_value=0, raxis_range=(400,700), facecolor="r")
circle2.fillplot("megaplasmid", negative_skews2, rlim=(min(skews2),max(skews2)), base_value=0, raxis_range=(400,700), facecolor="b")

In [ ]:
#plasmid
skews3 = garc3.calc_nnskew(n1="G", n2="C", window_size=1000, step_size=100)  
positive_skews3=copy.deepcopy(skews3)
positive_skews3[skews3<0]=0
negative_skews3=copy.deepcopy(skews3)
negative_skews3[skews3>=0]=0
circle2.fillplot("plasmid", positive_skews3, rlim=(min(skews3),max(skews3)), base_value=0, raxis_range=(400,700), facecolor="r")
circle2.fillplot("plasmid", negative_skews3, rlim=(min(skews3),max(skews3)), base_value=0, raxis_range=(400,700), facecolor="b")

In [ ]:
circle2.fillplot("chromosome", positive_skews, rlim=(min(skews),max(skews)), base_value=0, raxis_range=(400,700), facecolor="r")
circle2.fillplot("chromosome", negative_skews, rlim=(min(skews),max(skews)), base_value=0, raxis_range=(400,700), facecolor="b")

In [ ]:
circle2.save("plot1.pdf")